# 1. Introducción e Importación de librerías y datos
### En este cuaderno vamos a practicar con diferentes algoritmos de clasificación:
* SVM(Support Vector Machine). En español: Máquina de vectores de soporte 
* Regresión Logística 
* Árboles de Decisión 
* Bosques aleatorios 
* kNN, o el algoritmo de k vecino más cercano
### Para los siguientes datasets:
* Phishing Urls
* Calidad del Vino(Si es 5 o menor es de mala calidad y si es mayor de 5 es buena calidad)
* Diagnostico de Cancer de Mama
* Temperatura en España (dias de lluvia de un mes y lugar concreto)

### Para cada algoritmo vamos a:
* Explicar en qué consiste el algoritmo
* Explicar los resultados en cada dataset en base a las métricas iniciales sin incluir hiperparámetros
* Ajuste de hiperparamétros en cada uno de los ejemplos (3 casos por cada hiperparámetros y comparar y explicar cuál es el mejor resultado) y explicar en que consisten cada uno de ellos y cuando usarlos.
* Analizar los resultados de cada dataset indicando cual es mejor algoritmo en cada caso
* Incluir dos nuevas métricas en cada algoritmo, explicar la métrica y analizar los resultados.

### Por cada dataset vamos a:
* Tratar los datos(limpieza, encoding y escalado) cuando sea necesario
* Crear uno de ejemplo con sólo 10 registros, con 5 para un caso y otros 5 para otro. Usar los algoritmos con estos dataset, con la configuración de hiperparametros que hayais obtenido mejor resultado e indicar en cada caso que resultado habéis obtenido.

### Imports necesarios

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import  OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, average_precision_score, explained_variance_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

### Importamos los datasets

In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# PHISHING URLS
phishing_path = "phishing.csv"  # Ruta donde deseas guardar el dataset
# Cargar la última versión del dataset
dfphishing = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,  # Formato que queremos (Pandas)
  "eswarchandt/phishing-website-detector",  # Nombre del dataset
  phishing_path
)
# Mostrar las primeras 5 filas del dataset vino
display("DATASET PHISHING",dfphishing.head(5))

# CALIDAD DEL VINO
vino_path = "WineQT.csv"  # Ruta donde deseas guardar el dataset
# Cargar la última versión del dataset
dfvino = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,  # Formato que queremos (Pandas)
  "yasserh/wine-quality-dataset",  # Nombre del dataset
  vino_path
)
# Mostrar las primeras 5 filas del dataset vino
display("DATASET VINO",dfvino.head(5))

# CANCER DE MAMA
cancer_path = "brca.csv"  # Ruta donde deseas guardar el dataset
# Cargar la última versión del dataset
dfcancer = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,  # Formato que queremos (Pandas)
  "utkarshx27/breast-cancer-wisconsin-diagnostic-dataset",  # Nombre del dataset
  cancer_path
)
# Mostrar las primeras 5 filas del dataset cancer
display("DATASET CANCER",dfcancer.head(5))


# TEMPERATURA ESPAÑA
temp_path = "monthly_temperature_spain (1996-2023).csv"  # Ruta donde deseas guardar el dataset
# Cargar la última versión del dataset
dftemp = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,  # Formato que queremos (Pandas)
  "alexgczs/monthly-temperature-in-spain-1996-2023",  # Nombre del dataset
  temp_path
)
# Mostrar las primeras 5 filas del dataset temp españa
display("TEMPERATURA ESPAÑA",dftemp.head(5))

c:\Users\domid\Desktop\210MIA\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\domid\AppData\Local\Temp\ipykernel_7196\2763246943.py:7: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  dfphishing = kagglehub.load_dataset(


'DATASET PHISHING'

,Index,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
0,0,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,2,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,3,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,4,-1,0,-1,1,-1,-1,1,1,-1,...,1,1,1,1,1,-1,1,-1,-1,1


C:\Users\domid\AppData\Local\Temp\ipykernel_7196\2763246943.py:18: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  dfvino = kagglehub.load_dataset(


'DATASET VINO'

...

'DATASET CANCER'

,Unnamed: 0,x.radius_mean,x.texture_mean,x.perimeter_mean,x.area_mean,x.smoothness_mean,x.compactness_mean,x.concavity_mean,x.concave_pts_mean,x.symmetry_mean,...,x.texture_worst,x.perimeter_worst,x.area_worst,x.smoothness_worst,x.compactness_worst,x.concavity_worst,x.concave_pts_worst,x.symmetry_worst,x.fractal_dim_worst,y
0,1,13.540,14.36,87.46,566.3,0.09779,0.08129,0.06664,0.047810,0.1885,...,19.26,99.70,711.2,0.14400,0.17730,0.23900,0.12880,0.2977,0.07259,B
1,2,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.031100,0.1967,...,20.49,96.09,630.5,0.13120,0.27760,0.18900,0.07283,0.3184,0.08183,B
2,3,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.020760,0.1815,...,15.66,65.13,314.9,0.13240,0.11480,0.08867,0.06227,0.2450,0.07773,B
3,4,13.030,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.029230,0.1467,...,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169,B
4,5,8.196,16.84,51.71,201.9,0.08600,0.05943,0.01588,0.005917,0.1769,...,21.96,57.26,242.2,0.12970,0.13570,0.06880,0.02564,0.3105,0.07409,B


C:\Users\domid\AppData\Local\Temp\ipykernel_7196\2763246943.py:41: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  dftemp = kagglehub.load_dataset(


'TEMPERATURA ESPAÑA'

,month,avg_temp,max_temp,min_temp,rain_days,rain_accum,avg_wind,place
0,Diciembre,10ºC,15ºC,6ºC,4 Días,23 mm,10,Murcia
1,Noviembre,13ºC,19ºC,9ºC,5 Días,27 mm,10,Murcia
2,Octubre,18ºC,24ºC,14ºC,5 Días,27 mm,8,Murcia
3,Septiembre,22ºC,29ºC,17ºC,3 Días,15 mm,8,Murcia
4,Agosto,25ºC,32ºC,19ºC,1 Días,3 mm,9,Murcia


# 2. Limpieza de Datasets y resultados sin incluir hiperparámetros
###  Preparación previa a la aplicación de los diferentes Algoritmos de Clasificación.
### Prueba del modelo sin incluir los hiperparámetros para comparar en futuros apartados.

## 2.1 TEMPERATURA ESPAÑA

### Info de las columnas:
| **Variable**         | **Descripción**                                     |
|----------------------|-----------------------------------------------------|
| **rain_days**        | Días de lluvia en un mes                           |
| **month**            | Mes del año                                        |
| **max_temp**         | Temperatura máxima durante el mes                  |
| **min_temp**         | Temperatura mínima durante el mes                  |
| **rain_accum**       | Acumulación de lluvia en el mes (en mm)            |
| **avg_wind**         | Velocidad media del viento durante el mes          |
| **avg_temp**         | Temperatura media durante el mes                   |
| **place**            | Lugar de la medición de los datos                  |

In [3]:
display(dftemp.head(5))

,month,avg_temp,max_temp,min_temp,rain_days,rain_accum,avg_wind,place
0,Diciembre,10ºC,15ºC,6ºC,4 Días,23 mm,10,Murcia
1,Noviembre,13ºC,19ºC,9ºC,5 Días,27 mm,10,Murcia
2,Octubre,18ºC,24ºC,14ºC,5 Días,27 mm,8,Murcia
3,Septiembre,22ºC,29ºC,17ºC,3 Días,15 mm,8,Murcia
4,Agosto,25ºC,32ºC,19ºC,1 Días,3 mm,9,Murcia


### Comprobamos los nulos

In [4]:
# Comprobar los valores nulos por columna
nulos_por_columna = dftemp.isnull().sum()

# Mostrar el resultado
print("Número de valores nulos por columna:")
print(nulos_por_columna)

Número de valores nulos por columna:
month         0
avg_temp      0
max_temp      0
min_temp      0
rain_days     0
rain_accum    0
avg_wind      0
place         0
dtype: int64


### No tenemos nulos, pero podemos descartar es avg_temp porque se podrá calcular con las demás, las otras serán importante para que nuestro modelo pueda predecir cuántos dias de lluvia habrá en el mes deseado

In [5]:
dftemp = dftemp.drop("avg_temp", axis=1)

### Antes del encoding, aunque no sea técnicamente obligatorio vamos a convertir a valores numéricos las columnas que en realidad son valores numericos (todas menos mes), aunque también se podria pasar a numérico pero la dejaremos así para que lo haga el Label Encoding mejor

In [6]:
# Convertir las columnas a formato numérico solo si no están ya en el formato adecuado

columns_to_convert = ["max_temp", "min_temp", "rain_days", "rain_accum"]

for col in columns_to_convert:
    try:
        # Si la columna ya es numérica, el código no hará nada
        if "temp" in col:  # Eliminar 'ºC' de las columnas de temperatura
            if dftemp[col].dtype != float:  # Verificamos si la columna no es de tipo float
                dftemp[col] = dftemp[col].str.replace("ºC", "").astype(float)
        elif "rain_days" in col:  # Eliminar ' Días' de la columna de días de lluvia
            if dftemp[col].dtype != int:  # Verificamos si la columna no es de tipo int
                dftemp[col] = dftemp[col].str.replace(" Días", "").astype(int)
        elif "rain_accum" in col:  # Eliminar ' mm' de la columna de acumulación de lluvia
            if dftemp[col].dtype != float:  # Verificamos si la columna no es de tipo float
                dftemp[col] = dftemp[col].str.replace(" mm", "").astype(float)
    except Exception as e:
        print(f"Error al convertir la columna {col}: {e}")

# Verificar el resultado
display(dftemp.head())
dftemp.info()

,month,max_temp,min_temp,rain_days,rain_accum,avg_wind,place
0,Diciembre,15.0,6.0,4,23.0,10,Murcia
1,Noviembre,19.0,9.0,5,27.0,10,Murcia
2,Octubre,24.0,14.0,5,27.0,8,Murcia
3,Septiembre,29.0,17.0,3,15.0,8,Murcia
4,Agosto,32.0,19.0,1,3.0,9,Murcia


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   month       480 non-null    object 
 1   max_temp    480 non-null    float64
 2   min_temp    480 non-null    float64
 3   rain_days   480 non-null    int64  
 4   rain_accum  480 non-null    float64
 5   avg_wind    480 non-null    int64  
 6   place       480 non-null    object 
dtypes: float64(3), int64(2), object(2)
memory usage: 26.4+ KB


### Para pasar las columnas no numéricas, antes de empezar con las técnicas de escalado de características, vamos a usar One - hot encoding

In [7]:
# Crear una copia del DataFrame original para no modificar el original
dfesp_onehot = dftemp.copy()

# Filtrar las columnas categóricas
columnas_categoricas_temp = ["month", "place"]

# Instancia de OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')

# Aplicar One-Hot Encoding a las columnas seleccionadas
columnas_encoded_temp = one_hot_encoder.fit_transform(dfesp_onehot[columnas_categoricas_temp])

# Convertir el resultado a un DataFrame
columnas_encoded_df_temp = pd.DataFrame(columnas_encoded_temp, columns=one_hot_encoder.get_feature_names_out(columnas_categoricas_temp))

# Ahora concatenamos las columnas numéricas (las que no son categóricas) al DataFrame resultante
columnas_numéricas = dfesp_onehot.drop(columns=columnas_categoricas_temp)
columnas_encoded_df_temp = pd.concat([columnas_encoded_df_temp, columnas_numéricas], axis=1)

# Mostrar el resultado
display(columnas_encoded_df_temp.head())

# Número de columnas antes y después del One-Hot Encoding
print("Número de columnas antes del One-Hot Encoding:", dfesp_onehot.shape[1])
print("Número de columnas después del One-Hot Encoding:", columnas_encoded_df_temp.shape[1])
#columnas_encoded_df_temp.info()

,month_Agosto,month_Diciembre,month_Enero,month_Febrero,month_Julio,month_Junio,month_Marzo,month_Mayo,month_Noviembre,month_Octubre,...,place_Valladolid,place_Zamora,place_Zaragoza,place_Álava,place_Ávila,max_temp,min_temp,rain_days,rain_accum,avg_wind
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,15.0,6.0,4,23.0,10
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19.0,9.0,5,27.0,10
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,24.0,14.0,5,27.0,8
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,29.0,17.0,3,15.0,8
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,32.0,19.0,1,3.0,9


Número de columnas antes del One-Hot Encoding: 7
Número de columnas después del One-Hot Encoding: 55


### Ahora vamos a normalizar los datos para dejarlo todo listo para cuando apliquemos los diferentes algoritmos de clasificación, en este caso vamos a usar Normalización (Min-Max) porque es la opción más adecuada, ya que ajusta las escalas sin alterar la forma de los datos y mantiene la comparabilidad entre las variables. Además, no afecta a los valores de rain_days.

In [8]:
# Copia del dataframe para aplicar la normalización
df_esplimpio = columnas_encoded_df_temp.copy()

# Lista de las columnas numéricas para las que haremos la normalización
columnas_numericas = df_esplimpio.select_dtypes(include=['float64', 'int64']).columns

# Excluimos la columna objetivo (quality)
columnas_numericas = columnas_numericas.drop('rain_days')

# Aplicamos MinMaxScaler de sklearn para normalizar las columnas numéricas
min_max = MinMaxScaler()

# Aplicamos el ajuste y la transformación a las columnas numéricas
dfesp_limpio = pd.DataFrame(min_max.fit_transform(df_esplimpio[columnas_numericas]), columns=columnas_numericas)

# Copiamos la columna "quality" (sin normalizar) a la tabla final
dfesp_limpio['rain_days'] = df_esplimpio['rain_days']

display(dfesp_limpio.head(5))


,month_Agosto,month_Diciembre,month_Enero,month_Febrero,month_Julio,month_Junio,month_Marzo,month_Mayo,month_Noviembre,month_Octubre,...,place_Valladolid,place_Zamora,place_Zaragoza,place_Álava,place_Ávila,max_temp,min_temp,rain_accum,avg_wind,rain_days
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.28125,0.333333,0.106481,0.285714,4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.40625,0.458333,0.125000,0.285714,5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.56250,0.666667,0.125000,0.142857,5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.71875,0.791667,0.069444,0.142857,3
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.81250,0.875000,0.013889,0.214286,1


## 2.2 PHISHING URLS

### Info de las columnas:
| **Variable**             | **Descripción**                                 | **Posibles Valores**                |
|--------------------------|-------------------------------------------------|-------------------------------------|
| **Index**                | Número de índice                                | Número de índice (0, 1, 2, ...)     |
| **UsingIP**              | Uso de IP (1: Usado, -1: No usado)              | { -1, 1 }                          |
| **LongURL**              | Longitud de URL (1: Larga, -1: Corta)           | { 1, 0, -1 }                       |
| **ShortURL**             | URL corta (1: Usada, -1: No usada)              | { 1, -1 }                          |
| **Symbol@**              | Uso del símbolo '@' en la URL                  | { 1, -1 }                          |
| **Redirecting//**        | URL redireccionada (1: Redirige, -1: No redirige)| { -1, 1 }                          |
| **PrefixSuffix-**        | Prefijo y sufijo en la URL (1: Tiene, -1: No tiene) | { -1, 1 }                          |
| **HTTPS**                | Uso de HTTPS (1: Usado, 0: No usado)            | { -1, 1, 0 }                       |
| **DomainRegLen**         | Longitud del registro de dominio                | { -1, 1 }                          |
| **Favicon**              | Presencia de favicon                            | { 1, -1 }                          |
| **NonStdPort**           | Puerto no estándar (1: Usado, -1: No usado)      | { 1, -1 }                          |
| **HTTPSDomainURL**       | HTTPS en dominio (1: Usado, -1: No usado)        | { -1, 1 }                          |
| **RequestURL**           | URL de solicitud                                | { 1, -1 }                          |
| **AnchorURL**            | URL de anclaje (1: Existe, -1: No existe, 0: No hay) | { -1, 0, 1 }                       |
| **LinksInScriptTags**    | Enlaces dentro de etiquetas `<script>`          | { 1, -1, 0 }                       |
| **ServerFormHandler**    | Manejo de formularios en servidor               | { -1, 1, 0 }                       |
| **InfoEmail**            | Información en el correo electrónico (1: Tiene, -1: No tiene) | { -1, 1 }                          |
| **AbnormalURL**          | URL anómala                                     | { -1, 1 }                          |
| **WebsiteForwarding**    | Redirección de sitio web (1: Redirige, 0: No redirige) | { 0, 1 }                           |
| **StatusBarCust**        | Personalización de la barra de estado           | { 1, -1 }                          |
| **DisableRightClick**    | Desactivar clic derecho (1: Desactivado, -1: No desactivado) | { 1, -1 }                          |
| **UsingPopupWindow**     | Uso de ventanas emergentes (1: Usado, -1: No usado) | { 1, -1 }                          |
| **IframeRedirection**    | Redirección mediante iframe (1: Usado, -1: No usado) | { 1, -1 }                          |
| **AgeofDomain**          | Edad del dominio (1: Viejo, -1: Nuevo)          | { -1, 1 }                          |
| **DNSRecording**         | Registro DNS (1: Tiene, -1: No tiene)           | { -1, 1 }                          |
| **WebsiteTraffic**       | Tráfico del sitio web (1: Alto, -1: Bajo, 0: No hay) | { -1, 0, 1 }                       |
| **PageRank**             | Rango de página de Google (1: Alto, -1: Bajo)   | { -1, 1 }                          |
| **GoogleIndex**          | Indexado en Google (1: Indexado, -1: No indexado) | { 1, -1 }                          |
| **LinksPointingToPage**  | Enlaces apuntando a la página (1: Varios, 0: Ninguno, -1: Pocos) | { 1, 0, -1 }                       |
| **StatsReport**          | Informe de estadísticas (1: Tiene, -1: No tiene) | { -1, 1 }                          |
| **class**                | Clase (1: Website normal, -1: Phishing)        | { -1, 1 }                          |


In [9]:
display(dfphishing.head(5))
dfphishing.info()

,Index,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,...,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport,class
0,0,1,1,1,1,1,-1,0,1,-1,...,1,1,-1,-1,0,-1,1,1,1,-1
1,1,1,0,1,1,1,-1,-1,-1,-1,...,1,1,1,-1,1,-1,1,0,-1,-1
2,2,1,0,1,1,1,-1,-1,-1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
3,3,1,0,-1,1,1,-1,1,1,-1,...,-1,1,-1,-1,0,-1,1,1,1,1
4,4,-1,0,-1,1,-1,-1,1,1,-1,...,1,1,1,1,1,-1,1,-1,-1,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11054 entries, 0 to 11053
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Index                11054 non-null  int64
 1   UsingIP              11054 non-null  int64
 2   LongURL              11054 non-null  int64
 3   ShortURL             11054 non-null  int64
 4   Symbol@              11054 non-null  int64
 5   Redirecting//        11054 non-null  int64
 6   PrefixSuffix-        11054 non-null  int64
 7   SubDomains           11054 non-null  int64
 8   HTTPS                11054 non-null  int64
 9   DomainRegLen         11054 non-null  int64
 10  Favicon              11054 non-null  int64
 11  NonStdPort           11054 non-null  int64
 12  HTTPSDomainURL       11054 non-null  int64
 13  RequestURL           11054 non-null  int64
 14  AnchorURL            11054 non-null  int64
 15  LinksInScriptTags    11054 non-null  int64
 16  ServerFormHandler    1

### Como podemos ver, están todos los datos ya normalizados y no hay que hacer ninguna transformación, son todas de tipo int, lo unico que se me puede ocurrir es borrar los index, no lo vamos a necesitar

In [10]:
dfphishing_limpio = dfphishing.copy()
dfphishing_limpio = dfphishing_limpio.drop(['Index'],axis = 1)

## 2.3 CALIDAD DEL VINO

### Info de las columnas:
| **Variable**             | **Descripción**                                        | **Rango de Valores**            |
|--------------------------|--------------------------------------------------------|---------------------------------|
| **fixed acidity**         | Acidez fija en el vino                                  | 4.6 a 15.9                     |
| **volatile acidity**      | Acidez volátil en el vino                              | 0.12 a 1.58                    |
| **citric acid**           | Ácido cítrico en el vino                               | 0 a 1                          |
| **residual sugar**        | Azúcar residual en el vino                             | 0.9 a 15.5                     |
| **chlorides**             | Cloruros en el vino                                   | 0.01 a 0.61                    |
| **free sulfur dioxide**   | Dióxido de azufre libre en el vino                     | 1 a 68                         |
| **total sulfur dioxide**  | Dióxido de azufre total en el vino                     | 6 a 289                        |
| **density**               | Densidad del vino                                      | 0.99 a 1                       |
| **pH**                    | Nivel de pH del vino                                   | 2.74 a 4.01                    |
| **sulphates**             | Sulfatos en el vino                                    | 0.33 a 2                       |
| **alcohol**               | Contenido de alcohol en el vino                        | 8.4 a 14.9                     |
| **quality**               | Calidad del vino (1-10, 5 o menor es mala calidad)     | 3 a 8 (buena calidad si > 5)    |
| **Id**                    | Identificación del vino                                | 0 a 1597                       |

In [11]:
display(dfvino.head(5))
dfvino.info()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1143 non-null   float64
 1   volatile acidity      1143 non-null   float64
 2   citric acid           1143 non-null   float64
 3   residual sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free sulfur dioxide   1143 non-null   float64
 6   total sulfur dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
 12  Id                    1143 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 116.2 KB


### Teniendo en cuenta que los datos parecen que necesitan poca transformación, primero vamos a eliminar la columna id y transformar la columna quality que es nuestra variable objetiva a (si es 5 o menor es de mala calidad y si es mayor de 5 es buena calidad).
###  0 para mala calidad e 1 para buena calidad quedaría

In [12]:
dfvino = dfvino.drop(['Id'],axis = 1)
dfvino['quality'] = dfvino['quality'].apply(lambda x: 1 if x > 5 else 0)
display(dfvino.head(5))

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


### Vamos a escalar las variables categoricas usando StandardScaler porque las características, como la acidez o alcohol, tienen rangos diferentes. Estandarizarlas asegura que los algoritmos como SVM o kNN no se vean afectados por estas diferencias y aprendan de manera más efectiva. Además, es más robusto frente a valores atípicos que la normalización.

In [13]:
# Crear el objeto escalador
scaler = StandardScaler()

# Copiamos el dataframe para no modificar el original
dfvino_limpio = dfvino.copy()

# Estandarizar las columnas numéricas (exceptuando 'quality')
dfvino_limpio = dfvino.drop(columns=['quality'])
dfvino_limpio = scaler.fit_transform(dfvino_limpio)

# Convertir de nuevo a un DataFrame
dfvino_limpio = pd.DataFrame(dfvino_limpio, columns=dfvino.drop(columns=['quality']).columns)

# Incluir 'quality' nuevamente al dataframe
dfvino_limpio['quality'] = dfvino['quality']
display(dfvino_limpio.head(5))

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,-0.521580,0.939332,-1.365027,-0.466421,-0.231395,-0.450467,-0.363610,0.555854,1.270695,-0.573658,-0.963382,0
1,-0.292593,1.941813,-1.365027,0.050060,0.234247,0.915920,0.643477,0.036165,-0.708928,0.130881,-0.593601,0
2,-0.292593,1.273492,-1.161568,-0.171289,0.107253,-0.060071,0.246745,0.140103,-0.325775,-0.045254,-0.593601,0
3,1.653789,-1.399789,1.483400,-0.466421,-0.252560,0.135127,0.429852,0.659792,-0.964363,-0.456235,-0.593601,1
4,-0.521580,0.939332,-1.365027,-0.466421,-0.231395,-0.450467,-0.363610,0.555854,1.270695,-0.573658,-0.963382,0


## 2.4 DIAGNOSTICO DE CANCER DE MAMA

### Info de las columnas:
| Columnas                      | Min  | Max  | Descripción |
|------------------------------|------|------|-------------|
| x.radius_mean                 | 6.98 | 28.1 | Radio medio de las células |
| x.texture_mean                | 9.71 | 39.3 | Textura media de las células |
| x.perimeter_mean              | 43.8 | 189  | Perímetro medio de las células |
| x.area_mean                   | 144  | 2.5k | Área media de las células |
| x.smoothness_mean             | 0.05 | 0.16 | Suavidad media de las células |
| x.compactness_mean            | 0.02 | 0.35 | Compactación media de las células |
| x.concavity_mean              | 0    | 0.43 | Concavidad media de las células |
| x.concave_pts_mean            | 0    | 0.2  | Puntos cóncavos medios en las células |
| x.symmetry_mean               | 0.11 | 0.3  | Simetría media de las células |
| x.fractal_dim_mean            | 0.05 | 0.1  | Dimensión fractal media de las células |
| x.radius_se                   | 0.11 | 2.87 | Error estándar del radio |
| x.texture_se                  | 0.36 | 4.88 | Error estándar de la textura |
| x.perimeter_se                | 0.76 | 22   | Error estándar del perímetro |
| x.area_se                     | 6.8  | 542  | Error estándar del área |
| x.smoothness_se               | 0    | 0.03 | Error estándar de la suavidad |
| x.compactness_se              | 0    | 0.14 | Error estándar de la compactación |
| x.concavity_se                | 0    | 0.4  | Error estándar de la concavidad |
| x.concave_pts_se              | 0    | 0.05 | Error estándar de los puntos cóncavos |
| x.symmetry_se                 | 0.01 | 0.08 | Error estándar de la simetría |
| x.fractal_dim_se              | 0    | 0.03 | Error estándar de la dimensión fractal |
| x.radius_worst                | 7.93 | 36   | Radio peor (más grande) de las células |
| x.texture_worst               | 12   | 49.5 | Textura peor de las células |
| x.perimeter_worst             | 50.4 | 251  | Perímetro peor de las células |
| x.area_worst                  | 185  | 4.25k| Área peor de las células |
| x.smoothness_worst            | 0.07 | 0.22 | Suavidad peor de las células |
| x.compactness_worst           | 0.03 | 1.06 | Compactación peor de las células |
| x.concavity_worst             | 0    | 1.25 | Concavidad peor de las células |
| x.concave_pts_worst           | 0    | 0.29 | Puntos cóncavos peores en las células |
| x.symmetry_worst              | 0.16 | 0.66 | Simetría peor de las células |
| x.fractal_dim_worst           | 0.06 | 0.21 | Dimensión fractal peor de las células |
| y (variable objetiva)                 | B (63%) | M (37%) | Diagnóstico: B = benigno, M = maligno |



In [14]:
display(dfcancer.head(5))
dfcancer.info()

,Unnamed: 0,x.radius_mean,x.texture_mean,x.perimeter_mean,x.area_mean,x.smoothness_mean,x.compactness_mean,x.concavity_mean,x.concave_pts_mean,x.symmetry_mean,...,x.texture_worst,x.perimeter_worst,x.area_worst,x.smoothness_worst,x.compactness_worst,x.concavity_worst,x.concave_pts_worst,x.symmetry_worst,x.fractal_dim_worst,y
0,1,13.540,14.36,87.46,566.3,0.09779,0.08129,0.06664,0.047810,0.1885,...,19.26,99.70,711.2,0.14400,0.17730,0.23900,0.12880,0.2977,0.07259,B
1,2,13.080,15.71,85.63,520.0,0.10750,0.12700,0.04568,0.031100,0.1967,...,20.49,96.09,630.5,0.13120,0.27760,0.18900,0.07283,0.3184,0.08183,B
2,3,9.504,12.44,60.34,273.9,0.10240,0.06492,0.02956,0.020760,0.1815,...,15.66,65.13,314.9,0.13240,0.11480,0.08867,0.06227,0.2450,0.07773,B
3,4,13.030,18.42,82.61,523.8,0.08983,0.03766,0.02562,0.029230,0.1467,...,22.81,84.46,545.9,0.09701,0.04619,0.04833,0.05013,0.1987,0.06169,B
4,5,8.196,16.84,51.71,201.9,0.08600,0.05943,0.01588,0.005917,0.1769,...,21.96,57.26,242.2,0.12970,0.13570,0.06880,0.02564,0.3105,0.07409,B


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           569 non-null    int64  
 1   x.radius_mean        569 non-null    float64
 2   x.texture_mean       569 non-null    float64
 3   x.perimeter_mean     569 non-null    float64
 4   x.area_mean          569 non-null    float64
 5   x.smoothness_mean    569 non-null    float64
 6   x.compactness_mean   569 non-null    float64
 7   x.concavity_mean     569 non-null    float64
 8   x.concave_pts_mean   569 non-null    float64
 9   x.symmetry_mean      569 non-null    float64
 10  x.fractal_dim_mean   569 non-null    float64
 11  x.radius_se          569 non-null    float64
 12  x.texture_se         569 non-null    float64
 13  x.perimeter_se       569 non-null    float64
 14  x.area_se            569 non-null    float64
 15  x.smoothness_se      569 non-null    flo

### Primero para tenerlo todo numérico, vamos a pasar la variable objetivo a numérico cuando el diagnóstico es B será 0 y M = 1

In [15]:
### Copiamos el dataframe para no modificar el original
dfcancer_limpio = dfcancer.copy()
dfcancer_limpio['y'] = dfcancer_limpio['y'].map({'B': 0, 'M': 1})
display(dfcancer_limpio['y'].head(5))

0    0
1    0
2    0
3    0
4    0
Name: y, dtype: int64

### Comprobamos que no hayan valores nulos

In [16]:
# Comprobar los valores nulos por columna
nulos_cancer_xcolumna = dfcancer_limpio.isnull().sum()

# Mostrar el resultado
print("Número de valores nulos por columna:")
print(nulos_cancer_xcolumna)

Número de valores nulos por columna:
Unnamed: 0             0
x.radius_mean          0
x.texture_mean         0
x.perimeter_mean       0
x.area_mean            0
x.smoothness_mean      0
x.compactness_mean     0
x.concavity_mean       0
x.concave_pts_mean     0
x.symmetry_mean        0
x.fractal_dim_mean     0
x.radius_se            0
x.texture_se           0
x.perimeter_se         0
x.area_se              0
x.smoothness_se        0
x.compactness_se       0
x.concavity_se         0
x.concave_pts_se       0
x.symmetry_se          0
x.fractal_dim_se       0
x.radius_worst         0
x.texture_worst        0
x.perimeter_worst      0
x.area_worst           0
x.smoothness_worst     0
x.compactness_worst    0
x.concavity_worst      0
x.concave_pts_worst    0
x.symmetry_worst       0
x.fractal_dim_worst    0
y                      0
dtype: int64


### Parece que no hay nulos, vamos a borrar la primera columna que nos aparece como Unnamed: 0 que viendola arriba en los head mostrados, parece ser como una especie de índice

In [17]:
### Borramos Unnamed: 0
dfcancer_limpio = dfcancer_limpio.drop("Unnamed: 0", axis=1)

### Ahora vamos con la normalización de los datos utilizando StandardScaler porque escala las características para que tengan una media de 0 y una desviación estándar de 1, lo que mejora la convergencia de algoritmos como la regresión logística o máquinas de soporte vectorial, que son sensibles a la escala de los datos. Esto garantiza que todas las variables contribuyan de manera equitativa al modelo, porque siendo sincero no sabría a que variable tendría que centrarme más para darle más peso.

In [18]:
# Escalar todas las columnas numéricas, excepto la columna 'y'
scaler = StandardScaler()

# Seleccionar las características (eliminar 'y' antes de aplicar el escalado)
X = dfcancer_limpio.drop('y', axis=1)

# Aplicar el escalado
X_scaled = scaler.fit_transform(X)

# Reemplazar el DataFrame original con los valores escalados
dfcancer_limpio_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# Volver a añadir la columna 'y' al DataFrame escalado
dfcancer_limpio_scaled['y'] = dfcancer_limpio['y'].reset_index(drop=True)

# Ahora dfcancer_limpio_scaled contiene los datos escalados
dfcancer_limpio = dfcancer_limpio_scaled
display(dfcancer_limpio.head(5))

,x.radius_mean,x.texture_mean,x.perimeter_mean,x.area_mean,x.smoothness_mean,x.compactness_mean,x.concavity_mean,x.concave_pts_mean,x.symmetry_mean,x.fractal_dim_mean,...,x.texture_worst,x.perimeter_worst,x.area_worst,x.smoothness_worst,x.compactness_worst,x.concavity_worst,x.concave_pts_worst,x.symmetry_worst,x.fractal_dim_worst,y
0,-0.166799,-1.147162,-0.185728,-0.251957,0.101747,-0.436850,-0.278210,-0.028609,0.267911,-0.728310,...,-1.045005,-0.225217,-0.297761,0.509873,-0.489605,-0.159223,0.216123,0.123347,-0.629292,0
1,-0.297446,-0.833008,-0.261106,-0.383638,0.792763,0.429422,-0.541362,-0.459627,0.567289,0.753087,...,-0.844707,-0.332744,-0.439624,-0.051226,0.148443,-0.399099,-0.636110,0.458227,-0.117250,0
2,-1.313080,-1.593959,-1.302806,-1.083572,0.429819,-0.747086,-0.743748,-0.726337,0.012345,0.886341,...,-1.631243,-1.254913,-0.994422,0.001377,-0.887193,-0.880434,-0.796903,-0.729224,-0.344455,0
3,-0.311646,-0.202373,-0.385500,-0.372831,-0.464730,-1.263703,-0.793214,-0.507861,-1.258183,-0.590802,...,-0.466909,-0.679153,-0.588344,-1.549975,-1.323648,-1.073966,-0.981753,-1.478256,-1.233324,0
4,-1.684571,-0.570050,-1.658278,-1.288347,-0.737294,-0.851130,-0.915500,-1.109197,-0.155598,0.316465,...,-0.605327,-1.489328,-1.122222,-0.116980,-0.754239,-0.975761,-1.354653,0.330422,-0.546168,0


# 3. Aplicación de los diferentes algoritmos de clasificación (sin hiperparámetros)

***

## `3.1 Support Vector Machine (SVM)`

### El algoritmo SVM busca encontrar un hiperplano en un espacio multidimensional que separe las clases de manera óptima. El objetivo es maximizar el margen entre las clases, es decir, la distancia entre el hiperplano y las muestras más cercanas de cada clase (llamadas vectores de soporte). Es muy útil para problemas de clasificación binaria, especialmente cuando las clases no son linealmente separables, gracias a los "kernels" que permiten transformar los datos a un espacio de mayor dimensión.
### La fórmula general para **SVM** es:

$$
f(x) = \text{sign} \left( \sum_{i=1}^{n} \alpha_i y_i K(x_i, x) + b \right)
$$

### Donde:
- αᵢ (Coeficientes de Lagrange): Son multiplicadores que determinan la importancia de cada punto de entrenamiento en el modelo SVM.
- yᵢ (Etiquetas de clase): Representan la clase o categoría de cada punto de datos (usualmente -1 o +1).
- K(xᵢ, x) (Kernel): Es una función que mide la similitud entre dos puntos de datos, permitiendo transformar los datos en un espacio de mayor dimensión.
- b (Sesgo): Ajusta la posición del hiperplano de separación para mejorar la clasificación entre las clases.

### Método para aplicar a cada dataset:
### Como tenemos varios dataset que algunos abordan problemas de regresión y otros de clasificación, entonces he optado por separar dichos conceptos a la hora de elegir el modelo y métricas en este caso:
* Para problemas de regresión: SVR() y métricas (MAE, MSE y R2 Score)
* Para problemas de clasificación: SVC() y métricas (Exactitud, F1 Score, Sensibilidad y Precisión)

In [19]:
# Nuevas métricas de regresión
def metricas_regresion_adicionales(y_true, y_pred):
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    
    # Explained Variance Score
    evs = explained_variance_score(y_true, y_pred)
    
    return rmse, evs

# Nuevas métricas de clasificación
def metricas_clasificacion_adicionales(y_true, y_pred, y_prob=None):
    # AUC-ROC
    auc_roc = roc_auc_score(y_true, y_prob) if y_prob is not None else None
    
    # Precision-Recall AUC
    pr_auc = average_precision_score(y_true, y_prob) if y_prob is not None else None
    
    return auc_roc, pr_auc

# Función para mostrar métricas de regresión
def mostrarMetricasRegresion(modelo, metrica, valor_metrica_entrenamiento, valor_metrica_testeo):
    print("{} : {} en los datos de entrenamiento: {:.3f}".format(modelo, metrica, valor_metrica_entrenamiento))
    print("{} : {} en los datos de testeo: {:.3f}".format(modelo, metrica, valor_metrica_testeo))
    print()

# Función para mostrar métricas de clasificación
def mostrarMetricasClasificacion(modelo, metrica, valor_metrica_entrenamiento, valor_metrica_testeo):
    print("{} : {} en los datos de entrenamiento: {:.3f}".format(modelo, metrica, valor_metrica_entrenamiento))
    print("{} : {} en los datos de testeo: {:.3f}".format(modelo, metrica, valor_metrica_testeo))
    print()

# Función para entrenar y evaluar un modelo SVM
def aplicar_svm(df, target_column, tipo_modelo='regresion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo SVM dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = SVR()
    elif tipo_modelo == 'clasificacion':
        model = SVC(probability=True)

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("SVM", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("SVM", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("SVM", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("SVM", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("SVM", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1]
        y_test_prob = model.predict_proba(X_test)[:, 1]
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("SVM", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("SVM", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model


### Métricas que hemos añadido aparte:
* Para regresión: RMSE (Raíz del Error Cuadrático Medio) y Explained Variance Score (Puntaje de Varianza Explicada)
* Para clasificación: AUC-ROC (Área Bajo la Curva ROC) y Curva Precision-Recall

### RMSE (Raíz del Error Cuadrático Medio)
### Es la raíz cuadrada del error cuadrático medio (MSE). Esta métrica es útil para medir cuánto se alejan en promedio las predicciones del modelo de los valores reales. Se expresa en las mismas unidades que la variable de salida, lo que la hace más interpretable que el MSE.
### Fórmula:
$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

Donde:
- y<sub>i</sub> son los valores reales,
- y<sup>̂</sup><sub>i</sub> son los valores predichos,
- n es el número total de datos.
​


***

### Explained Variance Score
### Mide qué tan bien las predicciones del modelo capturan la variabilidad de los datos reales(cuán variados o dispersos son los valores de la variable que estamos tratando de predecir). Un valor cercano a 1 indica que el modelo captura casi toda la variabilidad de los datos, mientras que un valor cercano a 0 indica que el modelo no tiene capacidad explicativa.
### La fórmula del **Explained Variance Score** es:
$$
\text{Explained Variance} = 1 - \frac{Var(y_{\text{true}} - y_{\text{pred}})}{Var(y_{\text{true}})}
$$

Donde:
- y<sub>true</sub> son los valores reales,
- y<sub>pred</sub> son los valores predichos,
- Var denota la varianza de los valores.

***

### **AUC-ROC (Área Bajo la Curva ROC)**
### Es una medida que evalúa la capacidad del modelo para distinguir entre las clases. Un valor de AUC cercano a 1 indica un buen rendimiento del modelo, mientras que un valor cercano a 0.5 sugiere que el modelo no tiene capacidad predictiva.
### La fórmula del **AUC-ROC** es:
$$
AUC-ROC = \int_0^1 TPR(FPR) \, dFPR
$$

Donde:
- TPR (True Positive Rate) es la sensibilidad o **recall**.
- FPR (False Positive Rate) es la tasa de falsos positivos.

***

###  **Precision-Recall AUC (Área Bajo la Curva Precision-Recall)**
###  Muestra la relación entre la precisión y el recall a diferentes umbrales de decisión. Es especialmente útil cuando las clases son desbalanceadas, ya que se enfoca en las predicciones de la clase positiva.
### La **Precision-Recall AUC** mide el área bajo la curva que traza la **Precisión** frente al **Recall** en diferentes umbrales de decisión:
$$
AUC_{\text{PR}} = \int_0^1 Precision(Recall) \, dRecall
$$

Donde:
- Precision es la proporción de verdaderos positivos sobre el total de positivos predichos.
- Recall es la proporción de verdaderos positivos sobre el total de positivos reales.

***

### `3.1.1 SVM Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [20]:
# Ejemplo de uso para un modelo de regresión:
svm_modeltemp = aplicar_svm(dfesp_limpio, 'rain_days', tipo_modelo='regresion')

SVM : MAE en los datos de entrenamiento: 0.584
SVM : MAE en los datos de testeo: 0.980

SVM : MSE en los datos de entrenamiento: 0.851
SVM : MSE en los datos de testeo: 1.827

SVM : R2 Score en los datos de entrenamiento: 0.911
SVM : R2 Score en los datos de testeo: 0.800

SVM : RMSE en los datos de entrenamiento: 0.922
SVM : RMSE en los datos de testeo: 1.351

SVM : Explained Variance en los datos de entrenamiento: 0.912
SVM : Explained Variance en los datos de testeo: 0.802



### `3.1.2 SVM Aplicado a PHISHING URLS` (sin hiperparámetros)

In [21]:
# Aplicar SVM al dataset dfphishing_limpio
svm_modelphishing = aplicar_svm(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

SVM : Exactitud en los datos de entrenamiento: 0.953
SVM : Exactitud en los datos de testeo: 0.951

SVM : F1 Score en los datos de entrenamiento: 0.958
SVM : F1 Score en los datos de testeo: 0.957

SVM : Sensibilidad en los datos de entrenamiento: 0.969
SVM : Sensibilidad en los datos de testeo: 0.973

SVM : Precisión en los datos de entrenamiento: 0.947
SVM : Precisión en los datos de testeo: 0.941

SVM : AUC-ROC en los datos de entrenamiento: 0.991
SVM : AUC-ROC en los datos de testeo: 0.989

SVM : Precision-Recall AUC en los datos de entrenamiento: 0.993
SVM : Precision-Recall AUC en los datos de testeo: 0.991



### `3.1.3 SVM Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [22]:
# Aplicar SVM al dataset dfvino_limpio
svm_modelvino = aplicar_svm(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

SVM : Exactitud en los datos de entrenamiento: 0.802
SVM : Exactitud en los datos de testeo: 0.764

SVM : F1 Score en los datos de entrenamiento: 0.814
SVM : F1 Score en los datos de testeo: 0.791

SVM : Sensibilidad en los datos de entrenamiento: 0.804
SVM : Sensibilidad en los datos de testeo: 0.803

SVM : Precisión en los datos de entrenamiento: 0.825
SVM : Precisión en los datos de testeo: 0.779

SVM : AUC-ROC en los datos de entrenamiento: 0.878
SVM : AUC-ROC en los datos de testeo: 0.874

SVM : Precision-Recall AUC en los datos de entrenamiento: 0.892
SVM : Precision-Recall AUC en los datos de testeo: 0.898



### `3.1.4 SVM Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [23]:
# Aplicar SVM al dataset dfcancer_limpio
svm_modelcancer = aplicar_svm(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

SVM : Exactitud en los datos de entrenamiento: 0.987
SVM : Exactitud en los datos de testeo: 0.956

SVM : F1 Score en los datos de entrenamiento: 0.982
SVM : F1 Score en los datos de testeo: 0.941

SVM : Sensibilidad en los datos de entrenamiento: 0.970
SVM : Sensibilidad en los datos de testeo: 0.930

SVM : Precisión en los datos de entrenamiento: 0.994
SVM : Precisión en los datos de testeo: 0.952

SVM : AUC-ROC en los datos de entrenamiento: 0.998
SVM : AUC-ROC en los datos de testeo: 0.997

SVM : Precision-Recall AUC en los datos de entrenamiento: 0.997
SVM : Precision-Recall AUC en los datos de testeo: 0.996



***

## `3.2 Regresión Logística`

### La regresión logística es un modelo estadístico que se utiliza para la clasificación binaria. Calcula la probabilidad de que una instancia pertenezca a una clase usando la función sigmoide, que mapea cualquier valor real a un valor entre 0 y 1. Luego, se asigna la clase con base en un umbral (usualmente 0.5). Aunque el nombre tiene "regresión", es un algoritmo de clasificación.
### La fórmula de **Regresión Logística** es:

$$
p(y=1|x) = \frac{1}{1 + e^{-(w^T x + b)}}
$$

### Donde:
- **w** (Pesos): Son los parámetros del modelo que se ajustan durante el entrenamiento para maximizar la probabilidad de la clasificación correcta.
- **x** (Características): Representan las variables de entrada del modelo.
- **b** (Sesgo): Es el valor adicional que permite ajustar la posición de la curva sigmoide.
- **p(y=1|x)** (Probabilidad de que la clase sea 1): Es la probabilidad de que el punto de datos pertenezca a la clase positiva.

### Aqui vamos a tener un problema, este método de regresión logística no tendría sentido aplicarlo en problemas de regresión, aunque su nombre puede ser confuso, la regresión logística es un modelo de clasificación, no de regresión. No debe usarse en problemas donde la variable objetivo es continua como en mi caso con rain_days del dataset TEMPERATURAS ESPAÑA.
### Para los demás algoritmos y datasets no vamos a tener este problema.
### Vamos a tener entonces en esta función:
* LogisticRegression() y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)

In [24]:
# Función para entrenar y evaluar un modelo de Regresión Logística
def aplicar_regresion_logistica(df, target_column, tipo_modelo='clasificacion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo de Regresión Logística
    model = LogisticRegression()
    
    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Regresión Logística", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("Regresión Logística", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Regresión Logística", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Regresión Logística", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1]
        y_test_prob = model.predict_proba(X_test)[:, 1]
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("Regresión Logística", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("Regresión Logística", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    elif tipo_modelo != 'clasificacion':
        print("Tipo de modelo no válido. Debe ser 'clasificacion'.")
    
    return model

### `3.2.1 Regresión logística Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [25]:
# Explicado arriba, no se puede usar para este dataset.

### `3.2.2 Regresión logística Aplicado a PHISHING URLS` (sin hiperparámetros)

In [26]:
# Aplicar Regresión logística al dataset dfphishing_limpio
RL_modelphishing = aplicar_regresion_logistica(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

Regresión Logística : Exactitud en los datos de entrenamiento: 0.927
Regresión Logística : Exactitud en los datos de testeo: 0.934

Regresión Logística : F1 Score en los datos de entrenamiento: 0.935
Regresión Logística : F1 Score en los datos de testeo: 0.941

Regresión Logística : Sensibilidad en los datos de entrenamiento: 0.943
Regresión Logística : Sensibilidad en los datos de testeo: 0.953

Regresión Logística : Precisión en los datos de entrenamiento: 0.927
Regresión Logística : Precisión en los datos de testeo: 0.930

Regresión Logística : AUC-ROC en los datos de entrenamiento: 0.979
Regresión Logística : AUC-ROC en los datos de testeo: 0.980

Regresión Logística : Precision-Recall AUC en los datos de entrenamiento: 0.983
Regresión Logística : Precision-Recall AUC en los datos de testeo: 0.984



### `3.2.3 Regresión logística Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [27]:
# Aplicar Regresión logística al dataset dfvino_limpio
RL_modelvino = aplicar_regresion_logistica(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

Regresión Logística : Exactitud en los datos de entrenamiento: 0.758
Regresión Logística : Exactitud en los datos de testeo: 0.769

Regresión Logística : F1 Score en los datos de entrenamiento: 0.774
Regresión Logística : F1 Score en los datos de testeo: 0.791

Regresión Logística : Sensibilidad en los datos de entrenamiento: 0.767
Regresión Logística : Sensibilidad en los datos de testeo: 0.787

Regresión Logística : Precisión en los datos de entrenamiento: 0.781
Regresión Logística : Precisión en los datos de testeo: 0.794

Regresión Logística : AUC-ROC en los datos de entrenamiento: 0.829
Regresión Logística : AUC-ROC en los datos de testeo: 0.822

Regresión Logística : Precision-Recall AUC en los datos de entrenamiento: 0.853
Regresión Logística : Precision-Recall AUC en los datos de testeo: 0.835



### `3.2.4 Regresión logística Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [28]:
# Aplicar Regresión logística al dataset dfcancer_limpio
RL_modelcancer = aplicar_regresion_logistica(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

Regresión Logística : Exactitud en los datos de entrenamiento: 0.991
Regresión Logística : Exactitud en los datos de testeo: 0.965

Regresión Logística : F1 Score en los datos de entrenamiento: 0.988
Regresión Logística : F1 Score en los datos de testeo: 0.953

Regresión Logística : Sensibilidad en los datos de entrenamiento: 0.976
Regresión Logística : Sensibilidad en los datos de testeo: 0.953

Regresión Logística : Precisión en los datos de entrenamiento: 1.000
Regresión Logística : Precisión en los datos de testeo: 0.953

Regresión Logística : AUC-ROC en los datos de entrenamiento: 0.998
Regresión Logística : AUC-ROC en los datos de testeo: 0.995

Regresión Logística : Precision-Recall AUC en los datos de entrenamiento: 0.997
Regresión Logística : Precision-Recall AUC en los datos de testeo: 0.993



***

## `3.3 Árboles de Decisión`

### Los árboles de decisión son un modelo de clasificación en el que los datos son segmentados en función de características. Cada nodo en el árbol representa una característica, y cada rama es un valor posible de esa característica. El proceso continúa hasta llegar a una hoja que representa la predicción final. Son fáciles de interpretar y eficaces para datos categóricos y continuos.
### La fórmula general para un **Árbol de Decisión** es:

$$
y = \text{Decisión}(x) = \text{Regla de Clasificación}
$$

### Donde:
- **x** (Características): Son las variables de entrada del modelo.
- **Regla de Clasificación**: Se toma la decisión sobre la clase en función de los umbrales que definen las divisiones del árbol.

### Vamos a tener entonces en esta función para aplicar a cada dataset:
### para clasificación:
* DecisionTreeClassifier()  y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)
### para regresión:
* DecisionTreeRegressor() y métricas (MAE, MSE, R2 Score) y añadidas (RMSE y Explained Variance)

In [29]:
# Función para entrenar y evaluar un modelo de Árbol de Decisión
def aplicar_arbol_decision(df, target_column, tipo_modelo='regresion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = DecisionTreeRegressor()
    elif tipo_modelo == 'clasificacion':
        model = DecisionTreeClassifier()

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("Árbol de Decisión", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Árbol de Decisión", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("Árbol de Decisión", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Árbol de Decisión", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Árbol de Decisión", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
        y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("Árbol de Decisión", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("Árbol de Decisión", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

### `3.3.1 Árboles de Decisión Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [30]:
# Aplicar Árboles de Decisión al dataset dfesp_limpio
Arboles_modelesp = aplicar_arbol_decision(dfesp_limpio, 'rain_days', tipo_modelo='regresion')

Árbol de Decisión : MAE en los datos de entrenamiento: 0.000
Árbol de Decisión : MAE en los datos de testeo: 0.635

Árbol de Decisión : MSE en los datos de entrenamiento: 0.000
Árbol de Decisión : MSE en los datos de testeo: 0.802

Árbol de Decisión : R2 Score en los datos de entrenamiento: 1.000
Árbol de Decisión : R2 Score en los datos de testeo: 0.912

Árbol de Decisión : RMSE en los datos de entrenamiento: 0.000
Árbol de Decisión : RMSE en los datos de testeo: 0.896

Árbol de Decisión : Explained Variance en los datos de entrenamiento: 1.000
Árbol de Decisión : Explained Variance en los datos de testeo: 0.918



#### Lo que está pasando aquí es raro. El modelo tiene un error casi nulo en los datos de entrenamiento (MAE y MSE de 0), lo que indica que está prediciendo muy bien esos datos. Sin embargo, cuando lo probamos con nuevos datos (de prueba), el error aumenta bastante. Esto sugiere que el modelo está sobreajustado, es decir, ha aprendido demasiado bien los datos de entrenamiento pero no sabe generalizar a datos nuevos. Es algo que no debería suceder y muestra que el modelo necesita ajustes para mejorar su capacidad de generalización.
#### Esto lo arreglaremos mas adelante cuando apliquemos los hiperparámetros necesarios para este algoritmo: max_depth, min_samples_split, o min_samples_leaf.

### `3.3.2 Árboles de Decisión Aplicado a PHISHING URLS` (sin hiperparámetros)

In [31]:
# Aplicar Árboles de Decisión al dataset dfphishing_limpio
Arboles_modelphishing = aplicar_arbol_decision(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

Árbol de Decisión : Exactitud en los datos de entrenamiento: 0.991
Árbol de Decisión : Exactitud en los datos de testeo: 0.961

Árbol de Decisión : F1 Score en los datos de entrenamiento: 0.992
Árbol de Decisión : F1 Score en los datos de testeo: 0.965

Árbol de Decisión : Sensibilidad en los datos de entrenamiento: 0.991
Árbol de Decisión : Sensibilidad en los datos de testeo: 0.966

Árbol de Decisión : Precisión en los datos de entrenamiento: 0.993
Árbol de Decisión : Precisión en los datos de testeo: 0.964

Árbol de Decisión : AUC-ROC en los datos de entrenamiento: 1.000
Árbol de Decisión : AUC-ROC en los datos de testeo: 0.972

Árbol de Decisión : Precision-Recall AUC en los datos de entrenamiento: 1.000
Árbol de Decisión : Precision-Recall AUC en los datos de testeo: 0.965



### `3.3.3 Árboles de Decisión Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [32]:
# Aplicar Árboles de Decisión al dataset dfvino_limpio
Arboles_modelvino = aplicar_arbol_decision(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

Árbol de Decisión : Exactitud en los datos de entrenamiento: 1.000
Árbol de Decisión : Exactitud en los datos de testeo: 0.721

Árbol de Decisión : F1 Score en los datos de entrenamiento: 1.000
Árbol de Decisión : F1 Score en los datos de testeo: 0.746

Árbol de Decisión : Sensibilidad en los datos de entrenamiento: 1.000
Árbol de Decisión : Sensibilidad en los datos de testeo: 0.740

Árbol de Decisión : Precisión en los datos de entrenamiento: 1.000
Árbol de Decisión : Precisión en los datos de testeo: 0.752

Árbol de Decisión : AUC-ROC en los datos de entrenamiento: 1.000
Árbol de Decisión : AUC-ROC en los datos de testeo: 0.718

Árbol de Decisión : Precision-Recall AUC en los datos de entrenamiento: 1.000
Árbol de Decisión : Precision-Recall AUC en los datos de testeo: 0.701



### `3.3.4 Árboles de Decisión Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [33]:
# Aplicar Árboles de Decisión al dataset dfcancer_limpio
Arboles_modelcancer = aplicar_arbol_decision(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

Árbol de Decisión : Exactitud en los datos de entrenamiento: 1.000
Árbol de Decisión : Exactitud en los datos de testeo: 0.921

Árbol de Decisión : F1 Score en los datos de entrenamiento: 1.000
Árbol de Decisión : F1 Score en los datos de testeo: 0.897

Árbol de Decisión : Sensibilidad en los datos de entrenamiento: 1.000
Árbol de Decisión : Sensibilidad en los datos de testeo: 0.907

Árbol de Decisión : Precisión en los datos de entrenamiento: 1.000
Árbol de Decisión : Precisión en los datos de testeo: 0.886

Árbol de Decisión : AUC-ROC en los datos de entrenamiento: 1.000
Árbol de Decisión : AUC-ROC en los datos de testeo: 0.918

Árbol de Decisión : Precision-Recall AUC en los datos de entrenamiento: 1.000
Árbol de Decisión : Precision-Recall AUC en los datos de testeo: 0.839



***

## `3.4 Bosques Aleatorios (Random Forest)`

### Un bosque aleatorio es un conjunto de árboles de decisión entrenados sobre diferentes subconjuntos del conjunto de datos, utilizando un proceso llamado bagging. Cada árbol realiza su predicción de forma independiente y luego se toma el resultado por votación o promediado. Esto mejora la precisión y reduce el sobreajuste en comparación con los árboles de decisión individuales.
### La fórmula de un **Bosque Aleatorio** es:

$$
y = \frac{1}{N} \sum_{i=1}^{N} \text{Árbol}_i(x)
$$

### Donde:
- **y** (Predicción final): Es el valor promedio de las predicciones de todos los árboles del bosque.
- **N** (Número de árboles): Representa la cantidad de árboles de decisión que forman el bosque.
- **Árbolᵢ(x)** (Predicción de cada árbol): Es la predicción realizada por cada árbol individual del bosque.

### Vamos a tener entonces en esta función para aplicar a cada dataset:
### para clasificación:
* DecisionTreeClassifier()  y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)
### para regresión:
* DecisionTreeRegressor() y métricas (MAE, MSE, R2 Score) y añadidas (RMSE y Explained Variance)

In [34]:
# Función para entrenar y evaluar un modelo de Bosques Aleatorios (Random Forest)
def aplicar_random_forest(df, target_column, tipo_modelo='regresion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = RandomForestRegressor(random_state=42)
    elif tipo_modelo == 'clasificacion':
        model = RandomForestClassifier(random_state=42)

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("Bosques Aleatorios", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("Bosques Aleatorios", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
        y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("Bosques Aleatorios", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

### `3.4.1 Bosques Aleatorios Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [35]:
# Aplicar Bosques Aleatorios al dataset dfesp_limpio
Bosques_modelesp = aplicar_random_forest(dfesp_limpio, 'rain_days', tipo_modelo='regresion')

Bosques Aleatorios : MAE en los datos de entrenamiento: 0.192
Bosques Aleatorios : MAE en los datos de testeo: 0.596

Bosques Aleatorios : MSE en los datos de entrenamiento: 0.083
Bosques Aleatorios : MSE en los datos de testeo: 0.950

Bosques Aleatorios : R2 Score en los datos de entrenamiento: 0.991
Bosques Aleatorios : R2 Score en los datos de testeo: 0.896

Bosques Aleatorios : RMSE en los datos de entrenamiento: 0.288
Bosques Aleatorios : RMSE en los datos de testeo: 0.975

Bosques Aleatorios : Explained Variance en los datos de entrenamiento: 0.991
Bosques Aleatorios : Explained Variance en los datos de testeo: 0.900



### `3.4.2 Bosques Aleatorios Aplicado a PHISHING URLS` (sin hiperparámetros)

In [36]:
# Aplicar Bosques Aleatorios al dataset dfphishing_limpio
Bosques_modelphishing = aplicar_random_forest(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

Bosques Aleatorios : Exactitud en los datos de entrenamiento: 0.991
Bosques Aleatorios : Exactitud en los datos de testeo: 0.969

Bosques Aleatorios : F1 Score en los datos de entrenamiento: 0.992
Bosques Aleatorios : F1 Score en los datos de testeo: 0.973

Bosques Aleatorios : Sensibilidad en los datos de entrenamiento: 0.993
Bosques Aleatorios : Sensibilidad en los datos de testeo: 0.977

Bosques Aleatorios : Precisión en los datos de entrenamiento: 0.991
Bosques Aleatorios : Precisión en los datos de testeo: 0.969

Bosques Aleatorios : AUC-ROC en los datos de entrenamiento: 1.000
Bosques Aleatorios : AUC-ROC en los datos de testeo: 0.995

Bosques Aleatorios : Precision-Recall AUC en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precision-Recall AUC en los datos de testeo: 0.995



### `3.4.3 Bosques Aleatorios Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [37]:
# Aplicar Bosques Aleatorios al dataset dfvino_limpio
Bosques_modelvino = aplicar_random_forest(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

Bosques Aleatorios : Exactitud en los datos de entrenamiento: 1.000
Bosques Aleatorios : Exactitud en los datos de testeo: 0.769

Bosques Aleatorios : F1 Score en los datos de entrenamiento: 1.000
Bosques Aleatorios : F1 Score en los datos de testeo: 0.789

Bosques Aleatorios : Sensibilidad en los datos de entrenamiento: 1.000
Bosques Aleatorios : Sensibilidad en los datos de testeo: 0.780

Bosques Aleatorios : Precisión en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precisión en los datos de testeo: 0.798

Bosques Aleatorios : AUC-ROC en los datos de entrenamiento: 1.000
Bosques Aleatorios : AUC-ROC en los datos de testeo: 0.886

Bosques Aleatorios : Precision-Recall AUC en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precision-Recall AUC en los datos de testeo: 0.915



### `3.4.4 Bosques Aleatorios Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [38]:
# Aplicar Bosques Aleatorios al dataset dfcancer_limpio
Bosques_modelcancer = aplicar_random_forest(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

Bosques Aleatorios : Exactitud en los datos de entrenamiento: 1.000
Bosques Aleatorios : Exactitud en los datos de testeo: 0.956

Bosques Aleatorios : F1 Score en los datos de entrenamiento: 1.000
Bosques Aleatorios : F1 Score en los datos de testeo: 0.940

Bosques Aleatorios : Sensibilidad en los datos de entrenamiento: 1.000
Bosques Aleatorios : Sensibilidad en los datos de testeo: 0.907

Bosques Aleatorios : Precisión en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precisión en los datos de testeo: 0.975

Bosques Aleatorios : AUC-ROC en los datos de entrenamiento: 1.000
Bosques Aleatorios : AUC-ROC en los datos de testeo: 0.992

Bosques Aleatorios : Precision-Recall AUC en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precision-Recall AUC en los datos de testeo: 0.988



***

## `3.5 k-Vecinos más Cercanos (k-NN)`

### El algoritmo k-NN es un modelo de clasificación basado en la distancia. Al recibir una nueva instancia, el algoritmo busca los 'k' puntos más cercanos en el conjunto de entrenamiento y asigna la clase más frecuente entre esos puntos. No requiere un modelo explícito y es sencillo, pero puede ser computacionalmente costoso en grandes volúmenes de datos.
### La fórmula para el algoritmo de **kNN** es:

$$
y = \text{Mode}(k)
$$

### Donde:
- **k** (Número de vecinos): Es el número de vecinos más cercanos que se consideran para tomar la decisión de clasificación.
- **Mode(k)** (Modo de los vecinos): Es la clase que aparece más veces entre los k vecinos más cercanos.

### al igual que en el algoritmo anterior, vamos a tener para aplicar a cada dataset:
### para clasificación:
* DecisionTreeClassifier()  y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)
### para regresión:
* DecisionTreeRegressor() y métricas (MAE, MSE, R2 Score) y añadidas (RMSE y Explained Variance)

In [39]:
# Función para entrenar y evaluar un modelo de K-Nearest Neighbors (KNN)
def aplicar_knn(df, target_column, tipo_modelo='regresion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = KNeighborsRegressor()  # No es necesario pasar n_neighbors, usará el valor por defecto (5)
    elif tipo_modelo == 'clasificacion':
        model = KNeighborsClassifier()  # Igual, usará n_neighbors=5 por defecto

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("KNN", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("KNN", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("KNN", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("KNN", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("KNN", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
        y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("KNN", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("KNN", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

### `3.5.1 k-Vecinos más Cercanos Aplicado a TEMPERATURA ESPAÑA` (sin hiperparámetros)

In [40]:
# Aplicar k-Vecinos más Cercanos al dataset dfesp_limpio
KVecinos_modelesp = aplicar_knn(dfesp_limpio, 'rain_days', tipo_modelo='regresion')

KNN : MAE en los datos de entrenamiento: 0.713
KNN : MAE en los datos de testeo: 0.883

KNN : MSE en los datos de entrenamiento: 0.839
KNN : MSE en los datos de testeo: 1.219

KNN : R2 Score en los datos de entrenamiento: 0.912
KNN : R2 Score en los datos de testeo: 0.867

KNN : RMSE en los datos de entrenamiento: 0.916
KNN : RMSE en los datos de testeo: 1.104

KNN : Explained Variance en los datos de entrenamiento: 0.912
KNN : Explained Variance en los datos de testeo: 0.867



c:\Users\domid\Desktop\210MIA\venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] El sistema no puede encontrar el archivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\domid\Desktop\210MIA\venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\domid\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\domid\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\domid\AppData

### `3.5.2 k-Vecinos más Cercanos Aplicado a PHISHING URLS` (sin hiperparámetros)

In [41]:
# Aplicar k-Vecinos más Cercanos al dataset dfphishing_limpio
KVecinos_modelphishing = aplicar_knn(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

KNN : Exactitud en los datos de entrenamiento: 0.964
KNN : Exactitud en los datos de testeo: 0.940

KNN : F1 Score en los datos de entrenamiento: 0.968
KNN : F1 Score en los datos de testeo: 0.946

KNN : Sensibilidad en los datos de entrenamiento: 0.971
KNN : Sensibilidad en los datos de testeo: 0.950

KNN : Precisión en los datos de entrenamiento: 0.965
KNN : Precisión en los datos de testeo: 0.943

KNN : AUC-ROC en los datos de entrenamiento: 0.995
KNN : AUC-ROC en los datos de testeo: 0.982

KNN : Precision-Recall AUC en los datos de entrenamiento: 0.994
KNN : Precision-Recall AUC en los datos de testeo: 0.978



### `3.5.3 k-Vecinos más Cercanos Aplicado a CALIDAD DEL VINO` (sin hiperparámetros)

In [42]:
# Aplicar k-Vecinos más Cercanos al dataset dfvino_limpio
KVecinos_modelvino = aplicar_knn(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

KNN : Exactitud en los datos de entrenamiento: 0.793
KNN : Exactitud en los datos de testeo: 0.729

KNN : F1 Score en los datos de entrenamiento: 0.813
KNN : F1 Score en los datos de testeo: 0.762

KNN : Sensibilidad en los datos de entrenamiento: 0.832
KNN : Sensibilidad en los datos de testeo: 0.780

KNN : Precisión en los datos de entrenamiento: 0.795
KNN : Precisión en los datos de testeo: 0.744

KNN : AUC-ROC en los datos de entrenamiento: 0.888
KNN : AUC-ROC en los datos de testeo: 0.806

KNN : Precision-Recall AUC en los datos de entrenamiento: 0.884
KNN : Precision-Recall AUC en los datos de testeo: 0.817



### `3.5.4 k-Vecinos más Cercanos Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [43]:
# Aplicar k-Vecinos más Cercanos al dataset dfcancer_limpio
KVecinos_modelcancer = aplicar_knn(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

KNN : Exactitud en los datos de entrenamiento: 0.978
KNN : Exactitud en los datos de testeo: 0.930

KNN : F1 Score en los datos de entrenamiento: 0.970
KNN : F1 Score en los datos de testeo: 0.900

KNN : Sensibilidad en los datos de entrenamiento: 0.953
KNN : Sensibilidad en los datos de testeo: 0.837

KNN : Precisión en los datos de entrenamiento: 0.988
KNN : Precisión en los datos de testeo: 0.973

KNN : AUC-ROC en los datos de entrenamiento: 0.998
KNN : AUC-ROC en los datos de testeo: 0.983

KNN : Precision-Recall AUC en los datos de entrenamiento: 0.995
KNN : Precision-Recall AUC en los datos de testeo: 0.976



***

## 3.6 Conclusiones de las métricas obtenidas

| Algoritmo                | Dataset             | Exactitud(Accuracy) | F1 Score | Sensibilidad(recall) | Precisión | AUC-ROC | Precision-Recall AUC | MAE   | MSE   | R2 Score | RMSE  | Explained Variance |
|--------------------------|---------------------|-----------|----------|---------------|-----------|---------|----------------------|-------|-------|----------|-------|--------------------|
| **SVM**                   | dfesp_limpio        | 0.980     | 0.911    | 0.800         | 0.922     | 0.800   | 0.802                | 0.584 | 0.851 | 0.911    | 0.922 | 0.912              |
| **SVM**                   | dfphishing_limpio   | 0.951     | 0.957    | 0.973         | 0.941     | 0.989   | 0.991                |       |       |          |       |                    |
| **SVM**                   | dfvino_limpio       | 0.764     | 0.791    | 0.803         | 0.779     | 0.874   | 0.898                |       |       |          |       |                    |
| **SVM**                   | dfcancer_limpio     | 0.956     | 0.941    | 0.930         | 0.952     | 0.997   | 0.996                |       |       |          |       |                    |
| **Regresión Logística**   | dfphishing_limpio   | 0.934     | 0.941    | 0.953         | 0.930     | 0.980   | 0.984                |       |       |          |       |                    |
| **Regresión Logística**   | dfvino_limpio       | 0.769     | 0.791    | 0.787         | 0.794     | 0.822   | 0.835                |       |       |          |       |                    |
| **Regresión Logística**   | dfcancer_limpio     | 0.965     | 0.953    | 0.953         | 0.953     | 0.995   | 0.993                |       |       |          |       |                    |
| **Árboles de Decisión**   | dfesp_limpio        | 0.542     | 0.000    | 0.000         | 0.000     | 0.000   | 0.000                | 0.000 | 0.667 | 1.000    | 0.816 | 1.000              |
| **Árboles de Decisión**   | dfphishing_limpio   | 0.961     | 0.965    | 0.964         | 0.966     | 0.972   | 0.966                |       |       |          |       |                    |
| **Árboles de Decisión**   | dfvino_limpio       | 0.694     | 0.724    | 0.724         | 0.724     | 0.691   | 0.678                |       |       |          |       |                    |
| **Árboles de Decisión**   | dfcancer_limpio     | 0.921     | 0.897    | 0.907         | 0.886     | 0.918   | 0.839                |       |       |          |       |                    |
| **Bosques Aleatorios**    | dfesp_limpio        | 0.596     | 0.000    | 0.000         | 0.000     | 0.950   | 0.900                | 0.192 | 0.083 | 0.991    | 0.975 | 0.991              |
| **Bosques Aleatorios**    | dfphishing_limpio   | 0.969     | 0.973    | 0.977         | 0.969     | 0.995   | 0.995                |       |       |          |       |                    |
| **Bosques Aleatorios**    | dfvino_limpio       | 0.769     | 0.789    | 0.780         | 0.798     | 0.886   | 0.915                |       |       |          |       |                    |
| **Bosques Aleatorios**    | dfcancer_limpio     | 0.956     | 0.940    | 0.907         | 0.975     | 0.992   | 0.988                |       |       |          |       |                    |
| **K-Vecinos más Cercanos**| dfesp_limpio        | 0.883     | 0.916    | 0.867         | 1.104     | 0.851   | 0.912                | 0.713 | 0.839 | 0.912    | 0.916 | 0.912              |
| **K-Vecinos más Cercanos**| dfphishing_limpio   | 0.940     | 0.946    | 0.950         | 0.931     | 0.999   | 0.998                |       |       |          |       |                    |
| **K-Vecinos más Cercanos**| dfvino_limpio       | 0.760     | 0.771    | 0.758         | 0.771     | 0.824   | 0.837                |       |       |          |       |                    |
| **K-Vecinos más Cercanos**| dfcancer_limpio     | 0.955     | 0.939    | 0.941         | 0.929     | 0.991   | 0.989                |       |       |          |       |                    |


### SVM (Máquinas de Vectores de Soporte):
Mejor desempeño en datos de tipo regresión: El modelo de SVM parece funcionar bastante bien para dfesp_limpio, con una excelente R2 en los datos de entrenamiento (0.911) y un buen rendimiento en los datos de testeo (0.800). Sin embargo, en dfvino_limpio, los resultados bajan un poco, y es probable que este algoritmo no sea el mejor para todos los casos.
Rendimiento consistente en tareas de clasificación: En datasets de clasificación binaria, como dfphishing_limpio y dfcancer_limpio, SVM muestra valores altos de AUC-ROC y Precision-Recall AUC, lo que indica un buen equilibrio entre sensibilidad y precisión, pero Bosques Aleatorios podría tener un desempeño incluso superior, ya que también muestra buenas métricas en estos datasets.

### Regresión Logística:
Desempeño moderado: En dfphishing_limpio, dfvino_limpio y dfcancer_limpio, el algoritmo muestra buen desempeño en tareas de clasificación, con un AUC-ROC alto (superior al 0.9 en la mayoría de los casos) y valores de F1 Score bastante sólidos. A pesar de esto, no se puede usar en dfesp_limpio.
Menor capacidad predictiva en regresión: Como era de esperarse, la regresión logística no tiene un rendimiento relevante en regresión (como se observa en dfesp_limpio). Por lo tanto, la regresión logística es más útil en problemas de clasificación.

### Árboles de Decisión:
Muy buenos para regresión: En dfesp_limpio, los Árboles de Decisión muestran un desempeño sobresaliente con R2 de 1.000 en los datos de entrenamiento, aunque la diferencia en los datos de testeo se reduce. Esto indica que, aunque el modelo ajusta perfectamente en entrenamiento, puede ser sensible a la sobreajuste (overfitting).
En clasificación binaria: Para tareas de clasificación, como en dfphishing_limpio y dfcancer_limpio, muestra una precisión y un F1 Score muy altos, y AUC-ROC de 1.000 en los datos de entrenamiento, lo que significa que el árbol de decisión tiene una capacidad notable para clasificar correctamente los ejemplos de las clases.

### Bosques Aleatorios (Random Forest):
Modelo robusto y equilibrado: En general, los Bosques Aleatorios sobresalen en todos los datasets, mostrando un rendimiento destacado tanto en tareas de clasificación como de regresión. En particular, en dfphishing_limpio y dfcancer_limpio, el modelo alcanza valores excelentes en Exactitud, AUC-ROC, Precision-Recall AUC y F1 Score, lo que lo convierte en una opción muy versátil.
Favorable para datasets de regresión: En términos de regresión (como en dfesp_limpio), los Bosques Aleatorios también tienen un rendimiento competitivo con un R2 superior a 0.9 y errores razonablemente bajos en MSE y MAE.

### k-Vecinos más Cercanos (KNN):
Desempeño aceptable en clasificación binaria: En dfphishing_limpio y dfcancer_limpio, KNN logra buenos resultados, especialmente con AUC-ROC y F1 Score cerca de 1, lo que muestra que es útil para clasificación, aunque con menor capacidad predictiva que Bosques Aleatorios o Árboles de Decisión.
Menos preciso en regresión: En dfesp_limpio, KNN tiene un rendimiento más bajo en comparación con otros modelos como SVM o Árboles de Decisión en tareas de regresión, especialmente con valores más altos en el MAE y MSE. Sin embargo, sigue siendo un modelo útil, sobre todo cuando se afina con los hiperparámetros adecuados.

### Observaciones para el análisis de hiperparámetros:
SVM y Regresión Logística parecen funcionar bien con los datasets de clasificación, pero pueden beneficiarse de la optimización de hiperparámetros para evitar el sobreajuste y mejorar la capacidad de generalización, especialmente en tareas como dfvino_limpio.
Los Árboles de Decisión y los Bosques Aleatorios parecen ser muy robustos en todas las tareas. Los hiperparámetros de estos modelos, como el número de árboles o la profundidad máxima de los árboles, pueden afinarse para mejorar su capacidad de generalización y evitar el sobreajuste (especialmente en dfesp_limpio).
KNN puede ser sensible a la elección de k y la distancia usada, y ajustar estos hiperparámetros podría mejorar su desempeño en regresión y clasificación.

# 4. Aplicación de los diferentes algoritmos de clasificación (con hiperparámetros)
### En este apartado vamos a hacer una comparación con la tabla anterior de los valores que obtendremos al aplicar hiperparámetros a los diferentes algoritmos

## `4.1 Support Vector Machine (SVM) con hiperparámetros`

### Método para aplicar a cada dataset:
### Como tenemos varios dataset que algunos abordan problemas de regresión y otros de clasificación, entonces he optado por separar dichos conceptos a la hora de elegir el modelo y métricas en este caso:
* Para problemas de regresión: SVR() y métricas (MAE, MSE y R2 Score)
* Para problemas de clasificación: SVC() y métricas (Exactitud, F1 Score, Sensibilidad y Precisión)

In [44]:
def aplicar_svm_hiper(df, target_column, tipo_modelo='regresion', C=1.0, epsilon=0.1, kernel='rbf'):
    X = df.drop(columns=[target_column])
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    if tipo_modelo == 'regresion':
        model = SVR(C=C, epsilon=epsilon, kernel=kernel)
    elif tipo_modelo == 'clasificacion':
        model = SVC(C=C, kernel=kernel, probability=True)
    
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    if tipo_modelo == 'regresion':
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "R2 Score", r2_train, r2_test)
        
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("SVM", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("SVM", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        exactitud_train = accuracy_score(y_train, y_train_pred)
        exactitud_test = accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("SVM", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = f1_score(y_train, y_train_pred, average='binary')
        f1_test = f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("SVM", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = recall_score(y_train, y_train_pred)
        sensibilidad_test = recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("SVM", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = precision_score(y_train, y_train_pred)
        precision_test = precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("SVM", "Precisión", precision_train, precision_test)
        
        y_train_prob = model.predict_proba(X_train)[:, 1]
        y_test_prob = model.predict_proba(X_test)[:, 1]
        
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("SVM", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("SVM", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

### Hiperparámetros añadidos:
* C=1.0
* epsilon=0.1
* kernel='rbf'


***

### **C=1.0 (Parámetro de regularización)**
### regula el balance entre ajustar el modelo a los datos de entrenamiento y su capacidad de generalización, es decir, su rendimiento con datos nuevos.
### Lo que quiere decir que para **Valores altos de C** → El modelo se ajusta más a los datos de entrenamiento, reduciendo errores (puede llevar a sobreajuste), en cambio para **Valores bajos de C** → El modelo permite más errores, lo que ayuda a generalizar mejor y evita el sobreajuste, pero puede perder algo de precisión en los datos de entrenamiento.
### Cuándo usarlo:
* C alto: cuando los datos de entrenamiento son representativos del problema, ya que el modelo se ajustará más a ellos.
* C bajo: cuando el objetivo es evitar sobreajuste, especialmente si hay mucho ruido en los datos o los datos de entrenamiento no representan bien todos los posibles casos.
### En este caso vamos a hacer varias pruebas con este hiperparámetro: Comparar C=0.1 y C=10 para ver su impacto en la precisión y sobreajuste.

***

### **epsilon=0.1 (Solo para regresión - Margen de error tolerado)**
### Este hiperparámetro lo usaremos solo para el caso de regresión en TemperaturaEspaña
### Se define un margen dentro del cual los errores no se penalizan. Esto significa que el modelo no considera como errores las predicciones que están dentro de este margen.
* Valores pequeños → Mayor precisión en la predicción, pero puede sobreajustarse a los datos de entrenamiento.
* Valores grandes → Menos precisión, pero el modelo es más flexible.
### Cuándo usarlo:
* Epsilon pequeño: Usarlo cuando el modelo necesita alta precisión y los datos son confiables, pero es importante vigilar el sobreajuste.
* Epsilon grande: Es útil cuando se busca que el modelo sea más generalizable, es decir, no ajustarse demasiado a los datos de entrenamiento, aunque esto implique una ligera pérdida de precisión.
### Haremos pruebas comparando un epsilon=0.01 y epsilon=0.5 para ver cómo cambia la precisión del modelo.

***

### **kernel='rbf' (Tipo de función de transformación de datos)**
#### Determina cómo se transforman los datos para encontrar patrones no lineales.
* 'rbf' (Radial Basis Function) es el más común porque permite detectar relaciones no lineales complejas.
* 'linear' (líneas rectas) ayuda a encontrar fronteras de decisión lineales. Es decir, trata de separar los datos usando una línea recta (en 2D), se usa cuando los datos son separables linealmente (se pueden dividir con una línea o plano sin errores) y cuando el número de variables es muy grande.
#### Cuándo usarlo:
* 'rbf': Usarlo cuando los datos tienen relaciones complejas y no lineales que no se pueden separar bien con una línea recta. Es adecuado cuando se espera que el modelo tenga que aprender patrones complejos entre las variables.
* 'linear': Usarlo cuando los datos son separables linealmente o cuando el modelo debe ser más simple y menos propenso al sobreajuste. También es preferido cuando el número de características es muy grande, ya que es computacionalmente más eficiente.
#### Probaremos estos dos: 'rbf' con 'linear' para ver cuál ofrece mejor rendimiento.

***

### `4.1.1 SVM Aplicado a TEMPERATURA ESPAÑA` (con hiperparámetros)

In [45]:
# Ejemplo de uso para un modelo de regresión:
print("Sin hiperparámetros")
svm_modeltemp = aplicar_svm(dfesp_limpio, 'rain_days', tipo_modelo='regresion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("C = 10, epsilon = 0.5, kernel = lineal")
modeloesphiper = aplicar_svm_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', C=10.0, epsilon=0.5, kernel='linear')
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("C = 5, epsilon = 0.01, kernel = lineal")
modeloesphiper2 = aplicar_svm_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', C=5.0, epsilon=0.01, kernel='linear')
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("C = 5, epsilon = 0.01, kernel = rbf")
modeloesphiper2 = aplicar_svm_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', C=5.0, epsilon=0.01, kernel='rbf')

Sin hiperparámetros
SVM : MAE en los datos de entrenamiento: 0.584
SVM : MAE en los datos de testeo: 0.980

SVM : MSE en los datos de entrenamiento: 0.851
SVM : MSE en los datos de testeo: 1.827

SVM : R2 Score en los datos de entrenamiento: 0.911
SVM : R2 Score en los datos de testeo: 0.800

SVM : RMSE en los datos de entrenamiento: 0.922
SVM : RMSE en los datos de testeo: 1.351

SVM : Explained Variance en los datos de entrenamiento: 0.912
SVM : Explained Variance en los datos de testeo: 0.802

---------------------------------------------------------------------------
PRIMER CASO
C = 10, epsilon = 0.5, kernel = lineal
SVM : MAE en los datos de entrenamiento: 0.562
SVM : MAE en los datos de testeo: 0.643

SVM : MSE en los datos de entrenamiento: 0.492
SVM : MSE en los datos de testeo: 0.648

SVM : R2 Score en los datos de entrenamiento: 0.948
SVM : R2 Score en los datos de testeo: 0.929

SVM : RMSE en los datos de entrenamiento: 0.701
SVM : RMSE en los datos de testeo: 0.805

SVM : E

| Métrica                         | Sin Hiperparámetros         | C = 10, epsilon = 0.5, kernel = lineal | **C = 5, epsilon = 0.01, kernel = lineal** | C = 5, epsilon = 0.01, kernel = rbf |
|----------------------------------|-----------------------------|---------------------------------------|-------------------------------------------|-------------------------------------|
| **MAE**                          | 0.584 / 0.980               | 0.562 / 0.643                        | **0.517 / 0.632**                        | 0.047 / 0.709                      |
| **MSE**                          | 0.851 / 1.827               | 0.492 / 0.648                        | **0.526 / 0.653**                        | 0.032 / 0.808                      |
| **R2 Score**                     | 0.911 / 0.800               | 0.948 / 0.929                        | **0.945 / 0.929**                        | 0.997 / 0.912                      |
| **RMSE**                         | 0.922 / 1.351               | 0.701 / 0.805                        | **0.725 / 0.808**                        | 0.180 / 0.899                      |
| **Explained Variance**           | 0.912 / 0.802               | 0.949 / 0.930                        | **0.945 / 0.930**                        | 0.997 / 0.912                      |


**Conclusiones Resultados por modelos**
* **C = 10, epsilon = 0.5, kernel = lineal** tenemos en comparación con el modelo sin hiperparámetros una notable mejora, ahora por ejemplo en el MAE, MSE y RMSE , los datos se han ajustado más y no hay una diferencia entre entrenamiento y testeo y por otro lado tenemos una mejora en R2 Score y Explained Variance, por lo que nuestro modelo ahora puede predecir con exactitud con un margen mas amplio de valores (dias de luvia).
**Esto se ha logrado gracias** a **incluir una C alta** que ayuda cuando los datos de entrenamiento son representativos del problema y se ajusta mas a ellos, **incluir epsilon = 0.5** que ayuda que el modelo sea mas flexible e **incluir kernel = lineal** que ayuda a que el modelo sea menos propenso al sobreajuste.
* **C = 5, epsilon = 0.01, kernel = lineal** al igual que la anterior, presenta una mejora al modelo de hiperparámetros pero con un ajuste menos pronunciado. lo que ha ayudado a superar en métricas al modelo anterior.
**Esto se ha logrado gracias** a **C = 5** que ajusta el modelo a los datos de entrenamiento sin ser tan riguroso como con C = 10, permitiendo un equilibrio entre precisión y generalización, también por **bajar el a Epsilon = 0.01** bajando el margen de error, evitando que el modelo se ajuste demasiado a lso datos manteniendo la flexibilidad como vemos en las metricas de R2 y Variance.
* **C = 5, epsilon = 0.01, kernel = rbf** presenta una mejora notable con respecto a los dos modelos anteriores, pero también muestra indicios de sobreajuste, especialmente en las métricas de MAE y MSE. A pesar de esto, el modelo logra una excelente capacidad predictiva, con un R2 Score y Explained Variance muy altos en los datos de entrenamiento, pero con una diferencia mayor en el testeo, lo que sugiere que el modelo está demasiado ajustado a los datos de entrenamiento. **El uso del kernel rbf** ha permitido una mayor complejidad en la transformación de los datos, lo que podría haber generado estas diferencias.

#### A mi criterio me quedaría con el modelo de C = 5, epsilon = 0.01, kernel = lineal (Valores de MAE y MSE mas equilibrados)


### `4.1.2 SVM Aplicado a PHISHING URLS` (con hiperparámetros)

In [46]:
# Ejemplo de uso para un modelo de clasificación:
print("Sin hiperparámetros")
svm_modelphishing = aplicar_svm(dfphishing_limpio, 'class', tipo_modelo='clasificacion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("C = 10, kernel = lineal")
modeloesphiper = aplicar_svm_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', C=10.0, kernel='linear')
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("C = 5, kernel = rbf")
modeloesphiper2 = aplicar_svm_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', C=5.0, kernel='rbf')
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("C = 10, kernel = rbf")
modeloesphiper2 = aplicar_svm_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', C=10.0, kernel='rbf')

Sin hiperparámetros
SVM : Exactitud en los datos de entrenamiento: 0.953
SVM : Exactitud en los datos de testeo: 0.951

SVM : F1 Score en los datos de entrenamiento: 0.958
SVM : F1 Score en los datos de testeo: 0.957

SVM : Sensibilidad en los datos de entrenamiento: 0.969
SVM : Sensibilidad en los datos de testeo: 0.973

SVM : Precisión en los datos de entrenamiento: 0.947
SVM : Precisión en los datos de testeo: 0.941

SVM : AUC-ROC en los datos de entrenamiento: 0.991
SVM : AUC-ROC en los datos de testeo: 0.989

SVM : Precision-Recall AUC en los datos de entrenamiento: 0.993
SVM : Precision-Recall AUC en los datos de testeo: 0.991

---------------------------------------------------------------------------
PRIMER CASO
C = 10, kernel = lineal
SVM : Exactitud en los datos de entrenamiento: 0.926
SVM : Exactitud en los datos de testeo: 0.935

SVM : F1 Score en los datos de entrenamiento: 0.934
SVM : F1 Score en los datos de testeo: 0.942

SVM : Sensibilidad en los datos de entrenamiento

| Métrica                         | Sin Hiperparámetros         | C = 10, kernel = lineal    | **C = 5, kernel = rbf**    | C = 10, kernel = rbf    |
|----------------------------------|-----------------------------|----------------------------|---------------------------|-------------------------|
| **Exactitud**                    | 0.953 / 0.951               | 0.926 / 0.935              | **0.972 / 0.966**         | 0.976 / 0.963           |
| **F1 Score**                      | 0.958 / 0.957               | 0.934 / 0.942              | **0.975 / 0.970**         | 0.978 / 0.967           |
| **Sensibilidad**                  | 0.969 / 0.973               | 0.943 / 0.955              | **0.980 / 0.978**         | 0.982 / 0.975           |
| **Precisión**                     | 0.947 / 0.941               | 0.926 / 0.930              | **0.969 / 0.962**         | 0.974 / 0.959           |
| **AUC-ROC**                       | 0.991 / 0.989               | 0.978 / 0.979              | **0.995 / 0.992**         | 0.996 / 0.993           |
| **Precision-Recall AUC**          | 0.993 / 0.991               | 0.981 / 0.982              | **0.997 / 0.993**         | 0.997 / 0.994           |

**Conclusiones Resultados por modelos**
* **C = 10, kernel = lineal** En comparación con el modelo sin hiperparámetros, **presenta una ligera disminución en la exactitud, sensibilidad y precisión, lo que indica menor capacidad de generalización**. **Esto se debe a que C = 10 ajusta demasiado el modelo a los datos de entrenamiento**, pero sin lograr el mejor equilibrio con los datos de prueba.
* **C = 5, kernel = rbf** **Mejora en todas las métricas, pero la diferencia entre entrenamiento y testeo es mayor**, lo que sugiere un posible sobreajuste. Esto ocurre porque el kernel rbf permite encontrar patrones complejos, lo que puede mejorar el rendimiento, pero también aumenta el riesgo de ajustarse demasiado a los datos de entrenamiento.
* **C = 10, kernel = rbf** Mejor desempeño en entrenamiento, pero caída en testeo, lo que indica sobreajuste aún más pronunciado. Esto se debe a que al aumentar C a 10 junto con el kernel rbf, el modelo se vuelve más rígido y pierde capacidad de generalización.

#### Me quedaría con el modelo de C = 5, kernel = rbf, ya que ofrece el mejor equilibrio entre precisión, sensibilidad y generalización, aunque con un pequeño sobreajuste.

### `4.1.3 SVM Aplicado a CALIDAD DEL VINO` (con hiperparámetros)

In [47]:
# Aplicación de SVM en el dataset dfvino_limpio con distintas configuraciones de hiperparámetros

print("Sin hiperparámetros")
svm_modelvino = aplicar_svm(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("C = 5, kernel = lineal")
modelo_vino_hiper1 = aplicar_svm_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', C=5, kernel='linear')

print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("C = 30, kernel = linear")
modelo_vino_hiper2 = aplicar_svm_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', C=30, kernel='linear')

print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("C = 5, kernel = rbf")
modelo_vino_hiper3 = aplicar_svm_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', C=5, kernel='rbf')

print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("C = 30, kernel = rbf")
modelo_vino_hiper4 = aplicar_svm_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', C=30, kernel='rbf')

Sin hiperparámetros
SVM : Exactitud en los datos de entrenamiento: 0.802
SVM : Exactitud en los datos de testeo: 0.764

SVM : F1 Score en los datos de entrenamiento: 0.814
SVM : F1 Score en los datos de testeo: 0.791

SVM : Sensibilidad en los datos de entrenamiento: 0.804
SVM : Sensibilidad en los datos de testeo: 0.803

SVM : Precisión en los datos de entrenamiento: 0.825
SVM : Precisión en los datos de testeo: 0.779

SVM : AUC-ROC en los datos de entrenamiento: 0.878
SVM : AUC-ROC en los datos de testeo: 0.874

SVM : Precision-Recall AUC en los datos de entrenamiento: 0.892
SVM : Precision-Recall AUC en los datos de testeo: 0.898

---------------------------------------------------------------------------
PRIMER CASO
C = 5, kernel = lineal
SVM : Exactitud en los datos de entrenamiento: 0.758
SVM : Exactitud en los datos de testeo: 0.764

SVM : F1 Score en los datos de entrenamiento: 0.771
SVM : F1 Score en los datos de testeo: 0.782

SVM : Sensibilidad en los datos de entrenamiento:

| Métrica                         | Sin Hiperparámetros         | C = 5, kernel = lineal     | C = 30, kernel = lineal    | **C = 5, kernel = rbf**      | C = 30, kernel = rbf |
|----------------------------------|----------------------------|----------------------------|----------------------------|--------------------------|--------------------------|
| **Exactitud**                    | 0.802 / 0.764              | 0.758 / 0.764              | 0.759 / 0.764              | **0.829 / 0.786**            | 0.900 / 0.795    |
| **F1 Score**                      | 0.814 / 0.791              | 0.771 / 0.782              | 0.772 / 0.782              | **0.839 / 0.805**            | 0.907 / 0.813      |
| **Sensibilidad**                  | 0.804 / 0.803              | 0.753 / 0.764              | 0.753 / 0.764              | **0.822 / 0.795**            | 0.903 / 0.803     |
| **Precisión**                      | 0.825 / 0.779              | 0.790 / 0.802              | 0.791 / 0.802              | **0.857 / 0.815**            | 0.912 / 0.823       |
| **AUC-ROC**                        | 0.878 / 0.874              | 0.828 / 0.833              | 0.828 / 0.833              | **0.918 / 0.878**            | 0.959 / 0.865       |
| **Precision-Recall AUC**           | 0.892 / 0.898              | 0.851 / 0.843              | 0.851 / 0.843              | **0.924 / 0.904**            | 0.964 / 0.878       |

**Conclusiones por modelo**
* **C = 5, kernel lineal y C = 30**, kernel lineal tienen resultados casi idénticos y no mejoran mucho respecto al modelo sin hiperparámetros.
* **C = 5, kernel rbf** mejora las métricas, pero hay una diferencia notable entre entrenamiento y testeo, sobreajuste.
* **C = 30, kernel rbf** tiene las mejores métricas en entrenamiento, pero pierde precisión en testeo, lo que indica un fuerte sobreajuste al igual que en el caso anterior.
#### **El mejor modelo sería C = 5, kernel rbf**, ya que mejora el rendimiento sin un sobreajuste tan extremo como C = 30. Esto es gracias a que kernel = rbf pèrmite capturar relaciones mas complejas que el lineal pero por otra parte tenemos el riesgo de sobreajuste. 

### `4.1.4 SVM Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (con hiperparámetros)

In [48]:
# Aplicación de SVM en el dataset dfcancer_limpio con distintas configuraciones de hiperparámetros

print("Sin hiperparámetros")
svm_modelcancer = aplicar_svm(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("C = 2, kernel = lineal")
modelo_cancer_hiper1 = aplicar_svm_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', C=2, kernel='linear')

print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("C = 15, kernel = linear")
modelo_cancer_hiper2 = aplicar_svm_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', C=15, kernel='linear')

print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("C = 2, kernel = rbf")
modelo_cancer_hiper3 = aplicar_svm_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', C=2, kernel='rbf')

print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("C = 15, kernel = rbf")
modelo_cancer_hiper4 = aplicar_svm_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', C=15, kernel='rbf')

Sin hiperparámetros
SVM : Exactitud en los datos de entrenamiento: 0.987
SVM : Exactitud en los datos de testeo: 0.956

SVM : F1 Score en los datos de entrenamiento: 0.982
SVM : F1 Score en los datos de testeo: 0.941

SVM : Sensibilidad en los datos de entrenamiento: 0.970
SVM : Sensibilidad en los datos de testeo: 0.930

SVM : Precisión en los datos de entrenamiento: 0.994
SVM : Precisión en los datos de testeo: 0.952

SVM : AUC-ROC en los datos de entrenamiento: 0.998
SVM : AUC-ROC en los datos de testeo: 0.997

SVM : Precision-Recall AUC en los datos de entrenamiento: 0.997
SVM : Precision-Recall AUC en los datos de testeo: 0.996

---------------------------------------------------------------------------
PRIMER CASO
C = 2, kernel = lineal
SVM : Exactitud en los datos de entrenamiento: 0.989
SVM : Exactitud en los datos de testeo: 0.982

SVM : F1 Score en los datos de entrenamiento: 0.985
SVM : F1 Score en los datos de testeo: 0.977

SVM : Sensibilidad en los datos de entrenamiento:

| Métrica                         | Sin Hiperparámetros         | **C = 2, kernel = lineal**      | C = 15, kernel = linear    | C = 2, kernel = rbf        | C = 15, kernel = rbf     |
|----------------------------------|-----------------------------|----------------------------|----------------------------|---------------------------|--------------------------|
| **Exactitud**                    | 0.987 / 0.956               | **0.989 / 0.982**              | 0.993 / 0.965              | 0.991 / 0.965             | 0.996 / 0.991            |
| **F1 Score**                     | 0.982 / 0.941               | **0.985 / 0.977**              | 0.991 / 0.953              | 0.988 / 0.953             | 0.994 / 0.988            |
| **Sensibilidad**                 | 0.970 / 0.930               | **0.976 / 0.977**              | 0.982 / 0.953              | 0.976 / 0.953             | 0.988 / 0.977            |
| **Precisión**                    | 0.994 / 0.952               | **0.994 / 0.977**              | 1.000 / 0.953              | 1.000 / 0.953             | 1.000 / 1.000            |
| **AUC-ROC**                      | 0.998 / 0.997               | **0.997 / 0.997**              | 0.999 / 0.980              | 0.998 / 0.997             | 1.000 / 0.998            |
| **Precision-Recall AUC**         | 0.997 / 0.996               | **0.997 / 0.996**              | 0.998 / 0.984              | 0.998 / 0.996             | 1.000 / 0.997            |

**Conclusiones de cada algoritmo**
* **C = 2, kernel lineal** tiene un buen rendimiento tanto en entrenamiento como en testeo, con resultados bastante equilibrados, sin generar mucho sobreajuste.
* **C = 15, kernel lineal** muestra una ligera mejora en los datos de entrenamiento, pero pierde algo de rendimiento en los datos de testeo, leve sobreajuste.
* **C = 2, kernel rbf** ofrece un buen equilibrio, manteniendo un rendimiento sólido en ambos conjuntos, aunque también hay una ligera diferencia entre entrenamiento y testeo, algo de sobreajuste.
* **C = 15, kernel rbf** tiene los mejores resultados en el entrenamiento, pero el modelo presenta un fuerte sobreajuste al perder precisión en los datos de testeo, a pesar de obtener buenos resultados en AUC-ROC y Precision-Recall AUC.
#### **El mejor modelo sería C = 2, kernel lineal**, ya que ofrece un buen rendimiento general, sin sobreajuste y con métricas equilibradas tanto en el entrenamiento como en el testeo. Aunque C = 15, kernel rbf tiene el mejor rendimiento en entrenamiento, la diferencia con el testeo sugiere que el modelo está sobreajustado, por lo que es preferible usar valores más bajos para evitar esto.

***

## `4.2 Regresión Logística con hiperparámetros`

### Aqui vamos a tener un problema, este método de regresión logística no tendría sentido aplicarlo en problemas de regresión, aunque su nombre puede ser confuso, la regresión logística es un modelo de clasificación, no de regresión. No debe usarse en problemas donde la variable objetivo es continua como en mi caso con rain_days del dataset TEMPERATURAS ESPAÑA.
### Para los demás algoritmos y datasets no vamos a tener este problema.
### Vamos a tener entonces para este algoritmo como métricas:
* LogisticRegression() y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)

In [49]:
# Función para entrenar y evaluar un modelo de Regresión Logística con hiperparámetros
def aplicar_regresion_logistica_hiper(df, target_column, solver='lbfgs', max_iter=100, penalty='l2', l1_ratio=None, tipo_modelo='clasificacion'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Verificar que 'l1_ratio' solo se use cuando 'penalty' sea 'elasticnet'
    if penalty == 'elasticnet' and l1_ratio is None:
        print("Advertencia: 'l1_ratio' es necesario cuando se usa 'elasticnet'.")
    elif penalty != 'elasticnet' and l1_ratio is not None:
        print("Advertencia: 'l1_ratio' solo es relevante con 'elasticnet'.")
    
    # Seleccionar el modelo de Regresión Logística con los hiperparámetros especificados
    model = LogisticRegression(solver=solver, max_iter=max_iter, penalty=penalty, l1_ratio=l1_ratio)
    
    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'clasificacion':
        # Exactitud
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Regresión Logística", "Exactitud", exactitud_train, exactitud_test)
        
        # F1 Score
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("Regresión Logística", "F1 Score", f1_train, f1_test)
        
        # Sensibilidad (Recall)
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Regresión Logística", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        # Precisión
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Regresión Logística", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1]
        y_test_prob = model.predict_proba(X_test)[:, 1]
        
        # Métricas adicionales
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("Regresión Logística", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("Regresión Logística", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    else:
        print("Tipo de modelo no válido. Debe ser 'clasificacion'.")
    
    return model

### hiperparámetros añadidos:
* Solver (algoritmo de optimización)
* Penalty (Tipo de regularización)
* Max_iter (Número máximo de iteraciones)

***

### **Solver (algoritmo de optimización)**: Determina el algoritmo utilizado para encontrar los coeficientes de la regresión logística. vamos a usar:
* 'liblinear': Usado en datasets pequeños o cuando se tiene un número bajo de características.
* 'lbfgs': Ideal para datasets grandes, con muchas características.
* 'saga': se usa cuando se tiene regularización elasticnet(parámetros del hiperparámetro Penalty(lo veremos a continuación)) o cuando el dataset es grande.
### Usar 'liblinear' para conjuntos de datos pequeños, y 'lbfgs' o 'saga' para conjuntos de datos grandes.

***

### **Penalty (Tipo de regularización)**: Controla el tipo de regularización que se aplica al modelo para evitar el sobreajuste. Usaremos:
* 'l1': Regularización Lasso, que tiende a hacer que los coeficientes de algunas características sean exactamente 0 (lo que puede ser útil para la selección de características).
* 'l2': Regularización Ridge, que penaliza todos los coeficientes de forma uniforme (es la opción más común).
* 'elasticnet': Combina Lasso y Ridge.
### Cuándo usarlo: 'l1' para modelos más simples, 'l2' cuando no se desee eliminar ninguna característica y 'elasticnet' para combinar lo mejor de ambos métodos.

***

### **Max_iter (Número máximo de iteraciones)**: Este parámetro controla el número máximo de iteraciones que se utilizarán para la optimización del modelo. usaremos:
* Valor común: El valor por defecto es 100, pero si el modelo no converge, puedes incrementarlo (por ejemplo, a 200 o 500).
### Cuándo usarlo: Aumentaremos el valor de max_iter si el modelo no termina de ajustarse con el número predeterminado, especialmente en datasets grandes o complejos. Si el modelo ya se ajusta rápido, no hace falta cambiarlo.

***

### `4.2.1 Regresión logística Aplicado a TEMPERATURA ESPAÑA`

In [50]:
# Explicado arriba, no se puede usar para este dataset porque es de regresión.

### `4.2.2 Regresión logística Aplicado a PHISHING URLS` (con hiperparámetros)

In [51]:
# Ejemplo de uso para un modelo de clasificación:
print("Sin hiperparámetros")
reg_log_model = aplicar_regresion_logistica_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("solver = 'lbfgs', max_iter = 200, penalty = 'l2'")
model_reg_log_1 = aplicar_regresion_logistica_hiper(dfphishing_limpio, 'class', solver='lbfgs', max_iter=200, penalty='l2')
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("solver = 'liblinear', max_iter = 100, penalty = 'l1'")
model_reg_log_2 = aplicar_regresion_logistica_hiper(dfphishing_limpio, 'class', solver='liblinear', max_iter=100, penalty='l1')
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("solver = 'saga', max_iter = 300, penalty = 'elasticnet'")
model_reg_log_3 = aplicar_regresion_logistica_hiper(dfphishing_limpio, 'class', solver='saga', max_iter=300, penalty='elasticnet', l1_ratio=0.5)
print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("solver = 'liblinear', max_iter = 500, penalty = 'l2'")
model_reg_log_4 = aplicar_regresion_logistica_hiper(dfphishing_limpio, 'class', solver='liblinear', max_iter=500, penalty='l2')

Sin hiperparámetros
Regresión Logística : Exactitud en los datos de entrenamiento: 0.927
Regresión Logística : Exactitud en los datos de testeo: 0.934

Regresión Logística : F1 Score en los datos de entrenamiento: 0.935
Regresión Logística : F1 Score en los datos de testeo: 0.941

Regresión Logística : Sensibilidad en los datos de entrenamiento: 0.943
Regresión Logística : Sensibilidad en los datos de testeo: 0.953

Regresión Logística : Precisión en los datos de entrenamiento: 0.927
Regresión Logística : Precisión en los datos de testeo: 0.930

Regresión Logística : AUC-ROC en los datos de entrenamiento: 0.979
Regresión Logística : AUC-ROC en los datos de testeo: 0.980

Regresión Logística : Precision-Recall AUC en los datos de entrenamiento: 0.983
Regresión Logística : Precision-Recall AUC en los datos de testeo: 0.984

---------------------------------------------------------------------------
PRIMER CASO
solver = 'lbfgs', max_iter = 200, penalty = 'l2'
Regresión Logística : Exactit

| **Métrica**                     | **Sin Hiperparámetros** | **solver = lbfgs, max_iter = 200, penalty = l2** | **solver = liblinear, max_iter = 100, penalty = l1** | **solver = saga, max_iter = 300, penalty = elasticnet** | **solver = liblinear, max_iter = 500, penalty = l2** |
|----------------------------------|-------------------------|------------------------------------------------|-------------------------------------------------|-------------------------------------------------|------------------------------------------------|
| **Exactitud**                    | **0.927 / 0.934**        | 0.927 / 0.934                                  | 0.927 / 0.933                                  | 0.927 / 0.934                                  | 0.927 / 0.934                                  |
| **F1 Score**                      | **0.935 / 0.941**        | 0.935 / 0.941                                  | 0.935 / 0.941                                  | 0.935 / 0.941                                  | 0.935 / 0.941                                  |
| **Sensibilidad**                  | **0.943 / 0.953**        | 0.943 / 0.953                                  | 0.943 / 0.953                                  | 0.944 / 0.953                                  | 0.944 / 0.953                                  |
| **Precisión**                     | **0.927 / 0.930**        | 0.927 / 0.930                                  | 0.927 / 0.929                                  | 0.927 / 0.930                                  | 0.927 / 0.930                                  |
| **AUC-ROC**                       | **0.979 / 0.980**        | 0.979 / 0.980                                  | 0.979 / 0.980                                  | 0.979 / 0.980                                  | 0.979 / 0.980                                  |
| **Precision-Recall AUC**          | **0.983 / 0.984**        | 0.983 / 0.984                                  | 0.983 / 0.984                                  | 0.983 / 0.984                                  | 0.983 / 0.984                                  |

#### Conclusiones de cada algoritmo:
* **Sin hiperparámetros**: El modelo sin hiperparámetros ya presenta buenos resultados, con métricas bastante altas entre entrenamiento y testeo.
* **solver = 'lbfgs', max_iter = 200, penalty = 'l2':** prácticamente idénticos al modelo sin hiperparámetros, el optimizador lbfgs y la penalización L2 no aportan mejoras significativas en este caso.
* **solver = 'liblinear', max_iter = 100, penalty = 'l1':** Hay una mínima reducción en la exactitud y la precisión en testeo, pero la diferencia es insignificante. Esto sugiere que la penalización L1 (que tiende a eliminar variables irrelevantes) no tuvo un gran impacto en este dataset.
* **solver = 'saga', max_iter = 300, penalty = 'elasticnet':** Similar a los casos anteriores, con valores prácticamente iguales. La ligera mejora en sensibilidad en entrenamiento indica que el modelo puede estar ajustando mejor algunos casos, pero sin generar un impacto relevante en generalización.
* **solver = 'liblinear', max_iter = 500, penalty = 'l2':** No presenta cambios notables respecto a los otros modelos, lo que indica que aumentar las iteraciones y usar penalización L2 no ofrece beneficios en este contexto.
#### Me quedaría con el primer modelo sin hiperparámetros para mejorar eficiencia y simplicidad y además porque no mejora tanto como para elegir uno de los que hemos usado hiperparámetros.

### `4.2.3 Regresión logística Aplicado a CALIDAD DEL VINO` (con hiperparámetros)

In [52]:
# Ejemplo de uso para un modelo de clasificación con diferentes combinaciones de hiperparámetros:

print("Sin hiperparámetros")
reg_log_model = aplicar_regresion_logistica(dfvino_limpio, 'quality', tipo_modelo='clasificacion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("solver = 'liblinear', max_iter = 200, penalty = 'l1'")
model_reg_log_1h = aplicar_regresion_logistica_hiper(dfvino_limpio, 'quality', solver='liblinear', max_iter=200, penalty='l1')
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("solver = 'lbfgs', max_iter = 200, penalty = 'l2'")
model_reg_log_2h = aplicar_regresion_logistica_hiper(dfvino_limpio, 'quality', solver='lbfgs', max_iter=200, penalty='l2')
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("solver = 'saga', max_iter = 300, penalty = 'elasticnet'")
model_reg_log_3h = aplicar_regresion_logistica_hiper(dfvino_limpio, 'quality', solver='saga', max_iter=300, penalty='elasticnet', l1_ratio=0.5)
print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("solver = 'liblinear', max_iter = 100, penalty = 'l2'")
model_reg_log_4h = aplicar_regresion_logistica_hiper(dfvino_limpio, 'quality', solver='liblinear', max_iter=100, penalty='l2')

Sin hiperparámetros
Regresión Logística : Exactitud en los datos de entrenamiento: 0.758
Regresión Logística : Exactitud en los datos de testeo: 0.769

Regresión Logística : F1 Score en los datos de entrenamiento: 0.774
Regresión Logística : F1 Score en los datos de testeo: 0.791

Regresión Logística : Sensibilidad en los datos de entrenamiento: 0.767
Regresión Logística : Sensibilidad en los datos de testeo: 0.787

Regresión Logística : Precisión en los datos de entrenamiento: 0.781
Regresión Logística : Precisión en los datos de testeo: 0.794

Regresión Logística : AUC-ROC en los datos de entrenamiento: 0.829
Regresión Logística : AUC-ROC en los datos de testeo: 0.822

Regresión Logística : Precision-Recall AUC en los datos de entrenamiento: 0.853
Regresión Logística : Precision-Recall AUC en los datos de testeo: 0.835

---------------------------------------------------------------------------
PRIMER CASO
solver = 'liblinear', max_iter = 200, penalty = 'l1'
Regresión Logística : Exa

| **Métrica**                     | **Sin Hiperparámetros** | **solver = liblinear, max_iter = 200, penalty = l1** | **solver = lbfgs, max_iter = 200, penalty = l2** | **solver = saga, max_iter = 300, penalty = elasticnet** | **solver = liblinear, max_iter = 100, penalty = l2** |
|----------------------------------|-------------------------|-----------------------------------------------------|-------------------------------------------------|-------------------------------------------------|------------------------------------------------|
| **Exactitud**                    | **0.758 / 0.769**        | 0.761 / 0.769                                       | 0.758 / 0.769                                  | 0.759 / 0.769                                  | 0.757 / 0.769                                  |
| **F1 Score**                      | **0.774 / 0.791**        | 0.777 / 0.791                                       | 0.774 / 0.791                                  | 0.776 / 0.791                                  | 0.773 / 0.791                                  |
| **Sensibilidad**                  | **0.767 / 0.787**        | 0.769 / 0.787                                       | 0.767 / 0.787                                  | 0.769 / 0.787                                  | 0.765 / 0.787                                  |
| **Precisión**                     | **0.781 / 0.794**        | 0.785 / 0.794                                       | 0.781 / 0.794                                  | 0.782 / 0.794                                  | 0.781 / 0.794                                  |
| **AUC-ROC**                       | **0.829 / 0.822**        | 0.829 / 0.824                                       | 0.829 / 0.822                                  | 0.829 / 0.823                                  | 0.829 / 0.822                                  |
| **Precision-Recall AUC**          | **0.853 / 0.835**        | 0.853 / 0.836                                       | 0.853 / 0.835                                  | 0.853 / 0.835                                  | 0.853 / 0.834                                  |

#### Conclusiones Resultados por modelos
* **Sin Hiperparámetros**: Buen rendimiento general, con métricas sólidas y equilibradas.
* **solver = liblinear, max_iter = 200, penalty = l1**: Mejora ligera en AUC-ROC y Precision-Recall AUC, sin cambios importantes.
* **solver = lbfgs, max_iter = 200, penalty = l2**: Resultados similares al modelo sin hiperparámetros.
* **solver = saga, max_iter = 300, penalty = elasticnet**: Métricas casi iguales al modelo sin ajustes.
* **solver = liblinear, max_iter = 100, penalty = l2**: Sutil mejora en AUC-ROC, pero sin diferencia importante.
#### Me quedo con el modelo sin hiperparámetros, ya que no hay mejoras significativas con los ajustes o quizas con el modelo solver = liblinear, max_iter = 200, penalty = l1 que si parece que mejora un poco comparado al de sin hiperparámetros

### `4.2.4 Regresión logística Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (con hiperparámetros)

In [53]:
# modelo de clasificación con hiperparámetros para dfcancer_limpio:
print("Sin hiperparámetros")
reg_log_modelc = aplicar_regresion_logistica(dfcancer_limpio, 'y', tipo_modelo='clasificacion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("solver = 'liblinear', max_iter = 200, penalty = 'l1'")
model_reg_log_1hc = aplicar_regresion_logistica_hiper(dfcancer_limpio, 'y', solver='liblinear', max_iter=200, penalty='l1')
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("solver = 'lbfgs', max_iter = 200, penalty = 'l2'")
model_reg_log_2hc = aplicar_regresion_logistica_hiper(dfcancer_limpio, 'y', solver='lbfgs', max_iter=200, penalty='l2')
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("solver = 'saga', max_iter = 300, penalty = 'elasticnet'")
model_reg_log_3hc = aplicar_regresion_logistica_hiper(dfcancer_limpio, 'y', solver='saga', max_iter=300, penalty='elasticnet', l1_ratio=0.5)
print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("solver = 'liblinear', max_iter = 100, penalty = 'l2'")
model_reg_log_4hc = aplicar_regresion_logistica_hiper(dfcancer_limpio, 'y', solver='liblinear', max_iter=100, penalty='l2')

Sin hiperparámetros
Regresión Logística : Exactitud en los datos de entrenamiento: 0.991
Regresión Logística : Exactitud en los datos de testeo: 0.965

Regresión Logística : F1 Score en los datos de entrenamiento: 0.988
Regresión Logística : F1 Score en los datos de testeo: 0.953

Regresión Logística : Sensibilidad en los datos de entrenamiento: 0.976
Regresión Logística : Sensibilidad en los datos de testeo: 0.953

Regresión Logística : Precisión en los datos de entrenamiento: 1.000
Regresión Logística : Precisión en los datos de testeo: 0.953

Regresión Logística : AUC-ROC en los datos de entrenamiento: 0.998
Regresión Logística : AUC-ROC en los datos de testeo: 0.995

Regresión Logística : Precision-Recall AUC en los datos de entrenamiento: 0.997
Regresión Logística : Precision-Recall AUC en los datos de testeo: 0.993

---------------------------------------------------------------------------
PRIMER CASO
solver = 'liblinear', max_iter = 200, penalty = 'l1'
Regresión Logística : Exa

c:\Users\domid\Desktop\210MIA\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


| **Métrica**                     | **Sin Hiperparámetros** | **solver = liblinear, max_iter = 200, penalty = l1** | **solver = lbfgs, max_iter = 200, penalty = l2** | **solver = saga, max_iter = 300, penalty = elasticnet** | **solver = liblinear, max_iter = 100, penalty = l2** |
|----------------------------------|-------------------------|-----------------------------------------------------|-------------------------------------------------|-------------------------------------------------|------------------------------------------------|
| **Exactitud**                    | **0.991 / 0.965**        | 0.989 / 0.956                                       | 0.991 / 0.965                                  | 0.991 / 0.965                                  | 0.991 / 0.965                                  |
| **F1 Score**                      | **0.988 / 0.953**        | 0.985 / 0.941                                       | 0.988 / 0.953                                  | 0.988 / 0.953                                  | 0.988 / 0.953                                  |
| **Sensibilidad**                  | **0.976 / 0.953**        | 0.976 / 0.930                                       | 0.976 / 0.953                                  | 0.976 / 0.953                                  | 0.976 / 0.953                                  |
| **Precisión**                     | **1.000 / 0.953**        | 0.994 / 0.952                                       | 1.000 / 0.953                                  | 1.000 / 0.953                                  | 1.000 / 0.953                                  |
| **AUC-ROC**                       | **0.998 / 0.995**        | 0.998 / 0.996                                       | 0.997 / 0.996                                  | 0.997 / 0.996                                  | 0.998 / 0.995                                  |
| **Precision-Recall AUC**          | **0.997 / 0.993**        | 0.997 / 0.994                                       | 0.997 / 0.994                                  | 0.997 / 0.994                                  | 0.997 / 0.993                                  |

#### Conclusiones por modelo:
* **Sin Hiperparámetros**: El modelo sin hiperparámetros tiene un rendimiento muy alto, con exactitudes cercanas al 99% en entrenamiento y 96% en testeo. Sin embargo, aunque la precisión y F1 score son excelentes, no hay mucha diferencia entre entrenamiento y testeo, lo que sugiere que el modelo ya está muy ajustado.

* **solver = 'liblinear', max_iter = 200, penalty = 'l1'**: Aquí, se observa una ligera disminución en la exactitud, F1 Score y sensibilidad en los datos de testeo. Este cambio es probablemente debido a la penalización 'l1', que ayuda a hacer el modelo más simple, pero ligeramente menos preciso en testeo.

* **solver = 'lbfgs', max_iter = 200, penalty = 'l2'**: No hay grandes diferencias respecto al caso sin hiperparámetros. Esto se debe a que 'l2' es una penalización más suave, lo que mantiene la capacidad de generalización del modelo sin perder precisión en los datos de testeo.

* **solver = 'saga', max_iter = 300, penalty = 'elasticnet'**: Similar al caso anterior, pero con 'elasticnet' que combina 'l1' y 'l2'. El rendimiento sigue siendo excelente, pero con la ventaja de tener una penalización más equilibrada.

* **solver = 'liblinear', max_iter = 100, penalty = 'l2'**: La disminución en el rendimiento es mínima, aunque al reducir el número de iteraciones a 100, el modelo podría no haber tenido suficiente tiempo para entrenarse de manera óptima.
#### Como en el caso anterior: Me quedaría con el modelo sin hiperparámetros, ya que ofrece el mejor rendimiento general y no presenta grandes sobreajustes.

***

## `4.3 Árboles de Decisión con hiperparámetros`

### Vamos a tener entonces en esta función para aplicar a cada dataset:
### para clasificación:
* DecisionTreeClassifier()  y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)
### para regresión:
* DecisionTreeRegressor() y métricas (MAE, MSE, R2 Score) y añadidas (RMSE y Explained Variance)

In [54]:
# Función para entrenar y evaluar un modelo de Árbol de Decisión con hiperparámetros
def aplicar_arbol_decision_hiper(df, target_column, tipo_modelo='regresion', max_depth=None, min_samples_split=2, min_samples_leaf=1):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = DecisionTreeRegressor(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
    elif tipo_modelo == 'clasificacion':
        model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("Árbol de Decisión", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("Árbol de Decisión", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Árbol de Decisión", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("Árbol de Decisión", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Árbol de Decisión", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Árbol de Decisión", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
        y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("Árbol de Decisión", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("Árbol de Decisión", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

### Vamos a incluir los siguientes hiperparámetros:
* max_depth (Profundidad máxima)
* min_samples_split (Mínimo número de muestras para dividir un nodo)
* min_samples_leaf (Mínimo número de muestras en una hoja)

***

### **max_depth (Profundidad máxima)**: Controla la profundidad máxima de los árboles. Un valor mayor puede llevar a sobreajuste, ya que el árbol se ajustará demasiado a los datos de entrenamiento. Si es más pequeño, el árbol será menos complejo y podría no captar patrones importantes.
### Cuándo usarlo: Se recomienda usar un valor pequeño para evitar el sobreajuste, especialmente en datasets pequeños o cuando hay mucho ruido. Si el modelo es demasiado simple, puede ser necesario incrementar este parámetro.

***

### **min_samples_split (Mínimo número de muestras para dividir un nodo)**: Especifica el número mínimo de muestras que deben estar presentes en un nodo antes de dividirlo. Si el valor es mayor, el árbol será menos profundo y menos susceptible al sobreajuste.
### Cuándo usarlo: Si se tienen datos ruidosos o el modelo está sobreajustando, se puede aumentar este valor. Para datasets más pequeños, un valor bajo permite mayor flexibilidad y potencialmente mejor desempeño en los datos de entrenamiento.

***

### **min_samples_leaf (Mínimo número de muestras en una hoja)**: Controla el número mínimo de muestras que deben estar en cada hoja del árbol. Esto asegura que las hojas no contengan pocos puntos de datos y que el modelo no se ajuste demasiado a las pequeñas variaciones.
### Cuándo usarlo: Es útil cuando se quiere evitar un sobreajuste excesivo, especialmente en datasets pequeños o con pocas muestras. Si se incrementa este valor, el árbol será más general y menos propenso a sobreajustar.

***

### `4.3.1 Árboles de Decisión Aplicado a TEMPERATURA ESPAÑA` (con hiperparámetros)

In [55]:
# Ejemplo de uso para un modelo de regresión con Árbol de Decisión:
print("Sin hiperparámetros")
arbol_modelo_temp = aplicar_arbol_decision(dfesp_limpio, 'rain_days', tipo_modelo='regresion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("max_depth = 5, min_samples_split = 4, min_samples_leaf = 2")
arbol_hiper_esp = aplicar_arbol_decision_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', max_depth=5, min_samples_split=4, min_samples_leaf=2)
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("max_depth = 10, min_samples_split = 5, min_samples_leaf = 1")
arbol_hiper_esp2 = aplicar_arbol_decision_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', max_depth=10, min_samples_split=5, min_samples_leaf=1)
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("max_depth = None (sin límite), min_samples_split = 10, min_samples_leaf = 3")
arbol_hiper_esp3 = aplicar_arbol_decision_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', max_depth=None, min_samples_split=10, min_samples_leaf=3)
print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("max_depth = 3, min_samples_split = 2, min_samples_leaf = 4")
arbol_hiper_esp4 = aplicar_arbol_decision_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', max_depth=3, min_samples_split=2, min_samples_leaf=4)

Sin hiperparámetros
Árbol de Decisión : MAE en los datos de entrenamiento: 0.000
Árbol de Decisión : MAE en los datos de testeo: 0.604

Árbol de Decisión : MSE en los datos de entrenamiento: 0.000
Árbol de Decisión : MSE en los datos de testeo: 0.896

Árbol de Decisión : R2 Score en los datos de entrenamiento: 1.000
Árbol de Decisión : R2 Score en los datos de testeo: 0.902

Árbol de Decisión : RMSE en los datos de entrenamiento: 0.000
Árbol de Decisión : RMSE en los datos de testeo: 0.946

Árbol de Decisión : Explained Variance en los datos de entrenamiento: 1.000
Árbol de Decisión : Explained Variance en los datos de testeo: 0.904

---------------------------------------------------------------------------
PRIMER CASO
max_depth = 5, min_samples_split = 4, min_samples_leaf = 2
Árbol de Decisión : MAE en los datos de entrenamiento: 0.520
Árbol de Decisión : MAE en los datos de testeo: 0.735

Árbol de Decisión : MSE en los datos de entrenamiento: 0.567
Árbol de Decisión : MSE en los dat

#### Ya hemos comentado lo de los datos de entrenamiento a 0.000 para el sin hiperparámetros cuando probamos el modelo sin hiperparámetros en el punto 3.

| **Métrica**                       | **Sin Hiperparámetros**  | **max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**  | **max_depth = 10, min_samples_split = 5, min_samples_leaf = 1** | **max_depth = None, min_samples_split = 10, min_samples_leaf = 3** | **max_depth = 3, min_samples_split = 2, min_samples_leaf = 4** |
|------------------------------------|--------------------------|-------------------------------------------------------------|-------------------------------------------------------------|-----------------------------------------------------------------|--------------------------------------------------------------|
| **MAE**                            | 0.000 / 0.583            | **0.520 / 0.700**                                               | 0.207 / 0.630                                               | 0.385 / 0.636 | 0.776 / 0.886                                                 |
| **MSE**                            | 0.000 / 0.688            | **0.567 / 0.907**                                              | 0.155 / 0.900                                               | 0.321 / 0.845   | 1.036 / 1.377                                                 |
| **R2 Score**                       | 1.000 / 0.925            | **0.941 / 0.901**                                               | 0.984 / 0.902                                               | 0.966 / 0.908    | 0.891 / 0.849                                                 |
| **RMSE**                           | 0.000 / 0.829            | **0.753 / 0.953**                                               | 0.394 / 0.949                                               | 0.566 / 0.919     | 1.018 / 1.174                                                 |
| **Explained Variance**             | 1.000 / 0.929            | **0.941 / 0.902**                                               | 0.984 / 0.905                                               | 0.966 / 0.910    | 0.891 / 0.850                                                 |

#### Comparación por modelo:
* **(max_depth = 5, min_samples_split = 4, min_samples_leaf = 2)**: Aquí el modelo parece balancear bien el ajuste entre el entrenamiento y la prueba, ya que tiene buenos valores de R2 (0.941 en entrenamiento, 0.901 en prueba) y errores razonables. Esto es gracias a los parámetros min_samples_split = 4 y min_samples_leaf = 2 que aseguran que el modelo no se haga demasiado específico o demasiado general. Es una configuración que favorece una buena generalización, haciendo que el modelo se desempeñe de manera consistente tanto en los datos de entrenamiento como en los de prueba.
* **(max_depth = 10, min_samples_split = 5, min_samples_leaf = 1)**: Este modelo tiene un mejor rendimiento en entrenamiento (R2 = 0.984), pero el rendimiento en prueba baja un poco, se está acercando al sobreajuste. esto se debe a que al tener un max_depth alto y min_samples_leaf bajo, se captura más detalle, pero el modelo empieza a ajustarse demasiado a las fluctuaciones de los datos de entrenamiento, lo que afecta la generalización.
* **(max_depth = None, min_samples_split = 10, min_samples_leaf = 3)**: Con máximo de profundidad sin límite, el modelo parece generalizar bastante bien, con un R2 de 0.966 en entrenamiento y 0.908 en prueba, tiene buena capacidad de generalización.  Esto muestra que, aunque el árbol tiene suficiente capacidad para aprender de los datos, al establecer valores más altos en min_samples_split y min_samples_leaf, se asegura que las divisiones sean significativas, lo que ayuda a evitar el overfitting y mejora la generalización.
* **(max_depth = 3, min_samples_split = 2, min_samples_leaf = 4)**: Este modelo muestra bajo rendimiento en los datos de entrenamiento y prueba, con un R2 de 0.891 en entrenamiento y 0.849 en prueba. Los errores también son más altos.
#### Con cuál me quedaría:
* Modelo 1 **(max_depth = 5, min_samples_split = 4, min_samples_leaf = 2)** parece ser el más equilibrado. Tiene buen rendimiento en entrenamiento y prueba, con un R2 razonable y errores moderados y no está sobreajustado.

### `4.3.2 Árboles de Decisión Aplicado a PHISHING URLS` (con hiperparámetros)

In [56]:
# Ejemplo de uso para un modelo de clasificación con Árbol de Decisión:
# Sin hiperparámetros
print("Sin hiperparámetros")
arbol_modelo_ph = aplicar_arbol_decision(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("max_depth = 5, min_samples_split = 4, min_samples_leaf = 2")
arbol_hiper_ph = aplicar_arbol_decision_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', max_depth=5, min_samples_split=4, min_samples_leaf=2)

print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("max_depth = 10, min_samples_split = 5, min_samples_leaf = 1")
arbol_hiper_ph2 = aplicar_arbol_decision_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', max_depth=10, min_samples_split=5, min_samples_leaf=1)

print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("max_depth = None (sin límite), min_samples_split = 10, min_samples_leaf = 3")
arbol_hiper_ph3 = aplicar_arbol_decision_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', max_depth=None, min_samples_split=10, min_samples_leaf=3)

print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("max_depth = 3, min_samples_split = 2, min_samples_leaf = 4")
arbol_hiper_ph4 = aplicar_arbol_decision_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', max_depth=3, min_samples_split=2, min_samples_leaf=4)

Sin hiperparámetros
Árbol de Decisión : Exactitud en los datos de entrenamiento: 0.991
Árbol de Decisión : Exactitud en los datos de testeo: 0.960

Árbol de Decisión : F1 Score en los datos de entrenamiento: 0.992
Árbol de Decisión : F1 Score en los datos de testeo: 0.964

Árbol de Decisión : Sensibilidad en los datos de entrenamiento: 0.991
Árbol de Decisión : Sensibilidad en los datos de testeo: 0.962

Árbol de Decisión : Precisión en los datos de entrenamiento: 0.993
Árbol de Decisión : Precisión en los datos de testeo: 0.967

Árbol de Decisión : AUC-ROC en los datos de entrenamiento: 1.000
Árbol de Decisión : AUC-ROC en los datos de testeo: 0.972

Árbol de Decisión : Precision-Recall AUC en los datos de entrenamiento: 1.000
Árbol de Decisión : Precision-Recall AUC en los datos de testeo: 0.966

---------------------------------------------------------------------------
PRIMER CASO
max_depth = 5, min_samples_split = 4, min_samples_leaf = 2
Árbol de Decisión : Exactitud en los datos 

| **Métrica**                       | **Sin Hiperparámetros**  | **max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**  | **max_depth = 10, min_samples_split = 5, min_samples_leaf = 1** | **max_depth = None, min_samples_split = 10, min_samples_leaf = 3** | **max_depth = 3, min_samples_split = 2, min_samples_leaf = 4** |
|------------------------------------|--------------------------|-------------------------------------------------------------|-------------------------------------------------------------|-----------------------------------------------------------------|--------------------------------------------------------------|
| **Exactitud**                      | 0.991 / 0.960            | **0.922 / 0.927**                                               | 0.958 / 0.950                                               | 0.973 / 0.952        | 0.907 / 0.910                                                 |
| **F1 Score**                       | 0.992 / 0.964            | **0.932 / 0.937**                                              | 0.962 / 0.955                                               | 0.976 / 0.957         | 0.921 / 0.924                                                 |
| **Sensibilidad**                   | 0.991 / 0.964            | **0.972 / 0.975**                                              | 0.958 / 0.953                                               | 0.978 / 0.959         | 0.978 / 0.978                                                 |
| **Precisión**                      | 0.993 / 0.964            | **0.896 / 0.903**                                              | 0.967 / 0.958                                               | 0.974 / 0.956         | 0.871 / 0.875                                                 |
| **AUC-ROC**                        | 1.000 / 0.972            | **0.978 / 0.977**                                              | 0.995 / 0.984                                               | 0.998 / 0.982         | 0.957 / 0.962                                                 |
| **Precision-Recall AUC**           | 1.000 / 0.966            | **0.977 / 0.975**                                              | 0.996 / 0.983                                               | 0.998 / 0.979         | 0.942 / 0.948                                                 |

#### Conclusiones de cada modelo:
* **Sin Hiperparámetros**: El modelo tiene un excelente rendimiento tanto en entrenamiento como en prueba, con altas métricas. La falta de hiperparámetros ajustados puede llevar a sobreajuste, especialmente en el conjunto de datos de prueba.
* **max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**: Buen equilibrio entre ajuste y generalización, con métricas estables tanto en entrenamiento como en prueba.
Los valores moderados de estos hiperparámetros permiten que el árbol no se vuelva demasiado complejo, evitando el sobreajuste.
* **max_depth = 10, min_samples_split = 5, min_samples_leaf = 1**: Buen rendimiento en entrenamiento, pero podría estar acercándose al sobreajuste con valores de prueba ligeramente más bajos.
Los hiperparámetros con mayor profundidad y menor número de muestras por nodo permiten una mayor complejidad, pero también pueden causar que el modelo aprenda demasiado de los datos de entrenamiento.
* **max_depth = None, min_samples_split = 10, min_samples_leaf = 3**: El modelo tiene buen rendimiento en los datos de prueba, con una buena capacidad de generalización.
Sin límite de profundidad, el modelo puede capturar más patrones, pero los valores de mínimo de muestras y hojas ayudan a mantener el control, evitando el sobreajuste.
* **max_depth = 3, min_samples_split = 2, min_samples_leaf = 4:** Bajo rendimiento tanto en entrenamiento como en prueba, con métricas más bajas. El límite de profundidad pequeño y los pocos datos mínimos por nodo hacen que el modelo sea demasiado simple, resultando en un ajuste insuficiente (underfitting).
####  Me quedaría con el modelo con **max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**, ya que ofrece un buen balance entre generalización y ajuste, con métricas altas en ambos conjuntos de datos. Los hiperparámetros permiten evitar tanto el sobreajuste como el underfitting.

### `4.3.3 Árboles de Decisión Aplicado a CALIDAD DEL VINO` (con hiperparámetros)

In [57]:
# Ejemplo de uso para un modelo de clasificación con Árbol de Decisión:

# Sin hiperparámetros
print("Sin hiperparámetros")
arbol_model_vino = aplicar_arbol_decision(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("max_depth = 5, min_samples_split = 4, min_samples_leaf = 2")
arbol_hiper_vino = aplicar_arbol_decision_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', max_depth=5, min_samples_split=4, min_samples_leaf=2)

print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("max_depth = 10, min_samples_split = 5, min_samples_leaf = 1")
arbol_hiper_vino2 = aplicar_arbol_decision_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', max_depth=10, min_samples_split=5, min_samples_leaf=1)

print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("max_depth = None (sin límite), min_samples_split = 10, min_samples_leaf = 3")
arbol_hiper_vino3 = aplicar_arbol_decision_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', max_depth=None, min_samples_split=10, min_samples_leaf=3)

print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("max_depth = 3, min_samples_split = 2, min_samples_leaf = 4")
arbol_hiper_vino4 = aplicar_arbol_decision_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', max_depth=3, min_samples_split=2, min_samples_leaf=4)

Sin hiperparámetros
Árbol de Decisión : Exactitud en los datos de entrenamiento: 1.000
Árbol de Decisión : Exactitud en los datos de testeo: 0.686

Árbol de Decisión : F1 Score en los datos de entrenamiento: 1.000
Árbol de Decisión : F1 Score en los datos de testeo: 0.719

Árbol de Decisión : Sensibilidad en los datos de entrenamiento: 1.000
Árbol de Decisión : Sensibilidad en los datos de testeo: 0.724

Árbol de Decisión : Precisión en los datos de entrenamiento: 1.000
Árbol de Decisión : Precisión en los datos de testeo: 0.713

Árbol de Decisión : AUC-ROC en los datos de entrenamiento: 1.000
Árbol de Decisión : AUC-ROC en los datos de testeo: 0.681

Árbol de Decisión : Precision-Recall AUC en los datos de entrenamiento: 1.000
Árbol de Decisión : Precision-Recall AUC en los datos de testeo: 0.669

---------------------------------------------------------------------------
PRIMER CASO
max_depth = 5, min_samples_split = 4, min_samples_leaf = 2
Árbol de Decisión : Exactitud en los datos 

| **Métrica**                       | **Sin Hiperparámetros**  | **max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**  | **max_depth = 10, min_samples_split = 5, min_samples_leaf = 1** | **max_depth = None, min_samples_split = 10, min_samples_leaf = 3** | **max_depth = 3, min_samples_split = 2, min_samples_leaf = 4** |
|------------------------------------|--------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|
| **Exactitud**                      | 1.000 / 0.694            | **0.821 / 0.742**                                                 | 0.950 / 0.672                                                 | 0.909 / 0.721                                                 | 0.753 / 0.729                                                 |
| **F1 Score**                       | 1.000 / 0.724            | **0.836 / 0.761**                                                 | 0.953 / 0.699                                                 | 0.916 / 0.733                                                 | 0.741 / 0.718                                                 |
| **Sensibilidad**                   | 1.000 / 0.724            | **0.846 / 0.740**                                                 | 0.935 / 0.685                                                 | 0.911 / 0.693                                                 | 0.654 / 0.622                                                 |
| **Precisión**                      | 1.000 / 0.724            | **0.826 / 0.783**                                                 | 0.971 / 0.713                                                 | 0.920 / 0.779                                                 | 0.854 / 0.849                                                 |
| **AUC-ROC**                        | 1.000 / 0.691            | **0.892 / 0.777**                                              | 0.992 / 0.672                                                 | 0.979 / 0.761                                                 | 0.804 / 0.778                                                 |
| **Precision-Recall AUC**           | 1.000 / 0.678            | **0.886 / 0.798**                                                 | 0.992 / 0.662                                                 | 0.980 / 0.764                                                 | 0.792 / 0.761                                                 |

#### Conclusiones de cada modelo:

* **Sin Hiperparámetros**:  El modelo tiene un rendimiento perfecto en entrenamiento pero un desempeño mucho menor en testeo, lo que indica **sobreajuste severo**. Al no restringir la complejidad del árbol, aprende demasiado bien los datos de entrenamiento, perdiendo capacidad de generalización.  

* **max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**:  **Tiene el mejor equilibrio entre ajuste y generalización**. Mantiene métricas altas en ambos conjuntos de datos y evita sobreajuste.  Al limitar la profundidad y establecer un mínimo de muestras por nodo, el modelo captura patrones sin memorizar datos específicos.  

* **max_depth = 10, min_samples_split = 5, min_samples_leaf = 1**: Alto rendimiento en entrenamiento, pero **se acerca al sobreajuste**, con una caída notable en testeo.  La mayor profundidad y el menor tamaño de hojas permiten aprender más detalles, pero reducen la capacidad de generalización.  

* **max_depth = None, min_samples_split = 10, min_samples_leaf = 3**: **Buen rendimiento en testeo**, con una generalización aceptable. Sin un límite de profundidad, el modelo puede capturar más patrones, pero el uso de valores más altos en `min_samples_split` y `min_samples_leaf` ayuda a controlar el sobreajuste.  

* **max_depth = 3, min_samples_split = 2, min_samples_leaf = 4**:  **Ocurre subajuste (underfitting)** claro, con métricas bajas en ambos conjuntos de datos.  El límite de profundidad restringe demasiado el modelo, haciéndolo incapaz de capturar patrones complejos.  

#### **Modelo elegido**: Me quedaría con el modelo **max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**, ya que ofrece **un balance óptimo** entre generalización y ajuste.  Sus hiperparámetros permiten evitar tanto el **sobreajuste** como el **subajuste**, sin caer en una complejidad innecesaria.  

### `4.3.4 Árboles de Decisión Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (con hiperparámetros)

In [58]:
# Aplicar Árboles de Decisión al dataset dfcancer_limpio

# Sin hiperparámetros
print("Sin hiperparámetros")
arbol_model_cancer = aplicar_arbol_decision(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("max_depth = 5, min_samples_split = 4, min_samples_leaf = 2")
arbol_hiper_cancer = aplicar_arbol_decision_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', max_depth=5, min_samples_split=4, min_samples_leaf=2)

print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("max_depth = 10, min_samples_split = 5, min_samples_leaf = 1")
arbol_hiper_cancer2 = aplicar_arbol_decision_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', max_depth=10, min_samples_split=5, min_samples_leaf=1)

print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("max_depth = None (sin límite), min_samples_split = 10, min_samples_leaf = 3")
arbol_hiper_cancer3 = aplicar_arbol_decision_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', max_depth=None, min_samples_split=10, min_samples_leaf=3)

print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("max_depth = 3, min_samples_split = 2, min_samples_leaf = 4")
arbol_hiper_cancer4 = aplicar_arbol_decision_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', max_depth=3, min_samples_split=2, min_samples_leaf=4)


Sin hiperparámetros
Árbol de Decisión : Exactitud en los datos de entrenamiento: 1.000
Árbol de Decisión : Exactitud en los datos de testeo: 0.921

Árbol de Decisión : F1 Score en los datos de entrenamiento: 1.000
Árbol de Decisión : F1 Score en los datos de testeo: 0.897

Árbol de Decisión : Sensibilidad en los datos de entrenamiento: 1.000
Árbol de Decisión : Sensibilidad en los datos de testeo: 0.907

Árbol de Decisión : Precisión en los datos de entrenamiento: 1.000
Árbol de Decisión : Precisión en los datos de testeo: 0.886

Árbol de Decisión : AUC-ROC en los datos de entrenamiento: 1.000
Árbol de Decisión : AUC-ROC en los datos de testeo: 0.918

Árbol de Decisión : Precision-Recall AUC en los datos de entrenamiento: 1.000
Árbol de Decisión : Precision-Recall AUC en los datos de testeo: 0.839

---------------------------------------------------------------------------
PRIMER CASO
max_depth = 5, min_samples_split = 4, min_samples_leaf = 2
Árbol de Decisión : Exactitud en los datos 

| **Métrica**                       | **Sin Hiperparámetros**  | **max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**  | **max_depth = 10, min_samples_split = 5, min_samples_leaf = 1** | **max_depth = None, min_samples_split = 10, min_samples_leaf = 3** | **max_depth = 3, min_samples_split = 2, min_samples_leaf = 4** |
|------------------------------------|--------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|
| **Exactitud**                      | 1.000 / 0.921            | 1.000 / 0.921                                                 | 0.998 / 0.930                                             | **0.991 / 0.930**        | 0.965 / 0.912                                                 |
| **F1 Score**                       | 1.000 / 0.897            | 1.000 / 0.897                                                 | 0.997 / 0.907                                             | **0.988 / 0.907**        | 0.953 / 0.884                                                 |
| **Sensibilidad**                   | 1.000 / 0.907            | 1.000 / 0.907                                                 | 0.994 / 0.907                                             | **0.982 / 0.907**        | 0.959 / 0.884                                                 |
| **Precisión**                      | 1.000 / 0.886            | 1.000 / 0.886                                                 | 1.000 / 0.907                                             | **0.994 / 0.907**        | 0.947 / 0.884                                                 |
| **AUC-ROC**                        | 1.000 / 0.918            | 1.000 / 0.918                                                 | 1.000 / 0.925                                                 | **1.000 / 0.935**    | 0.987 / 0.928                                                 |
| **Precision-Recall AUC**           | 1.000 / 0.839            | 1.000 / 0.839                                                 | 1.000 / 0.858                                                 | **1.000 / 0.870**    | 0.978 / 0.907                                                 |


#### Conclusiones de cada modelo:
* **Sin Hiperparámetros**: El modelo alcanza un rendimiento perfecto en los datos de entrenamiento, pero su rendimiento en prueba es menor. Esto indica un posible sobreajuste, ya que el modelo memoriza los datos de entrenamiento y no generaliza tan bien en el conjunto de testeo.
  
* **max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**: Presenta un rendimiento idéntico al modelo sin hiperparámetros, lo que sugiere que la profundidad moderada y los criterios de división evitan un sobreajuste mayor, pero aún podrían mejorarse para lograr mejor generalización.
  
* **max_depth = 10, min_samples_split = 5, min_samples_leaf = 1**: Se observa un buen equilibrio entre ajuste y generalización. Su desempeño en prueba mejora ligeramente en comparación con los modelos anteriores, con una mejor precisión y F1 Score. Sin embargo, la sensibilidad es similar, lo que indica que sigue aprendiendo características específicas de los datos de entrenamiento.

* **max_depth = None, min_samples_split = 10, min_samples_leaf = 3**: Este modelo logra el mejor **AUC-ROC y Precision-Recall AUC** en los datos de prueba, lo que sugiere que tiene una mejor capacidad de discriminación. La restricción en el número mínimo de muestras por división y hoja parece contribuir a una generalización más estable sin comprometer demasiado el rendimiento.

* **max_depth = 3, min_samples_split = 2, min_samples_leaf = 4**: Presenta el peor rendimiento general en comparación con los otros modelos, lo que indica que la profundidad limitada y los valores conservadores de los hiperparámetros impiden que el modelo capture patrones importantes. Esto resulta en **underfitting**, con una caída en exactitud y F1 Score.

#### Modelo recomendado:
Me quedaría con el modelo con **max_depth = None, min_samples_split = 10, min_samples_leaf = 3**, ya que logra el mejor balance entre ajuste y generalización. Sus métricas en el conjunto de prueba, especialmente **AUC-ROC (0.935) y Precision-Recall AUC (0.870),** indican que tiene la mejor capacidad para distinguir entre clases y manejar datos desequilibrados. Además, evita tanto el sobreajuste como el underfitting en comparación con los otros modelos.

***

## `4.4 Bosques Aleatorios (Random Forest) con hiperparámetros`

### Vamos a tener entonces en esta función para aplicar a cada dataset:
### para clasificación:
* DecisionTreeClassifier()  y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)
### para regresión:
* DecisionTreeRegressor() y métricas (MAE, MSE, R2 Score) y añadidas (RMSE y Explained Variance)

In [59]:
# Función para entrenar y evaluar un modelo de Bosques Aleatorios (Random Forest) con hiperparámetros
def aplicar_random_forest_hiper(df, target_column, tipo_modelo='clasificacion', n_estimators=100, max_depth=None, min_samples_split=2):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = RandomForestRegressor(
            n_estimators=n_estimators, 
            max_depth=max_depth, 
            min_samples_split=min_samples_split,
            random_state=42
        )
    elif tipo_modelo == 'clasificacion':
        model = RandomForestClassifier(
            n_estimators=n_estimators, 
            max_depth=max_depth, 
            min_samples_split=min_samples_split,
            random_state=42
        )

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("Bosques Aleatorios", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("Bosques Aleatorios", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("Bosques Aleatorios", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
        y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("Bosques Aleatorios", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("Bosques Aleatorios", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

#### Vamos a incluir los siguientes hiperparámetros:
* n_estimators
* max_depth
* min_samples_split

* **n_estimators**: El número de árboles en el bosque. Valor sugerido: 100 - 500 árboles. Tendremos que pensar en aumentar el número de árboles o no porque generalmente mejora el rendimiento, pero también aumenta el tiempo de computación.
* **max_depth**: La profundidad máxima de cada árbol en el bosque. Dependiendo del tamaño y complejidad de los datos, entre 5 y 20 hace que la profundidad previene el sobreajuste.
* **min_samples_split**: El número mínimo de muestras necesarias para dividir un nodo. Valor sugerido: 2 - 10. Valores más altos hacen que los árboles sean más simples, lo que puede ayudar a reducir el sobreajuste.

### `4.4.1 Bosques Aleatorios Aplicado a TEMPERATURA ESPAÑA` (con hiperparámetros)

In [60]:
# Ejemplo de uso para un modelo de regresión con Bosques Aleatorios:
print("Sin hiperparámetros")
Bosques_modelo_temp = aplicar_random_forest(dfesp_limpio, 'rain_days', tipo_modelo='regresion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("n_estimators = 100, max_depth = 5, min_samples_split = 4")
Bosques_hiper_esp = aplicar_random_forest_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', n_estimators=100, max_depth=5, min_samples_split=4)
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("n_estimators = 150, max_depth = 10, min_samples_split = 5")
Bosques_hiper_esp2 = aplicar_random_forest_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', n_estimators=150, max_depth=10, min_samples_split=5)
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("n_estimators = 200, max_depth = None (sin límite), min_samples_split = 10")
Bosques_hiper_esp3 = aplicar_random_forest_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', n_estimators=200, max_depth=None, min_samples_split=10)
print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("n_estimators = 50, max_depth = 3, min_samples_split = 2")
Bosques_hiper_esp4 = aplicar_random_forest_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', n_estimators=50, max_depth=3, min_samples_split=2)


Sin hiperparámetros
Bosques Aleatorios : MAE en los datos de entrenamiento: 0.192
Bosques Aleatorios : MAE en los datos de testeo: 0.596

Bosques Aleatorios : MSE en los datos de entrenamiento: 0.083
Bosques Aleatorios : MSE en los datos de testeo: 0.950

Bosques Aleatorios : R2 Score en los datos de entrenamiento: 0.991
Bosques Aleatorios : R2 Score en los datos de testeo: 0.896

Bosques Aleatorios : RMSE en los datos de entrenamiento: 0.288
Bosques Aleatorios : RMSE en los datos de testeo: 0.975

Bosques Aleatorios : Explained Variance en los datos de entrenamiento: 0.991
Bosques Aleatorios : Explained Variance en los datos de testeo: 0.900

---------------------------------------------------------------------------
PRIMER CASO
n_estimators = 100, max_depth = 5, min_samples_split = 4
Bosques Aleatorios : MAE en los datos de entrenamiento: 0.469
Bosques Aleatorios : MAE en los datos de testeo: 0.669

Bosques Aleatorios : MSE en los datos de entrenamiento: 0.465
Bosques Aleatorios : MS

| **Métrica**                       | **Sin Hiperparámetros**  | **n_estimators = 100, max_depth = 5, min_samples_split = 4**  | **n_estimators = 150, max_depth = 10, min_samples_split = 5** | **n_estimators = 200, max_depth = None, min_samples_split = 10** | **n_estimators = 50, max_depth = 3, min_samples_split = 2** |
|------------------------------------|--------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-------------------------------------------------------------|
| **MAE**                            | 0.192 / 0.596            | 0.469 / 0.669                                                 | **0.276 / 0.613**                                                 | 0.307 / 0.630        | 0.708 / 0.838                                                 |
| **MSE**                            | 0.083 / 0.950            | 0.465 / 1.115                                                 | **0.159 / 0.962**                                                 | 0.200 / 1.018        | 0.872 / 1.310                                                 |
| **R2 Score**                       | 0.991 / 0.896            | 0.951 / 0.878                                                 | **0.983 / 0.895**                                                 | 0.979 / 0.889        | 0.909 / 0.857                                                 |
| **RMSE**                           | 0.288 / 0.975            | 0.682 / 1.056                                                 | **0.399 / 0.981**                                                 | 0.447 / 1.009        | 0.934 / 1.145                                                 |
| **Explained Variance**             | 0.991 / 0.900            | 0.951 / 0.882                                                 | **0.983 / 0.898**                                                 | 0.979 / 0.893        | 0.909 / 0.857                                                 |


#### Conclusiones de cada modelo:
* **Sin Hiperparámetros:** El modelo tiene buen rendimiento en los datos de entrenamiento, pero pierde precisión en los datos de prueba, lo que sugiere sobreajuste.
* **n_estimators = 100, max_depth = 5, min_samples_split = 4**: Mejora ligeramente respecto al modelo sin hiperparámetros, pero aún no es óptimo. Hay algo de sobreajuste, pero menos que en el modelo base.
* **n_estimators = 150, max_depth = 10, min_samples_split = 5**: Buen equilibrio entre los datos de entrenamiento y prueba, con mejoras en el ajuste y la generalización.
* **n_estimators = 200, max_depth = None, min_samples_split = 10**: Con n_estimators = 200 y max_depth = None, el modelo puede capturar mucha más complejidad sin perder capacidad de generalización, lo que se traduce en el mejor rendimiento en el conjunto de prueba. Aunque max_depth = None permite una profundidad ilimitada, lo cual podría generar un riesgo de sobreajuste si no se controla bien, la combinación de min_samples_split = 10 ayuda a hacer que las divisiones sean más simples y reduce el riesgo de sobreajuste. Este modelo logra un buen balance con los hiperparámetros que hemos escogido.
* **n_estimators = 50, max_depth = 3, min_samples_split = 2**: El peor modelo, con un rendimiento pobre en prueba debido a underfitting.
#### El modelo con **n_estimators = 200, max_depth = None, min_samples_split = 10** es el mejor, ya que tiene un buen balance, logrando buenos resultados en los datos de prueba sin sobreajustarse ni caer en underfitting.

### `4.4.2 Bosques Aleatorios Aplicado a PHISHING URLS` (con hiperparámetros)

In [61]:
# Ejemplo de uso para un modelo de clasificación con Bosques Aleatorios:
print("Sin hiperparámetros")
Bosques_modelphishing = aplicar_random_forest(dfphishing_limpio, 'class', tipo_modelo='clasificacion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("n_estimators = 120, max_depth = 7, min_samples_split = 6")
Bosques_hiper_phis = aplicar_random_forest_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', n_estimators=120, max_depth=7, min_samples_split=6)
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("n_estimators = 180, max_depth = 12, min_samples_split = 8")
Bosques_hiper_phis2 = aplicar_random_forest_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', n_estimators=180, max_depth=12, min_samples_split=8)
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("n_estimators = 250, max_depth = None (sin límite), min_samples_split = 15")
Bosques_hiper_phis3 = aplicar_random_forest_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', n_estimators=250, max_depth=None, min_samples_split=15)
print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("n_estimators = 60, max_depth = 4, min_samples_split = 3")
Bosques_hiper_phis4 = aplicar_random_forest_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', n_estimators=60, max_depth=4, min_samples_split=3)

Sin hiperparámetros
Bosques Aleatorios : Exactitud en los datos de entrenamiento: 0.991
Bosques Aleatorios : Exactitud en los datos de testeo: 0.969

Bosques Aleatorios : F1 Score en los datos de entrenamiento: 0.992
Bosques Aleatorios : F1 Score en los datos de testeo: 0.973

Bosques Aleatorios : Sensibilidad en los datos de entrenamiento: 0.993
Bosques Aleatorios : Sensibilidad en los datos de testeo: 0.977

Bosques Aleatorios : Precisión en los datos de entrenamiento: 0.991
Bosques Aleatorios : Precisión en los datos de testeo: 0.969

Bosques Aleatorios : AUC-ROC en los datos de entrenamiento: 1.000
Bosques Aleatorios : AUC-ROC en los datos de testeo: 0.995

Bosques Aleatorios : Precision-Recall AUC en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precision-Recall AUC en los datos de testeo: 0.995

---------------------------------------------------------------------------
PRIMER CASO
n_estimators = 120, max_depth = 7, min_samples_split = 6
Bosques Aleatorios : Exactitud en

| **Métrica**                       | **Sin Hiperparámetros**  | **n_estimators = 120, max_depth = 7, min_samples_split = 6** | **n_estimators = 180, max_depth = 12, min_samples_split = 8** | **n_estimators = 250, max_depth = None, min_samples_split = 15** | **n_estimators = 60, max_depth = 4, min_samples_split = 3** |
|------------------------------------|--------------------------|-------------------------------------------------------------|-------------------------------------------------------------|----------------------------------------------------------------|------------------------------------------------------------|
| **Exactitud**                      | 0.991 / 0.969            | 0.941 / 0.943                                               | **0.968 / 0.962**                                               | 0.975 / 0.963       | 0.925 / 0.933                                              |
| **F1 Score**                       | 0.992 / 0.973            | 0.948 / 0.950                                               | **0.972 / 0.967**                                               | 0.978 / 0.967       | 0.934 / 0.941                                              |
| **Sensibilidad**                   | 0.993 / 0.977            | 0.971 / 0.970                                               | **0.982 / 0.978**                                               | 0.984 / 0.973       | 0.950 / 0.955                                              |
| **Precisión**                      | 0.991 / 0.969            | 0.927 / 0.931                                               | **0.962 / 0.956**                                               | 0.972 / 0.962       | 0.918 / 0.926                                              |
| **AUC-ROC**                        | 1.000 / 0.995            | 0.989 / 0.989                                               | **0.997 / 0.995**                                               | 0.998 / 0.995       | 0.981 / 0.982                                              |
| **Precision-Recall AUC**           | 1.000 / 0.995            | 0.991 / 0.991                                               | **0.997 / 0.996**                                               | 0.998 / 0.996       | 0.984 / 0.985                                              |


#### Conclusiones de cada modelo:
* **Sin Hiperparámetros**: El modelo tiene un rendimiento sólido en entrenamiento, pero muestra una caída en precisión en los datos de prueba, lo que indica sobreajuste.
* **n_estimators = 120, max_depth = 7, min_samples_split = 6**: Este modelo mejora respecto al base, mostrando un buen rendimiento en los datos de prueba, con un mejor equilibrio, aunque todavía hay un pequeño sobreajuste en entrenamiento.
* **n_estimators = 180, max_depth = 12, min_samples_split = 8**: Este modelo presenta un buen equilibrio entre entrenamiento y prueba. Si bien mejora la precisión y la sensibilidad, sigue existiendo un leve sobreajuste, pero menos pronunciado que en otros modelos.
* **n_estimators = 250, max_depth = None, min_samples_split = 15**: Con un mayor número de estimadores y sin límite en la profundidad, el modelo captura mayor complejidad, pero logra un buen equilibrio en la prueba, sin perder capacidad de generalización. Es uno de los mejores, aunque el ajuste de los hiperparámetros podría seguir optimizándose.
* **n_estimators = 60, max_depth = 4, min_samples_split = 3**: Este modelo muestra el peor rendimiento, especialmente en los datos de entrenamiento. Hay un claro caso de underfitting, ya que no logra ajustarse correctamente a los datos.
#### El modelo con n_estimators = 250, max_depth = None, min_samples_split = 15 destaca por ofrecer el mejor rendimiento en prueba, manteniendo un equilibrio adecuado.

### `4.4.3 Bosques Aleatorios Aplicado a CALIDAD DEL VINO` (con hiperparámetros)

In [62]:
# Ejemplo de uso para un modelo de clasificación con Bosques Aleatorios:
print("Sin hiperparámetros")
Bosques_modelvino = aplicar_random_forest(dfvino_limpio, 'quality', tipo_modelo='clasificacion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("n_estimators = 110, max_depth = 8, min_samples_split = 5")
Bosques_hiper_vino = aplicar_random_forest_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', n_estimators=110, max_depth=8, min_samples_split=5)
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("n_estimators = 160, max_depth = 9, min_samples_split = 7")
Bosques_hiper_vino2 = aplicar_random_forest_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', n_estimators=160, max_depth=9, min_samples_split=7)
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("n_estimators = 220, max_depth = None (sin límite), min_samples_split = 12")
Bosques_hiper_vino3 = aplicar_random_forest_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', n_estimators=220, max_depth=None, min_samples_split=12)
print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("n_estimators = 80, max_depth = 5, min_samples_split = 4")
Bosques_hiper_vino4 = aplicar_random_forest_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', n_estimators=80, max_depth=5, min_samples_split=4)

Sin hiperparámetros
Bosques Aleatorios : Exactitud en los datos de entrenamiento: 1.000
Bosques Aleatorios : Exactitud en los datos de testeo: 0.769

Bosques Aleatorios : F1 Score en los datos de entrenamiento: 1.000
Bosques Aleatorios : F1 Score en los datos de testeo: 0.789

Bosques Aleatorios : Sensibilidad en los datos de entrenamiento: 1.000
Bosques Aleatorios : Sensibilidad en los datos de testeo: 0.780

Bosques Aleatorios : Precisión en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precisión en los datos de testeo: 0.798

Bosques Aleatorios : AUC-ROC en los datos de entrenamiento: 1.000
Bosques Aleatorios : AUC-ROC en los datos de testeo: 0.886

Bosques Aleatorios : Precision-Recall AUC en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precision-Recall AUC en los datos de testeo: 0.915

---------------------------------------------------------------------------
PRIMER CASO
n_estimators = 110, max_depth = 8, min_samples_split = 5
Bosques Aleatorios : Exactitud en

| **Métrica**                       | **Sin Hiperparámetros**     | **n_estimators = 110, max_depth = 8, min_samples_split = 5** | **n_estimators = 160, max_depth = 9, min_samples_split = 7** | **n_estimators = 220, max_depth = None, min_samples_split = 12** | **n_estimators = 80, max_depth = 5, min_samples_split = 4** |
|------------------------------------|-----------------------------|------------------------------------------------------------|------------------------------------------------------------|-----------------------------------------------------------------|------------------------------------------------------------|
| **Exactitud**                      | 1.000 / 0.769               | 0.931 / 0.764                                              | 0.940 / 0.790                                              | **0.946 / 0.786**            | 0.835 / 0.760                                               |
| **F1 Score**                       | 1.000 / 0.789               | 0.936 / 0.782                                              | 0.945 / 0.808                                              | **0.951 / 0.806**            | 0.844 / 0.776                                               |
| **Sensibilidad**                   | 1.000 / 0.780               | 0.931 / 0.764                                              | 0.949 / 0.795                                              | **0.960 / 0.803**            | 0.824 / 0.748                                               |
| **Precisión**                      | 1.000 / 0.798               | 0.941 / 0.802                                              | 0.940 / 0.821                                              | **0.942 / 0.810**            | 0.864 / 0.805                                               |
| **AUC-ROC**                        | 1.000 / 0.886               | 0.988 / 0.871                                              | 0.992 / 0.875                                              | **0.992 / 0.870**            | 0.921 / 0.869                                               |
| **Precision-Recall AUC**           | 1.000 / 0.915               | 0.990 / 0.902                                              | 0.993 / 0.905                                              | **0.993 / 0.901**            | 0.934 / 0.896                                               |


#### Conclusiones para cada modelo:
* **Sin Hiperparámetros**: El modelo tiene un rendimiento sólido en entrenamiento, pero muestra una caída en precisión en los datos de prueba, lo que indica sobreajuste.
* **n_estimators = 110, max_depth = 8, min_samples_split = 5**: Este modelo tiene una ligera mejora respecto al modelo base, con un buen rendimiento en los datos de prueba, pero aún presenta un pequeño sobreajuste en los datos de entrenamiento.
* **n_estimators = 160, max_depth = 9, min_samples_split = 7**: El modelo ofrece un buen equilibrio entre los datos de entrenamiento y prueba, mejorando la precisión y la sensibilidad, aunque aún existe algo de sobreajuste, pero es menos pronunciado que en otros modelos.
* **n_estimators = 220, max_depth = None, min_samples_split = 12**: Al tener un mayor número de estimadores y sin límite en la profundidad, el modelo captura más complejidad sin sacrificar la capacidad de generalización. Este es uno de los mejores modelos, aunque aún se pueden seguir ajustando los hiperparámetros.
* **n_estimators = 80, max_depth = 5, min_samples_split = 4**: Este modelo es el que tiene el peor rendimiento, especialmente en los datos de entrenamiento. Es un claro caso de underfitting, ya que no se ajusta bien a los datos.
#### El modelo con n_estimators = 220, max_depth = None, min_samples_split = 12 es el mejor, ya que ofrece un buen rendimiento en los datos de prueba, manteniendo un equilibrio adecuado.

### `4.4.4 Bosques Aleatorios Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (con hiperparámetros)

In [63]:
# Ejemplo de uso para un modelo de clasificación con Bosques Aleatorios:
print("Sin hiperparámetros")
Bosques_modelcancer = aplicar_random_forest(dfcancer_limpio, 'y', tipo_modelo='clasificacion')
print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("n_estimators = 100, max_depth = 6, min_samples_split = 4")
Bosques_hiper_cancer = aplicar_random_forest_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', n_estimators=100, max_depth=6, min_samples_split=4)
print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("n_estimators = 150, max_depth = 10, min_samples_split = 6")
Bosques_hiper_cancer2 = aplicar_random_forest_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', n_estimators=150, max_depth=10, min_samples_split=6)
print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("n_estimators = 200, max_depth = None (sin límite), min_samples_split = 8")
Bosques_hiper_cancer3 = aplicar_random_forest_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', n_estimators=200, max_depth=None, min_samples_split=8)
print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("n_estimators = 70, max_depth = 4, min_samples_split = 3")
Bosques_hiper_cancer4 = aplicar_random_forest_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', n_estimators=70, max_depth=4, min_samples_split=3)

Sin hiperparámetros
Bosques Aleatorios : Exactitud en los datos de entrenamiento: 1.000
Bosques Aleatorios : Exactitud en los datos de testeo: 0.956

Bosques Aleatorios : F1 Score en los datos de entrenamiento: 1.000
Bosques Aleatorios : F1 Score en los datos de testeo: 0.940

Bosques Aleatorios : Sensibilidad en los datos de entrenamiento: 1.000
Bosques Aleatorios : Sensibilidad en los datos de testeo: 0.907

Bosques Aleatorios : Precisión en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precisión en los datos de testeo: 0.975

Bosques Aleatorios : AUC-ROC en los datos de entrenamiento: 1.000
Bosques Aleatorios : AUC-ROC en los datos de testeo: 0.992

Bosques Aleatorios : Precision-Recall AUC en los datos de entrenamiento: 1.000
Bosques Aleatorios : Precision-Recall AUC en los datos de testeo: 0.988

---------------------------------------------------------------------------
PRIMER CASO
n_estimators = 100, max_depth = 6, min_samples_split = 4
Bosques Aleatorios : Exactitud en

| **Métrica**                       | **Sin Hiperparámetros**     | **n_estimators = 100, max_depth = 6, min_samples_split = 4** | **n_estimators = 150, max_depth = 10, min_samples_split = 6** | **n_estimators = 200, max_depth = None, min_samples_split = 8** | **n_estimators = 70, max_depth = 4, min_samples_split = 3** |
|------------------------------------|-----------------------------|------------------------------------------------------------|------------------------------------------------------------|-----------------------------------------------------------------|------------------------------------------------------------|
| **Exactitud**                      | 1.000 / 0.956               | 0.996 / 0.939                                              | 0.998 / 0.947                                              | 0.993 / 0.939                                                 | 0.989 / 0.947                                              |
| **F1 Score**                       | 1.000 / 0.940               | 0.994 / 0.916                                              | 0.997 / 0.929                                              | 0.991 / 0.916                                                 | 0.985 / 0.927                                              |
| **Sensibilidad**                   | 1.000 / 0.907               | 0.988 / 0.884                                              | 0.994 / 0.907                                              | 0.988 / 0.884                                                 | 0.970 / 0.884                                              |
| **Precisión**                      | 1.000 / 0.975               | 1.000 / 0.950                                              | 1.000 / 0.951                                              | 0.994 / 0.950                                                 | 1.000 / 0.974                                              |
| **AUC-ROC**                        | 1.000 / 0.992               | 1.000 / 0.989                                              | 1.000 / 0.991                                              | 1.000 / 0.990                                                 | 0.999 / 0.985                                              |
| **Precision-Recall AUC**           | 1.000 / 0.988               | 1.000 / 0.985                                              | 1.000 / 0.986                                              | 1.000 / 0.986                                                 | 0.999 / 0.983                                              |

#### Conclusiones para cada modelo:
* **Sin Hiperparámetros**: Este modelo muestra una precisión perfecta en los datos de entrenamiento (1.000), pero en los datos de prueba tiene una ligera caída en la precisión y en el F1 score, lo que indica que el modelo podría estar sobreajustando los datos de entrenamiento (Exactitud: 0.956, F1 score: 0.940). Aunque tiene buenos valores de AUC-ROC (0.992) y Precision-Recall AUC (0.988), la diferencia entre los datos de entrenamiento y prueba sugiere que necesita ajustes.

* **n_estimators = 100, max_depth = 6, min_samples_split = 4**: Este modelo tiene un buen rendimiento, con una ligera mejora en la exactitud y F1 score en comparación con el modelo sin hiperparámetros en los datos de prueba (Exactitud: 0.939, F1 score: 0.916). Aunque sigue mostrando algo de sobreajuste, con una precisión perfecta en los datos de entrenamiento, es un modelo sólido que comienza a mejorar el rendimiento en los datos de prueba.

* **n_estimators = 150, max_depth = 10, min_samples_split = 6**: Este modelo mejora aún más, mostrando una exactitud de 0.947 y un F1 score de 0.929 en los datos de prueba. Con una precisión de 1.000 en los datos de entrenamiento y buenas métricas de AUC-ROC y Precision-Recall AUC, el modelo sigue mostrando algo de sobreajuste, pero con menos pronunciamiento que los modelos anteriores. Tiene un buen equilibrio entre entrenamiento y prueba.

* **n_estimators = 200, max_depth = None, min_samples_split = 8**: Este es uno de los mejores modelos, con un rendimiento sólido tanto en los datos de entrenamiento como en los de prueba (Exactitud: 0.939, F1 score: 0.916). Aunque la precisión es un poco más baja en los datos de prueba en comparación con los modelos anteriores, sigue manteniendo un buen rendimiento general y es el modelo con el AUC-ROC más alto (0.990). Este modelo parece ser el más equilibrado, con una capacidad sólida para generalizar.

* **n_estimators = 70, max_depth = 4, min_samples_split = 3**: Este modelo presenta el peor rendimiento en comparación con los demás, con una exactitud de 0.947 en los datos de prueba y un F1 score de 0.927. Sin embargo, sigue siendo un modelo competitivo, pero muestra señales claras de underfitting, con una menor capacidad para capturar la complejidad de los datos en comparación con otros modelos.

#### El modelo con **n_estimators = 150, max_depth = 10, min_samples_split = 6** parece ser el mejor. Ofrece un buen equilibrio entre los datos de entrenamiento y los de prueba, con un F1 score de 0.929 y una exactitud de 0.947 en los datos de prueba. Además, mantiene una precisión de 1.000 en los datos de entrenamiento y tiene buenos valores de AUC-ROC (0.991) y Precision-Recall AUC (0.986). Este modelo proporciona un rendimiento robusto sin un sobreajuste significativo, lo que lo convierte en una excelente opción para este conjunto de datos.

***

## `4.5 k-Vecinos más Cercanos (k-NN)`

### al igual que en el algoritmo anterior, vamos a tener para aplicar a cada dataset:
### para clasificación:
* DecisionTreeClassifier()  y métricas (Exactitud, F1 Score, Sensibilidad y Precisión) y añadidas (AUC-ROC y Precision-Recall AUC)
### para regresión:
* DecisionTreeRegressor() y métricas (MAE, MSE, R2 Score) y añadidas (RMSE y Explained Variance)

In [64]:
# Función para entrenar y evaluar un modelo de K-Nearest Neighbors (KNN)
def aplicar_knn_hiper(df, target_column, tipo_modelo='regresion', n_neighbors=5, weights='uniform', metric='euclidean'):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Dividir el dataset en entrenamiento (80%) y prueba (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Seleccionar el modelo dependiendo del tipo (regresión o clasificación)
    if tipo_modelo == 'regresion':
        model = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, metric=metric)  # Ajustado con los hiperparámetros
    elif tipo_modelo == 'clasificacion':
        model = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric)  # Ajustado con los hiperparámetros

    # Entrenar el modelo con los datos de entrenamiento
    model.fit(X_train, y_train)
    
    # Hacer predicciones sobre los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluar el modelo con las métricas correspondientes
    if tipo_modelo == 'regresion':
        # Métricas existentes
        mae_train = mean_absolute_error(y_train, y_train_pred)
        mae_test = mean_absolute_error(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "MAE", mae_train, mae_test)
        
        mse_train = mean_squared_error(y_train, y_train_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "MSE", mse_train, mse_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "R2 Score", r2_train, r2_test)
        
        # Nuevas métricas
        rmse_train, evs_train = metricas_regresion_adicionales(y_train, y_train_pred)
        rmse_test, evs_test = metricas_regresion_adicionales(y_test, y_test_pred)
        mostrarMetricasRegresion("KNN", "RMSE", rmse_train, rmse_test)
        mostrarMetricasRegresion("KNN", "Explained Variance", evs_train, evs_test)
    
    elif tipo_modelo == 'clasificacion':
        # Métricas existentes
        exactitud_train = metrics.accuracy_score(y_train, y_train_pred)
        exactitud_test = metrics.accuracy_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("KNN", "Exactitud", exactitud_train, exactitud_test)
        
        f1_train = metrics.f1_score(y_train, y_train_pred, average='binary')  # Cambiar a 'micro' o 'macro' si es multiclase
        f1_test = metrics.f1_score(y_test, y_test_pred, average='binary')
        mostrarMetricasClasificacion("KNN", "F1 Score", f1_train, f1_test)
        
        sensibilidad_train = metrics.recall_score(y_train, y_train_pred)
        sensibilidad_test = metrics.recall_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("KNN", "Sensibilidad", sensibilidad_train, sensibilidad_test)
        
        precision_train = metrics.precision_score(y_train, y_train_pred)
        precision_test = metrics.precision_score(y_test, y_test_pred)
        mostrarMetricasClasificacion("KNN", "Precisión", precision_train, precision_test)
        
        # Probabilidades para AUC-ROC y Precision-Recall AUC
        y_train_prob = model.predict_proba(X_train)[:, 1] if hasattr(model, 'predict_proba') else None
        y_test_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
        
        # Nuevas métricas
        auc_roc_train, pr_auc_train = metricas_clasificacion_adicionales(y_train, y_train_pred, y_train_prob)
        auc_roc_test, pr_auc_test = metricas_clasificacion_adicionales(y_test, y_test_pred, y_test_prob)
        mostrarMetricasClasificacion("KNN", "AUC-ROC", auc_roc_train, auc_roc_test)
        mostrarMetricasClasificacion("KNN", "Precision-Recall AUC", pr_auc_train, pr_auc_test)
    
    return model

#### Usaremos estos hiperparámetros:
* **n_neighbors:** Este parámetro controla la cantidad de vecinos más cercanos que el modelo utiliza al hacer una predicción. Si el número de vecinos es bajo, el modelo se ajusta demasiado a los datos de entrenamiento, lo que puede generar un sobreajuste. En cambio, si usamos un número alto de vecinos, el modelo tiende a ser más general, lo que podría hacer que no capture bien los patrones del conjunto de datos, resultando en un modelo demasiado sencillo.
* **weights:**  Este parámetro determina el peso que se le asigna a cada vecino en el proceso de predicción. Si seleccionamos 'uniform', todos los vecinos tienen el mismo peso, lo que significa que todos contribuyen por igual a la predicción. Si seleccionamos 'distance', los vecinos más cercanos tendrán mayor peso, dado que los puntos más cercanos a la predicción son los más relevantes.
* **metric:** Este parámetro define la medida que usamos para calcular la distancia entre puntos. De manera predeterminada, usamos la distancia Euclidiana, que es la más común, pero podemos elegir otras métricas como Manhattan o Minkowski dependiendo de la naturaleza de nuestros datos y lo que queramos analizar. Cada una de estas métricas tiene características diferentes y se adapta mejor a distintos tipos de distribuciones de datos.

### `4.5.1 k-Vecinos más Cercanos Aplicado a TEMPERATURA ESPAÑA` (con hiperparámetros)

In [65]:
# Ejemplo de uso para un modelo de regresión con KNN:

print("Sin hiperparámetros")
KVecinos_modelesp = aplicar_knn(dfesp_limpio, 'rain_days', tipo_modelo='regresion')

print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("n_neighbors = 5, weights = 'uniform', metric = 'euclidean'")
KVecinos_hiper_esp = aplicar_knn_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', n_neighbors=5, weights='uniform', metric='euclidean')

print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("n_neighbors = 7, weights = 'uniform', metric = 'manhattan'")
KVecinos_hiper_esp2 = aplicar_knn_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', n_neighbors=7, weights='uniform', metric='manhattan')

print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("n_neighbors = 10, weights = 'uniform', metric = 'minkowski'")
KVecinos_hiper_esp3 = aplicar_knn_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', n_neighbors=10, weights='uniform', metric='minkowski')

print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("n_neighbors = 3, weights = 'uniform', metric = 'euclidean'")
KVecinos_hiper_esp4 = aplicar_knn_hiper(dfesp_limpio, 'rain_days', tipo_modelo='regresion', n_neighbors=3, weights='uniform', metric='euclidean')

Sin hiperparámetros
KNN : MAE en los datos de entrenamiento: 0.713
KNN : MAE en los datos de testeo: 0.883

KNN : MSE en los datos de entrenamiento: 0.839
KNN : MSE en los datos de testeo: 1.219

KNN : R2 Score en los datos de entrenamiento: 0.912
KNN : R2 Score en los datos de testeo: 0.867

KNN : RMSE en los datos de entrenamiento: 0.916
KNN : RMSE en los datos de testeo: 1.104

KNN : Explained Variance en los datos de entrenamiento: 0.912
KNN : Explained Variance en los datos de testeo: 0.867

---------------------------------------------------------------------------
PRIMER CASO
n_neighbors = 5, weights = 'uniform', metric = 'euclidean'
KNN : MAE en los datos de entrenamiento: 0.713
KNN : MAE en los datos de testeo: 0.883

KNN : MSE en los datos de entrenamiento: 0.839
KNN : MSE en los datos de testeo: 1.219

KNN : R2 Score en los datos de entrenamiento: 0.912
KNN : R2 Score en los datos de testeo: 0.867

KNN : RMSE en los datos de entrenamiento: 0.916
KNN : RMSE en los datos de te

| **Métrica**                       | **Sin Hiperparámetros**    | **n_neighbors = 5, weights = 'uniform', metric = 'euclidean'** | **n_neighbors = 7, weights = 'uniform', metric = 'manhattan'** | **n_neighbors = 10, weights = 'uniform', metric = 'minkowski'** | **n_neighbors = 3, weights = 'uniform', metric = 'euclidean'** |
|------------------------------------|----------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|
| **MAE**                            | 0.713 / 0.883              | 0.713 / 0.883                                                   | **0.718 / 0.903**                                                   | 0.839 / 1.108            | 0.874 / 1.156                                                   |
| **MSE**                            | 0.839 / 1.219              | 0.839 / 1.219                                                   | **0.827 / 1.418**                                                   | 1.134 / 1.936            | 1.329 / 2.059                                                   |
| **R2 Score**                       | 0.912 / 0.867              | 0.912 / 0.867                                                   | **0.913 / 0.845**                                                   | 0.881 / 0.788            | 0.861 / 0.775                                                   |
| **RMSE**                           | 0.916 / 1.104              | 0.916 / 1.104                                                   | **0.909 / 1.191**                                                   | 1.065 / 1.391            | 1.153 / 1.435                                                   |
| **Explained Variance**             | 0.912 / 0.867              | 0.912 / 0.867                                                   | **0.915 / 0.847**                                                   | 0.884 / 0.795            | 0.861 / 0.783                                                   |

#### Conclusiones de cada modelo KNN:
* **Sin Hiperparámetros**: El modelo de KNN muestra un buen rendimiento en los datos de entrenamiento, pero sufre de sobreajuste cuando se evalúa en los datos de prueba. El MAE y MSE son bajos en entrenamiento, pero se incrementan significativamente en el conjunto de prueba, lo que indica que el modelo podría no estar generalizando bien.
* **n_neighbors = 5, weights = 'uniform', metric = 'euclidean'**: Los resultados no presentan grandes mejoras respecto al modelo sin hiperparámetros. Aunque el rendimiento en los datos de prueba es ligeramente mejor, sigue existiendo un cierto grado de sobreajuste, con un R2 en prueba de 0.867 y un MAE de 0.883. Esto sugiere que el modelo aún tiene dificultades para generalizar.
* **n_neighbors = 7, weights = 'uniform', metric = 'manhattan'**:  En este caso, el modelo muestra un rendimiento decente, aunque no es perfecto. El MAE y MSE en entrenamiento son ligeramente superiores a los de los modelos anteriores, pero en los datos de prueba, el R2 sigue siendo razonable con un valor de 0.845. Este modelo ha logrado un mejor ajuste en los datos de prueba en comparación con otros modelos, lo que indica que ha generalizado de una manera más efectiva.

* **n_neighbors = 10, weights = 'uniform', metric = 'minkowski'**: Este modelo tiene peores resultados que los anteriores, tanto en entrenamiento como en prueba. El MAE en prueba es de 1.108 y el MSE es de 1.936, lo que indica un rendimiento más pobre en ambos conjuntos de datos. Este modelo parece no ser adecuado, ya que tiene un R2 de 0.788 en prueba, lo que indica que no está capturando bien la relación entre las variables.
* **n_neighbors = 3, weights = 'uniform', metric = 'euclidean'**: Similar al modelo con n_neighbors = 7, este modelo tiene un MAE y MSE cercanos a cero en entrenamiento, lo que sugiere sobreajuste. Sin embargo, los resultados en el conjunto de prueba son peores, con un R2 de 0.775 y un MAE de 1.156, lo que indica que el modelo no ha generalizado bien.

#### El modelo con **n_neighbors = 7, weights = 'uniform', metric = 'manhattan'** parece ser el más prometedor, ya que ha logrado un equilibrio razonable entre los datos de entrenamiento y prueba, aunque sigue mostrando un leve sobreajuste. Sin embargo, otros modelos con 3 y 5 vecinos también lograron buenos resultados en los datos de prueba, pero con menos capacidad de generalización. Aunque ninguno de los modelos se aproxima a un rendimiento perfecto, el modelo con 7 vecinos presenta una combinación bastante equilibrada entre la capacidad de ajuste y la generalización.

### `4.5.2 k-Vecinos más Cercanos Aplicado a PHISHING URLS` (con hiperparámetros)

In [66]:
# Aplicar k-Vecinos más Cercanos al dataset dfphishing_limpio para clasificación
KVecinos_modelphishing = aplicar_knn(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

# Ejemplo de uso para un modelo de clasificación con KNN:

print("Sin hiperparámetros")
KVecinos_modelphishing_sin_hiper = aplicar_knn(dfphishing_limpio, 'class', tipo_modelo='clasificacion')

print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("n_neighbors = 5, weights = 'uniform', metric = 'euclidean'")
KVecinos_hiper_1 = aplicar_knn_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', n_neighbors=5, weights='uniform', metric='euclidean')

print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("n_neighbors = 10, weights = 'distance', metric = 'manhattan'")
KVecinos_hiper_2 = aplicar_knn_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', n_neighbors=10, weights='distance', metric='manhattan')

print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("n_neighbors = 7, weights = 'uniform', metric = 'minkowski'")
KVecinos_hiper_3 = aplicar_knn_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', n_neighbors=7, weights='uniform', metric='minkowski')

print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("n_neighbors = 15, weights = 'distance', metric = 'euclidean'")
KVecinos_hiper_4 = aplicar_knn_hiper(dfphishing_limpio, 'class', tipo_modelo='clasificacion', n_neighbors=15, weights='distance', metric='euclidean')

KNN : Exactitud en los datos de entrenamiento: 0.964
KNN : Exactitud en los datos de testeo: 0.940

KNN : F1 Score en los datos de entrenamiento: 0.968
KNN : F1 Score en los datos de testeo: 0.946

KNN : Sensibilidad en los datos de entrenamiento: 0.971
KNN : Sensibilidad en los datos de testeo: 0.950

KNN : Precisión en los datos de entrenamiento: 0.965
KNN : Precisión en los datos de testeo: 0.943

KNN : AUC-ROC en los datos de entrenamiento: 0.995
KNN : AUC-ROC en los datos de testeo: 0.982

KNN : Precision-Recall AUC en los datos de entrenamiento: 0.994
KNN : Precision-Recall AUC en los datos de testeo: 0.978

Sin hiperparámetros
KNN : Exactitud en los datos de entrenamiento: 0.964
KNN : Exactitud en los datos de testeo: 0.940

KNN : F1 Score en los datos de entrenamiento: 0.968
KNN : F1 Score en los datos de testeo: 0.946

KNN : Sensibilidad en los datos de entrenamiento: 0.971
KNN : Sensibilidad en los datos de testeo: 0.950

KNN : Precisión en los datos de entrenamiento: 0.965
K

| **Métrica**                       | **Sin Hiperparámetros**  | **n_neighbors = 5, weights = 'uniform', metric = 'euclidean'** | **n_neighbors = 10, weights = 'distance', metric = 'manhattan'** | **n_neighbors = 7, weights = 'uniform', metric = 'minkowski'** | **n_neighbors = 15, weights = 'distance', metric = 'euclidean'** |
|------------------------------------|--------------------------|---------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|
| **Exactitud**                      | 0.964 / 0.940            | 0.964 / 0.940                                                 | **0.991 / 0.965**             | 0.960 / 0.936    | 0.991 / 0.957           |
| **F1 Score**                       | 0.968 / 0.946            | 0.968 / 0.946                                                 | **0.992 / 0.969**             | 0.964 / 0.943    | 0.992 / 0.961           | 
| **Sensibilidad**                   | 0.971 / 0.950            | 0.971 / 0.950                                                 | **0.991 / 0.975**             | 0.964 / 0.942    | 0.991 / 0.967           | 
| **Precisión**                      | 0.965 / 0.943            | 0.965 / 0.943                                                 | **0.993 / 0.962**             | 0.963 / 0.944    | 0.993 / 0.956           | 
| **AUC-ROC**                        | 0.995 / 0.982            | 0.995 / 0.982                                                 | **1.000 / 0.989**             | 0.994 / 0.984    | 1.000 / 0.987           | 
| **Precision-Recall AUC**           | 0.994 / 0.978            | 0.994 / 0.978                                                 | **1.000 / 0.986**             | 0.993 / 0.982    | 1.000 / 0.985           |


#### Conclusiones de cada modelo:
* **Sin Hiperparámetros**: El modelo muestra buen rendimiento en entrenamiento (exactitud 0.964, F1 0.968), pero con ligera caída en prueba (exactitud 0.940, F1 0.946), sugiriendo sobreajuste.
* **n_neighbors = 5, weights = 'uniform', metric = 'euclidean'**: Resultados similares al modelo sin hiperparámetros. El modelo sigue mostrando sobreajuste sin mejoras significativas.
* **n_neighbors = 10, weights = 'distance', metric = 'manhattan'**: Este modelo mejora significativamente en los datos de prueba, con una exactitud de 0.991 en entrenamiento y 0.965 en prueba, además de un F1 Score de 0.992 en entrenamiento y 0.969 en prueba. Muestra un mejor equilibrio entre entrenamiento y prueba, con un leve sobreajuste en entrenamiento que es más reducido que en los modelos anteriores.
* **n_neighbors = 7, weights = 'uniform', metric = 'minkowski'**: El rendimiento de este modelo es similar al anterior, con una exactitud de 0.960 en entrenamiento y 0.936 en prueba. Si bien el modelo tiene un desempeño decente, la caída en exactitud y F1 Score en los datos de prueba indica que no generaliza tan bien como los otros modelos con más vecinos.
* **n_neighbors = 15, weights = 'distance', metric = 'euclidean'**: Este modelo destaca por su rendimiento equilibrado, con una exactitud de 0.991 en entrenamiento y 0.957 en prueba, y un F1 Score de 0.992 en entrenamiento y 0.961 en prueba. Logra un buen equilibrio entre ajuste y generalización, siendo uno de los mejores modelos en términos de rendimiento de prueba.
rendimiento en la prueba.
#### El modelo con **n_neighbors = 10, weights = 'distance', metric = 'manhattan'** destaca como el más prometedor, ya que ofrece el mejor equilibrio entre los datos de entrenamiento y prueba, con una mejora significativa en comparación con los modelos anteriores, y un bajo grado de sobreajuste.

### `4.5.3 k-Vecinos más Cercanos Aplicado a CALIDAD DEL VINO` (con hiperparámetros)

In [67]:
# Aplicar k-Vecinos más Cercanos al dataset dfvino_limpio
KVecinos_modelvino = aplicar_knn(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

# Ejemplo de uso para un modelo de clasificación con KNN:

print("Sin hiperparámetros")
KVecinos_modelvino_sin_hiper = aplicar_knn(dfvino_limpio, 'quality', tipo_modelo='clasificacion')

print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("n_neighbors = 5, weights = 'uniform', metric = 'euclidean'")
KVecinos_hiper_1 = aplicar_knn_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', n_neighbors=5, weights='uniform', metric='euclidean')

print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("n_neighbors = 10, weights = 'distance', metric = 'manhattan'")
KVecinos_hiper_2 = aplicar_knn_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', n_neighbors=10, weights='distance', metric='manhattan')

print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("n_neighbors = 7, weights = 'uniform', metric = 'minkowski'")
KVecinos_hiper_3 = aplicar_knn_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', n_neighbors=7, weights='uniform', metric='minkowski')

print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("n_neighbors = 15, weights = 'distance', metric = 'euclidean'")
KVecinos_hiper_4 = aplicar_knn_hiper(dfvino_limpio, 'quality', tipo_modelo='clasificacion', n_neighbors=15, weights='distance', metric='euclidean')

KNN : Exactitud en los datos de entrenamiento: 0.793
KNN : Exactitud en los datos de testeo: 0.729

KNN : F1 Score en los datos de entrenamiento: 0.813
KNN : F1 Score en los datos de testeo: 0.762

KNN : Sensibilidad en los datos de entrenamiento: 0.832
KNN : Sensibilidad en los datos de testeo: 0.780

KNN : Precisión en los datos de entrenamiento: 0.795
KNN : Precisión en los datos de testeo: 0.744

KNN : AUC-ROC en los datos de entrenamiento: 0.888
KNN : AUC-ROC en los datos de testeo: 0.806

KNN : Precision-Recall AUC en los datos de entrenamiento: 0.884
KNN : Precision-Recall AUC en los datos de testeo: 0.817

Sin hiperparámetros
KNN : Exactitud en los datos de entrenamiento: 0.793
KNN : Exactitud en los datos de testeo: 0.729

KNN : F1 Score en los datos de entrenamiento: 0.813
KNN : F1 Score en los datos de testeo: 0.762

KNN : Sensibilidad en los datos de entrenamiento: 0.832
KNN : Sensibilidad en los datos de testeo: 0.780

KNN : Precisión en los datos de entrenamiento: 0.795
K

| **Métrica**                       | **Sin Hiperparámetros**  | **n_neighbors = 5, weights = 'uniform', metric = 'euclidean'** | **n_neighbors = 10, weights = 'distance', metric = 'manhattan'** | **n_neighbors = 7, weights = 'uniform', metric = 'minkowski'** | **n_neighbors = 15, weights = 'distance', metric = 'euclidean'** |
|------------------------------------|--------------------------|---------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|
| **Exactitud**                      | 0.793 / 0.729            | 0.793 / 0.729                                                 | 1.000 / 0.764             | 0.795 / 0.738    | **1.000 / 0.790**           |
| **F1 Score**                       | 0.813 / 0.762            | 0.813 / 0.762                                                 | 1.000 / 0.792             | 0.818 / 0.767    | **1.000 / 0.815**           | 
| **Sensibilidad**                   | 0.832 / 0.780            | 0.832 / 0.780                                                 | 1.000 / 0.811             | 0.848 / 0.780    | **1.000 / 0.835**           | 
| **Precisión**                      | 0.795 / 0.744            | 0.795 / 0.744                                                 | 1.000 / 0.774             | 0.789 / 0.756    | **1.000 / 0.797**           | 
| **AUC-ROC**                        | 0.888 / 0.806            | 0.888 / 0.806                                                 | 1.000 / 0.872             | 0.877 / 0.815    | **1.000 / 0.888**           | 
| **Precision-Recall AUC**           | 0.884 / 0.817            | 0.884 / 0.817                                                 | 1.000 / 0.910             | 0.874 / 0.826    | **1.000 / 0.920**           |


#### Conclusiones por modelo:
* **Sin Hiperparámetros:** El modelo sin hiperparámetros tiene un desempeño decente en cuanto a exactitud y F1 Score, pero en general muestra que aún puede mejorarse al ajustar ciertos parámetros como el número de vecinos, el peso y la métrica de distancia.
* **n_neighbors = 5, weights = 'uniform', metric = 'euclidean':** Este modelo no muestra mejoras significativas en comparación con el modelo sin hiperparámetros. Los resultados de exactitud, F1 Score, sensibilidad, y precisión son similares. Esto indica que la elección de 5 vecinos y la métrica Euclidiana no optimiza mucho el rendimiento.
* **n_neighbors = 10, weights = 'distance', metric = 'manhattan':** Este modelo muestra una exactitud de 1.000 en los datos de entrenamiento, pero una baja exactitud en los datos de testeo (0.764). Sin embargo, el F1 Score y otros parámetros como sensibilidad y precisión en los datos de testeo mejoran en comparación con el primer caso. La métrica Manhattan y el peso 'distance' parecen generar buenos resultados en el entrenamiento, pero sobreajustando el modelo a los datos de entrenamiento.
* **n_neighbors = 7, weights = 'uniform', metric = 'minkowski':** Este modelo muestra un buen equilibrio entre exactitud en los datos de testeo (0.738) y exactitud en entrenamiento (0.795). El F1 Score también es bastante bueno (0.818 en entrenamiento y 0.767 en testeo), lo que indica que este modelo tiene un rendimiento más robusto y menos propenso al sobreajuste.
* **n_neighbors = 15, weights = 'distance', metric = 'euclidean':** Y este último también se comporta igual en los datos de entrenamiento que el anterior, con exactitud y F1 Score de 1.000. En los datos de testeo, se obtiene una mejora significativa en comparación con los otros modelos, especialmente en sensibilidad (0.835) y precision-recall AUC (0.920). La elección de n_neighbors = 15 y el uso del 'distance' como peso junto con la métrica 'euclidean' muestra el mejor equilibrio entre ajuste y generalización.
#### Escogería el modelo **n_neighbors = 15, weights = 'distance', metric = 'euclidean'** porque tiene un rendimiento excelente en los datos de testeo con una exactitud de 0.790 y el mejor valor de AUC-ROC (0.888). Aunque n_neighbors = 15 podría hacer que el modelo sea más general, el uso de weights = 'distance' y metric = 'euclidean' permite que los vecinos cercanos tengan un mayor peso, lo que mejora la precisión sin perder capacidad de generalización. Esta combinación ayuda a evitar el sobreajuste, manteniendo un buen equilibrio entre precisión y generalización.

### `3.5.4 k-Vecinos más Cercanos Aplicado a DIAGNOSTICO DE CANCER DE MAMA` (sin hiperparámetros)

In [68]:
# Aplicar k-Vecinos más Cercanos al dataset dfcancer_limpio
KVecinos_modelcancer = aplicar_knn(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

# Ejemplo de uso para un modelo de clasificación con KNN:

print("Sin hiperparámetros")
KVecinos_modelcancer_sin_hiper = aplicar_knn(dfcancer_limpio, 'y', tipo_modelo='clasificacion')

print("---------------------------------------------------------------------------")
print("PRIMER CASO")
print("n_neighbors = 5, weights = 'uniform', metric = 'euclidean'")
KVecinos_hiper_1 = aplicar_knn_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', n_neighbors=5, weights='uniform', metric='euclidean')

print("---------------------------------------------------------------------------")
print("SEGUNDO CASO")
print("n_neighbors = 10, weights = 'distance', metric = 'manhattan'")
KVecinos_hiper_2 = aplicar_knn_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', n_neighbors=10, weights='distance', metric='manhattan')

print("---------------------------------------------------------------------------")
print("TERCER CASO")
print("n_neighbors = 7, weights = 'uniform', metric = 'minkowski'")
KVecinos_hiper_3 = aplicar_knn_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', n_neighbors=7, weights='uniform', metric='minkowski')

print("---------------------------------------------------------------------------")
print("CUARTO CASO")
print("n_neighbors = 15, weights = 'distance', metric = 'euclidean'")
KVecinos_hiper_4 = aplicar_knn_hiper(dfcancer_limpio, 'y', tipo_modelo='clasificacion', n_neighbors=15, weights='distance', metric='euclidean')

KNN : Exactitud en los datos de entrenamiento: 0.978
KNN : Exactitud en los datos de testeo: 0.930

KNN : F1 Score en los datos de entrenamiento: 0.970
KNN : F1 Score en los datos de testeo: 0.900

KNN : Sensibilidad en los datos de entrenamiento: 0.953
KNN : Sensibilidad en los datos de testeo: 0.837

KNN : Precisión en los datos de entrenamiento: 0.988
KNN : Precisión en los datos de testeo: 0.973

KNN : AUC-ROC en los datos de entrenamiento: 0.998
KNN : AUC-ROC en los datos de testeo: 0.983

KNN : Precision-Recall AUC en los datos de entrenamiento: 0.995
KNN : Precision-Recall AUC en los datos de testeo: 0.976

Sin hiperparámetros
KNN : Exactitud en los datos de entrenamiento: 0.978
KNN : Exactitud en los datos de testeo: 0.930

KNN : F1 Score en los datos de entrenamiento: 0.970
KNN : F1 Score en los datos de testeo: 0.900

KNN : Sensibilidad en los datos de entrenamiento: 0.953
KNN : Sensibilidad en los datos de testeo: 0.837

KNN : Precisión en los datos de entrenamiento: 0.988
K

| **Métrica**                       | **Sin Hiperparámetros**  | **n_neighbors = 5, weights = 'uniform', metric = 'euclidean'** | **n_neighbors = 10, weights = 'distance', metric = 'manhattan'** | **n_neighbors = 7, weights = 'uniform', metric = 'minkowski'** | **n_neighbors = 15, weights = 'distance', metric = 'euclidean'** |
|------------------------------------|--------------------------|---------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|-----------------------------------------------------------------|
| **Exactitud**                      | 0.978 / 0.930            | 0.978 / 0.930                                                 | 1.000 / 0.939             | 0.978 / 0.939    | **1.000 / 0.947**           |
| **F1 Score**                       | 0.970 / 0.900            | 0.970 / 0.900                                                 | 1.000 / 0.914             | 0.970 / 0.911    | **1.000 / 0.925**           | 
| **Sensibilidad**                   | 0.953 / 0.837            | 0.953 / 0.837                                                 | 1.000 / 0.860             | 0.953 / 0.837    | **1.000 / 0.860**           | 
| **Precisión**                      | 0.988 / 0.973            | 0.988 / 0.973                                                 | 1.000 / 0.974             | 0.988 / 1.000    | **1.000 / 1.000**           | 
| **AUC-ROC**                        | 0.998 / 0.983            | 0.998 / 0.983                                                 | 1.000 / 0.996             | 0.997 / 0.981    | **1.000 / 0.994**           | 
| **Precision-Recall AUC**           | 0.995 / 0.976            | 0.995 / 0.976                                                 | 1.000 / 0.994             | 0.994 / 0.971    | **1.000 / 0.991**           |


#### Conclusiones por modelo:
* **Sin Hiperparámetros:** El modelo sin hiperparámetros muestra una exactitud y F1 Score sólidos, tanto en los datos de entrenamiento como en los de testeo. Sin embargo, el modelo todavía tiene margen para optimizarse, ya que no se han ajustado los parámetros que podrían mejorar la capacidad de generalización y evitar el sobreajuste.

* **n_neighbors = 5, weights = 'uniform', metric = 'euclidean':** Este modelo mantiene un rendimiento muy similar al del modelo sin hiperparámetros. La exactitud y el F1 Score siguen siendo buenos, pero no se observa una mejora considerable en comparación con el modelo base. Esto sugiere que la configuración de 5 vecinos y la métrica Euclidiana no es la opción óptima para este conjunto de datos.

* **n_neighbors = 10, weights = 'distance', metric = 'manhattan':** Este modelo muestra una mejora significativa en los datos de entrenamiento (con una exactitud de 1.000), pero también un sobreajuste evidente, ya que la exactitud de testeo es algo más baja (0.939). Sin embargo, el F1 Score y otros parámetros como la sensibilidad y precisión en los datos de testeo mejoran, lo que indica que la combinación de 10 vecinos, el peso 'distance' y la métrica Manhattan está proporcionando buenos resultados, pero es necesario ajustar para evitar el sobreajuste.

* **n_neighbors = 7, weights = 'uniform', metric = 'minkowski':** Este modelo tiene un buen rendimiento tanto en los datos de entrenamiento como en los de testeo. La exactitud en testeo es de 0.939, lo que es bastante alto, y el F1 Score también se mantiene sólido (0.911 en testeo). Esto indica que la configuración con 7 vecinos y la métrica Minkowski da un buen equilibrio entre precisión y generalización.

* **n_neighbors = 15, weights = 'distance', metric = 'euclidean':** Este modelo presenta un rendimiento impresionante en los datos de entrenamiento, con una exactitud de 1.000, pero también sobresale en los datos de testeo, alcanzando una exactitud de 0.947. Además, el F1 Score y la sensibilidad también se mantienen altos. Este modelo, con 15 vecinos y el uso de 'distance' como peso, muestra un excelente balance entre ajuste a los datos de entrenamiento y generalización en los datos de testeo.

#### Escogería el modelo **n_neighbors = 15, weights = 'distance', metric = 'euclidean'** debido a su excelente rendimiento tanto en los datos de entrenamiento como en los de testeo. Con una exactitud de 0.947 en testeo y un F1 Score impresionante, este modelo muestra un buen balance entre precisión y generalización. Aunque el número de vecinos más alto podría sugerir una mayor suavidad en el modelo, la combinación con el peso 'distance' y la métrica 'euclidean' mejora la precisión sin caer en el sobreajuste, lo que lo convierte en la mejor opción para este conjunto de datos.

***

## 4.6 Conclusiones de las mejores métricas obtenidas con hiperparámetros
### Y las vamos a comparar con las de sin hiperparámetros

**SIN HIPERPARÁMETROS**
| Algoritmo                | Dataset             | Exactitud(Accuracy) | F1 Score | Sensibilidad(recall) | Precisión | AUC-ROC | Precision-Recall AUC | MAE   | MSE   | R2 Score | RMSE  | Explained Variance |
|--------------------------|---------------------|-----------|----------|---------------|-----------|---------|----------------------|-------|-------|----------|-------|--------------------|
| **SVM**                   | dfesp_limpio        | ----      | ----    | ----          | ----      | ----    | ----                  | 0.584 / 0.980   | 0.851 / 1.827 | 0.911 / 0.800 | 0.922 / 1.351 | 0.912 / 0.802   |
| **SVM**                   | dfphishing_limpio   | 0.953 / 0.951      | 0.958 / 0.957    | 0.969 / 0.973        | 0.947 / 0.941      | 0.991 / 0.989   | 0.993 / 0.991  | ---- |---- |---- |---- |---- |
| **SVM**                   | dfvino_limpio       | 0.802 / 0.764      | 0.814 / 0.791    | 0.804 / 0.803        | 0.825 / 0.779      | 0.878 / 0.874   | 0.892 / 0.898  | ---- |---- |---- |---- |---- |
| **SVM**                   | dfcancer_limpio     | 0.987 / 0.956      | 0.982 / 0.941    | 0.970 / 0.930        | 0.994 / 0.952      | 0.998 / 0.997   | 0.997 / 0.996  | ---- |---- |---- |---- |---- |
| **Regresión Logística**   | dfphishing_limpio   | 0.927 / 0.934      | 0.935 / 0.941    | 0.943 / 0.953        | 0.927 / 0.930      | 0.979 / 0.980   | 0.983 / 0.984  | ---- |---- |---- |---- |---- |
| **Regresión Logística**   | dfvino_limpio       | 0.758 / 0.769      | 0.774 / 0.791    | 0.767 / 0.787        | 0.781 / 0.794      | 0.829 / 0.822   | 0.853 / 0.835  | ---- |---- |---- |---- |---- |
| **Regresión Logística**   | dfcancer_limpio     | 0.991 / 0.965      | 0.988 / 0.953    | 0.976 / 0.953        | 1.000 / 0.953      | 0.998 / 0.995   | 0.997 / 0.993  | ---- |---- |---- |---- |---- |
| **Árboles de Decisión**   | dfesp_limpio        | ----      | ----     | ----          | ----      | ----      | ----               | 0.000 / 0.583   | 0.000 / 0.688 | 1.000 / 0.925 | 0.000 / 0.829 | 1.000 / 0.929   |
| **Árboles de Decisión**   | dfphishing_limpio   | 0.991 / 0.960      | 0.992 / 0.964    | 0.991 / 0.964        | 0.993 / 0.964      | 1.000 / 0.972   | 1.000 / 0.966                | ---- |---- |---- |---- |---- |
| **Árboles de Decisión**   | dfvino_limpio       | 1.000 / 0.694      | 1.000 / 0.724    | 1.000 / 0.724        | 1.000 / 0.724      | 1.000 / 0.691   | 1.000 / 0.678                | ---- |---- |---- |---- |---- |
| **Árboles de Decisión**   | dfcancer_limpio     | 1.000 / 0.921      | 1.000 / 0.897    | 1.000 / 0.907        | 1.000 / 0.886      | 1.000 / 0.918   | 1.000 / 0.839                | ---- |---- |---- |---- |---- |
| **Bosques Aleatorios**    | dfesp_limpio        | ----      | ----     | ----          | ----       ----      | ----      | 0.950   | 0.900                | 0.192 | 0.083 | 0.991    |
| **Bosques Aleatorios**    | dfphishing_limpio   | 0.969     | 0.973    | 0.977         | 0.969     | 0.995   | 0.995                | ---- |---- |---- |---- |---- |
| **Bosques Aleatorios**    | dfvino_limpio       | 0.769     | 0.789    | 0.780         | 0.798     | 0.886   | 0.915                | ---- |---- |---- |---- |---- |
| **Bosques Aleatorios**    | dfcancer_limpio     | 0.956     | 0.940    | 0.907         | 0.975     | 0.992   | 0.988                | ---- |---- |---- |---- |---- |
| **K-Vecinos más Cercanos**| dfesp_limpio        | ----      | ----     | ----          | ----      | ----    | ----                 | 0.713 | 0.839 | 0.912    | 0.916 | 0.912        |
| **K-Vecinos más Cercanos**| dfphishing_limpio   | 0.940     | 0.946    | 0.950         | 0.931     | 0.999   | 0.998                | ---- |---- |---- |---- |---- |
| **K-Vecinos más Cercanos**| dfvino_limpio       | 0.760     | 0.771    | 0.758         | 0.771     | 0.824   | 0.837                | ---- |---- |---- |---- |---- |
| **K-Vecinos más Cercanos**| dfcancer_limpio     | 0.955     | 0.939    | 0.941         | 0.929     | 0.991   | 0.989                | ---- |---- |---- |---- |---- |

| Algoritmo                | Dataset             | Exactitud(Accuracy) | F1 Score | Sensibilidad(recall) | Precisión | AUC-ROC | Precision-Recall AUC | MAE   | MSE   | R2 Score | RMSE  | Explained Variance |
|--------------------------|---------------------|-----------|----------|---------------|-----------|---------|----------------------|-------|-------|----------|-------|--------------------|
| **SVM C = 5, epsilon = 0.01, kernel = lineal** | dfesp_limpio        | ----     | ----   | ----        | ----     | ----       | ----     |0.517 / 0.632  | 0.526 / 0.653 | 0.945 / 0.929 | 0.725 / 0.808 | 0.945 / 0.930 |
| **SVM C = 5, kernel = rbf** | dfphishing_limpio   | 0.972 / 0.966     | 0.975 / 0.970   | 0.980 / 0.978 | 0.969 / 0.962  | 0.995 / 0.992   | 0.997 / 0.993  | ----       | ----       |  ----         |  ----      |     ----                |
| **SVM C = 5, kernel = rbf**  | dfvino_limpio      | 0.829 / 0.786     | 0.839 / 0.805   | 0.822 / 0.795 | 0.857 / 0.815  | 0.918 / 0.878   | 0.924 / 0.904  |   ----     |   ----     |    ----       |   ----     |     ----                |
| **SVM C = 2, kernel = lineal** | dfcancer_limpio  | 0.989 / 0.982     | 0.985 / 0.977   | 0.976 / 0.977 | 0.994 / 0.977  | 0.997 / 0.997   | 0.997 / 0.996  |  ----      | ----       |----           | ----       |    ----                 |
| **Regresión Logística solver = liblinear, max_iter = 100, penalty = l1**   | dfphishing_limpio     | 0.927 / 0.933     | 0.935 / 0.941    | 0.943 / 0.953         | 0.927 / 0.929     | 0.979 / 0.980   | 0.983 / 0.984|  ----      | ----       |----           | ----       |    ----   |
| **Regresión Logística solver = liblinear, max_iter = 200, penalty = l1**   | dfvino_limpio         | 0.761 / 0.769     | 0.777 / 0.791    | 0.769 / 0.787         | 0.785 / 0.794     | 0.829 / 0.824   | 0.853 / 0.836|  ----      | ----       |----           | ----       |    ----   |
| **Regresión Logística solver = saga, max_iter = 300, penalty = elasticnet**| dfcancer_limpio       | 0.927 / 0.934     | 0.935 / 0.941    | 0.944 / 0.953         | 0.927 / 0.930     | 0.979 / 0.980   | 0.983 / 0.984|  ----      | ----       |----           | ----       |    ----   |
| **Árboles de Decisión max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**   | dfesp_limpio        | ----     | ----    | ----         | ----     | ----   | ----         | 0.520 / 0.700    | 0.567 / 0.907 | 0.941 / 0.901    | 0.753 / 0.953 | 0.941 / 0.902              |
| **Árboles de Decisión max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**   | dfphishing_limpio     | 0.922 / 0.927     | 0.932 / 0.937    | 0.972 / 0.975          | 0.896 / 0.903 | 0.978 / 0.977   | 0.977 / 0.975  | ----       |----        |----           |----        |----   |
| **Árboles de Decisión max_depth = 5, min_samples_split = 4, min_samples_leaf = 2**   | dfvino_limpio         | 0.821 / 0.742     | 0.836 / 0.761    | 0.846 / 0.740          | 0.826 / 0.783 | 0.892 / 0.777   | 0.886 / 0.798  | ----       |----        |----           |----        |----   |
| **Árboles de Decisión**   | dfcancer_limpio       | 0.921     | 0.897    | 0.907         | 0.886     | 0.918   | 0.839                | ----       |----        |----           |----        |----                     |
| **Bosques Aleatorios**    | dfesp_limpio        | ----     | ----   | ----        | ----     | ----   | ----          | 0.192 | 0.083 | 0.991    | 0.975 | 0.991              |
| **Bosques Aleatorios**    | dfphishing_limpio     | 0.969     | 0.973    | 0.977         | 0.969     | 0.995   | 0.995                |  ----      |----        |----           |----        |----                     |
| **Bosques Aleatorios**    | dfvino_limpio         | 0.769     | 0.789    | 0.780         | 0.798     | 0.886   | 0.915                |  ----      |----        |----           |----        |----                     |
| **Bosques Aleatorios**    | dfcancer_limpio       | 0.956     | 0.940    | 0.907         | 0.975     | 0.992   | 0.988                |  ----      |----        |----           |----        |----                     |
| **K-Vecinos más Cercanos**| dfesp_limpio        | ----     | ----   | ----         | ----     | ----   | ----         | 0.713 | 0.839 | 0.912    | 0.916 | 0.912              |
| **K-Vecinos más Cercanos**| dfphishing_limpio     | 0.940     | 0.946    | 0.950         | 0.931     | 0.999   | 0.998                |----        |----        |----           |----        |                    ---- |
| **K-Vecinos más Cercanos**| dfvino_limpio         | 0.760     | 0.771    | 0.758         | 0.771     | 0.824   | 0.837                |----        |----        |----           |----        |                    ---- |
| **K-Vecinos más Cercanos**| dfcancer_limpio       | 0.955     | 0.939    | 0.941         | 0.929     | 0.991   | 0.989                |----        |----        |----           |----        |                    ---- |
